In [1]:
# =========================================================
# ✅ TARGET-ONLY OUTPUT + (1st gene/expr token drop) + (NO SMILES MSE)
#
# What changed vs your current version:
# 1) OUTPUT space is TARGET-ONLY (M_TGT = genes that ever appeared as targets, ~100)
#    - y_targets: (B, M_TGT)
#    - pos_weight: (M_TGT,)
#    - loss/eval gene bank = subset_gene_emb[target_sub_ids]  -> (M_TGT, d)
# 2) INPUT token space stays SUBSET = HVG ∪ TARGETS (context preserved)
# 3) Drop the first (genes[0], expressions[0]) as "service/non-real" token
# 4) Remove MSE from SMILES part:
#    - train smiles loss = SupCon + alpha_align * cosine_align
#    - validation metrics: only SMILES_SupCon, SMILES_COS (no SMILES_MSE)
# =========================================================

import os, glob, ast, random
from collections import defaultdict

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler

from tqdm import tqdm

import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split

import scanpy as sc
from scipy import sparse


In [2]:
# =========================================================
# 0) PATHS / HYPERPARAMS
# =========================================================
PARQUET_DIR    = "/data/aiffel/data/Tahoe-100M/data"
GENE_META_PATH = "/data/aiffel/data/Tahoe-100M/metadata/gene_metadata.parquet"
DRUG_META_PATH = "/data/aiffel/data/Tahoe-100M/metadata/drug_metadata.parquet"
COUNTS_CSV     = "/data/aiffel/babayakga/making_data/aiffel/babayakga/making_data/tahoe_counts_per_drug_cell_line.csv"
DMSO_PATH      = "/data/aiffel/babayakga/outputs/dmso.h5ad"

SMILES_EMB_PATH       = "/data/aiffel/babayakga/smiles_emb/drug_smiles_emb_all1.pt"
PRETRAINED_GENE_NPY   = "/data/aiffel/babayakga/pretraining/checkpoints_with_cell/gene_embeddings.npy"
CELL2ID_CSV           = "/data/aiffel/babayakga/pretraining/checkpoints_with_cell/cell2id.csv"
CELL_EMB_NPY          = "/data/aiffel/babayakga/pretraining/checkpoints_with_cell/cell_embeddings.npy"

CONTROL_DRUG = "DMSO_TF"
SEED = 42

MAX_SEQ_LEN = 256
BATCH_SIZE  = 128
STEPS_PER_EPOCH = 10000
VAL_STEPS       = 500

lambda_cos    = 1.0
lambda_bce    = 0.25
lambda_rank   = 0.75

lambda_smiles = 0.05
alpha_align   = 0.5
tau_smiles    = 0.07

rank_num_neg = 512
rank_num_pos = 8
bce_num_neg  = 2048
bce_pos_cap  = None

EPOCHS = 20
LR = 1e-5

POSW_MODE = "sqrt_clip"  # ["none", "clip", "sqrt_clip"]
POSW_MAX  = 50.0

NUM_WORKERS = 4
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

HVG_K = 4000
PAIRS_PER_BATCH = 16

# ✅ drop first "service" element from genes/expressions
DROP_FIRST_GENE_TOKEN = True

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ✅ TF32 safe speed-up on Ampere+
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass


# =========================================================
# (extra) checkpoint save
# =========================================================
CKPT_DIR = "/data/aiffel/babayakga/checkpoints/fp_target_only_no_mse_drop1"
SAVE_EVERY = 2
os.makedirs(CKPT_DIR, exist_ok=True)

def save_checkpoint(save_dir, model, optimizer, scaler, epoch, metrics=None, prefix="fp"):
    ckpt = {
        "epoch": int(epoch),
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scaler_state_dict": scaler.state_dict() if scaler is not None else None,
        "metrics": metrics if metrics is not None else {},
    }
    path = os.path.join(save_dir, f"{prefix}_epoch{epoch:03d}.pt")
    torch.save(ckpt, path)
    print(f"💾 checkpoint saved: {path}")


# =========================================================
# 1) gene_metadata load
# =========================================================
gene_md = pd.read_parquet(GENE_META_PATH).copy()
gene_md["gene_symbol"] = gene_md["gene_symbol"].astype(str)
gene_md["ensembl_id"]  = gene_md["ensembl_id"].astype(str)
gene_md["token_id"]    = gene_md["token_id"].astype(int)
gene_md = gene_md.sort_values("token_id").reset_index(drop=True)

N_GENES = int(gene_md["token_id"].max()) + 1
symbol_to_ensg_lower = dict(zip(gene_md["gene_symbol"].str.lower(), gene_md["ensembl_id"]))
ensg_to_token_id = dict(zip(gene_md["ensembl_id"].values, gene_md["token_id"].values))


# =========================================================
# 2) drug_metadata -> targets
# =========================================================
def parse_targets(x):
    if x is None:
        return []
    if isinstance(x, float) and np.isnan(x):
        return []
    if isinstance(x, (list, tuple)):
        return [str(t).strip() for t in x if str(t).strip()]
    if isinstance(x, str):
        s = x.strip()
        if (s.startswith("[") and s.endswith("]")) or (s.startswith("(") and s.endswith(")")):
            try:
                out = ast.literal_eval(s)
                if isinstance(out, (list, tuple)):
                    return [str(t).strip() for t in out if str(t).strip()]
            except Exception:
                pass
        for sep in [";", ","]:
            if sep in s:
                return [t.strip() for t in s.split(sep) if t.strip()]
        return [s]
    return [str(x).strip()]

drug_meta_df = pd.read_parquet(DRUG_META_PATH).copy()
drug_meta_df["drug"] = drug_meta_df["drug"].astype(str)

drug_to_target_tokenids = {}
all_target_tokenids = set()

for _, row in drug_meta_df.iterrows():
    drug = str(row["drug"])
    targets = parse_targets(row.get("targets", None))

    tids = []
    for t in targets:
        t = str(t).strip()
        if not t:
            continue
        if t.startswith("ENSG"):
            ensg = t
        else:
            ensg = symbol_to_ensg_lower.get(t.lower(), None)
        if ensg is None:
            continue
        tid = ensg_to_token_id.get(ensg, None)
        if tid is None:
            continue
        tid = int(tid)
        if 0 <= tid < N_GENES:
            tids.append(tid)

    tids = sorted(set(tids))
    drug_to_target_tokenids[drug] = tids
    all_target_tokenids.update(tids)

drug_has_targets = {d: (len(tids) > 0) for d, tids in drug_to_target_tokenids.items()}
print(f"drugs total={len(drug_to_target_tokenids)}, with>=1 target={sum(drug_has_targets.values())}")


# =========================================================
# 3) HVG from DMSO
# =========================================================
def compute_hvg_token_ids_from_dmso(dmso_h5ad_path: str, control_drug: str, HVG_K: int, ensg_to_token_id: dict):
    ad = sc.read_h5ad(dmso_h5ad_path)
    obs = ad.obs
    m = (obs["drug"].astype(str).values == control_drug)
    idx = np.where(m)[0]
    if idx.size == 0:
        raise ValueError(f"DMSO adata 안에 control drug({control_drug})가 없습니다.")

    X = ad.X.tocsr() if sparse.issparse(ad.X) else sparse.csr_matrix(ad.X)
    Xc = X[idx]

    mean = np.asarray(Xc.mean(axis=0)).ravel()
    mean2 = np.asarray(Xc.multiply(Xc).mean(axis=0)).ravel()
    var = (mean2 - mean**2).astype(np.float32)

    ensgs = ad.var_names.astype(str).tolist()

    token_ids = []
    vars_ = []
    for j, ensg in enumerate(ensgs):
        tid = ensg_to_token_id.get(ensg, None)
        if tid is None:
            continue
        token_ids.append(int(tid))
        vars_.append(float(var[j]))

    token_ids = np.asarray(token_ids, dtype=np.int64)
    vars_ = np.asarray(vars_, dtype=np.float32)

    if token_ids.size == 0:
        raise ValueError("DMSO var_names와 gene_metadata의 ENSG mapping이 거의 안 맞습니다.")

    k = min(int(HVG_K), token_ids.size)
    top = np.argpartition(-vars_, k-1)[:k]
    return set(token_ids[top].tolist())

hvg_token_ids = compute_hvg_token_ids_from_dmso(
    dmso_h5ad_path=DMSO_PATH,
    control_drug=CONTROL_DRUG,
    HVG_K=HVG_K,
    ensg_to_token_id=ensg_to_token_id,
)
print("HVG token_ids:", len(hvg_token_ids))

# INPUT subset = HVG ∪ TARGETS
subset_token_ids = sorted(set(hvg_token_ids) | set(all_target_tokenids))
M_SUB = len(subset_token_ids)
print("SUBSET (HVG ∪ TARGETS) size:", M_SUB)

# OUTPUT target-only = TARGETS only
target_token_ids = sorted(set(all_target_tokenids))
M_TGT = len(target_token_ids)
print("TARGET-ONLY size:", M_TGT)

old_tid_to_subid = {tid: i for i, tid in enumerate(subset_token_ids)}
old_tid_to_tgtid = {tid: i for i, tid in enumerate(target_token_ids)}


# =========================================================
# 4) subset vocab + LUT (fast mapping)
# =========================================================
SPECIAL_TOKENS = ["[PAD]", "[CLS]", "[DRUG]", "[TARGET]", "[CELL]", "[MASK]"]
local_token_to_id = {tok: i for i, tok in enumerate(SPECIAL_TOKENS)}
N_SPECIAL = len(SPECIAL_TOKENS)

VOCAB_SIZE = N_SPECIAL + M_SUB
PAD_ID = local_token_to_id["[PAD]"]
CLS_ID = local_token_to_id["[CLS]"]
CELLTOK_ID = local_token_to_id["[CELL]"]

print("VOCAB_SIZE:", VOCAB_SIZE)

old_tid_to_vocab_lut = np.full((N_GENES,), -1, dtype=np.int64)
for sid, old_tid in enumerate(subset_token_ids):
    if 0 <= old_tid < N_GENES:
        old_tid_to_vocab_lut[old_tid] = N_SPECIAL + sid

subset_token_ids_np = np.asarray(subset_token_ids, dtype=np.int64)


# =========================================================
# 5) y_targets (drug -> TARGET-ONLY multi-hot)
# =========================================================
drug_to_target_vec_tgt = {}
for d, tids in drug_to_target_tokenids.items():
    vec = np.zeros(M_TGT, dtype=np.float32)
    for tid in tids:
        j = old_tid_to_tgtid.get(int(tid), None)
        if j is not None:
            vec[j] = 1.0
    drug_to_target_vec_tgt[d] = vec

n_nonzero = sum(float(v.sum()) > 0 for v in drug_to_target_vec_tgt.values())
print("drugs with>=1 target (target-only vec):", n_nonzero)


# =========================================================
# 6) SMILES embeddings
# =========================================================
obj = torch.load(SMILES_EMB_PATH, map_location="cpu")
assert isinstance(obj, dict) and "drug" in obj and "emb" in obj

drug_list_saved = [str(d) for d in obj["drug"]]
emb_matrix = obj["emb"].to(dtype=torch.float32).cpu().numpy()
SMILES_DIM = int(emb_matrix.shape[1])

drug_to_smiles_np_raw = {d: emb_matrix[i].astype(np.float32, copy=False) for i, d in enumerate(drug_list_saved)}

drug_names = drug_meta_df["drug"].astype(str).tolist()
drug_to_smiles_np = {}
missing = 0
for d in drug_names:
    v = drug_to_smiles_np_raw.get(d, None)
    if v is None:
        drug_to_smiles_np[d] = np.zeros((SMILES_DIM,), dtype=np.float32)
        missing += 1
    else:
        drug_to_smiles_np[d] = v
print(f"✅ SMILES aligned | missing={missing}/{len(drug_names)}")

drug2id = {d: i for i, d in enumerate(sorted(set(drug_names)))}


# =========================================================
# 7) DMSO baselines (gene-space)
# =========================================================
def build_dmso_baselines_gene_space(dmso_h5ad_path: str, control_drug: str, N_GENES: int, ensg_to_token_id: dict):
    adata = sc.read_h5ad(dmso_h5ad_path)
    obs = adata.obs
    X = adata.X.tocsr() if sparse.issparse(adata.X) else sparse.csr_matrix(adata.X)

    m = (obs["drug"].astype(str).values == control_drug)
    idx = np.where(m)[0]
    if idx.size == 0:
        raise ValueError(f"DMSO adata 안에 control drug({control_drug})가 없습니다.")

    ensgs = adata.var_names.astype(str).tolist()

    token_ids, cols = [], []
    for j, ensg in enumerate(ensgs):
        tid = ensg_to_token_id.get(ensg, None)
        if tid is None:
            continue
        token_ids.append(int(tid))
        cols.append(j)

    token_ids = np.asarray(token_ids, dtype=np.int64)
    cols = np.asarray(cols, dtype=np.int64)

    Xc = X[idx][:, cols]
    mean_global_sub = np.asarray(Xc.mean(axis=0)).ravel().astype(np.float32)

    baseline_global = np.zeros(N_GENES, dtype=np.float32)
    baseline_global[token_ids] = mean_global_sub

    baseline_by_cl = {}
    cl_values = obs["cell_line_id"].astype(str).values
    for cl in np.unique(cl_values):
        cl_idx = np.where(m & (cl_values == cl))[0]
        if cl_idx.size == 0:
            continue
        Xcl = X[cl_idx][:, cols]
        mean_cl_sub = np.asarray(Xcl.mean(axis=0)).ravel().astype(np.float32)
        v = np.zeros(N_GENES, dtype=np.float32)
        v[token_ids] = mean_cl_sub
        baseline_by_cl[str(cl)] = v

    return baseline_global, baseline_by_cl

baseline_global, baseline_by_cl = build_dmso_baselines_gene_space(
    dmso_h5ad_path=DMSO_PATH,
    control_drug=CONTROL_DRUG,
    N_GENES=N_GENES,
    ensg_to_token_id=ensg_to_token_id,
)
print("baseline_global:", baseline_global.shape, "baseline_by_cl:", len(baseline_by_cl))


# =========================================================
# 8) split (drug, cell_line) pairs + weights
# =========================================================
DRUG_COL, CELL_COL, N_COL = "drug", "cell_line_id", "n_cells"
MIN_TRAIN = 1000
TEST_SIZE = 0.1

counts = pd.read_csv(COUNTS_CSV)
counts[DRUG_COL] = counts[DRUG_COL].astype(str)
counts[CELL_COL] = counts[CELL_COL].astype(str)

train_pool = counts[counts[N_COL] >= MIN_TRAIN].copy()
pairs_df = train_pool[[DRUG_COL, CELL_COL]].drop_duplicates()

pairs_df = pairs_df[pairs_df[DRUG_COL] != CONTROL_DRUG]
pairs_df = pairs_df[pairs_df[DRUG_COL].map(lambda d: drug_has_targets.get(str(d), False))]

train_df, val_df = train_test_split(
    pairs_df,
    test_size=TEST_SIZE,
    random_state=SEED,
    stratify=pairs_df[DRUG_COL],
)

train_pairs = list(zip(train_df[DRUG_COL], train_df[CELL_COL]))
val_pairs   = list(zip(val_df[DRUG_COL],   val_df[CELL_COL]))

print("train pairs:", len(train_pairs))
print("val pairs:", len(val_pairs))

def make_pair_weights_from_counts(counts_df, pairs, drug_col="drug", cell_col="cell_line_id", n_col="n_cells",
                                  mode="inv_sqrt", eps=1.0):
    tmp = counts_df[[drug_col, cell_col, n_col]].copy()
    tmp[drug_col] = tmp[drug_col].astype(str)
    tmp[cell_col] = tmp[cell_col].astype(str)

    pair2n = {(d, c): int(n) for d, c, n in tmp.values}

    w = []
    for p in pairs:
        n = pair2n.get(p, 0)
        if mode == "inv":
            ww = 1.0 / (n + eps)
        elif mode == "inv_log":
            ww = 1.0 / np.log1p(n + eps)
        else:
            ww = 1.0 / np.sqrt(n + eps)
        w.append(float(ww))

    w = np.asarray(w, dtype=np.float64)
    w = np.clip(w, 0.0, None)
    w = w / (w.sum() + 1e-12)
    return w

w_train = make_pair_weights_from_counts(counts, train_pairs, mode="inv_sqrt")
w_val   = make_pair_weights_from_counts(counts, val_pairs,   mode="inv_sqrt")


# =========================================================
# 9) parquet row-group indexing
# =========================================================
PARQUET_FILES = sorted(glob.glob(os.path.join(PARQUET_DIR, "**", "*.parquet"), recursive=True))
print("parquet files found:", len(PARQUET_FILES))

def build_pair_to_locations(parquet_files, valid_pairs_set, drug_col="drug", cell_col="cell_line_id"):
    out = defaultdict(list)
    for f in tqdm(parquet_files, desc="Index parquet row-groups", dynamic_ncols=True):
        pf = pq.ParquetFile(f)
        for rg in range(pf.num_row_groups):
            tbl = pf.read_row_group(rg, columns=[drug_col, cell_col])
            df = tbl.to_pandas()
            pairs_here = set(zip(df[drug_col].astype(str), df[cell_col].astype(str)))
            inter = pairs_here.intersection(valid_pairs_set)
            for p in inter:
                out[p].append((f, rg))
    return out

valid_pairs_set = set(train_pairs) | set(val_pairs)
pair_to_locations = build_pair_to_locations(PARQUET_FILES, valid_pairs_set, drug_col=DRUG_COL, cell_col=CELL_COL)
print("indexed pairs:", len(pair_to_locations))


# =========================================================
# 10) Dataset (drop first genes/expr token + target-only labels)
# =========================================================
class TahoeFPParquetDatasetMultiPair(torch.utils.data.IterableDataset):
    def __init__(
        self,
        pair_to_locations,
        pairs,
        baseline_global,
        baseline_by_cellline,
        drug_to_target_vec_target_only,   # (M_TGT,)
        drug2id,
        drug_to_smiles_np,
        n_genes_full,
        steps,
        max_seq_len=256,
        batch_size=128,
        pairs_per_batch=16,
        control_drug="DMSO_TF",
        pad_id=0,
        cls_id=1,
        celltok_id=4,
        cell_line2id=None,
        unk_cell_id=0,
        pair_weights=None,
        seed=42,
        drug_col="drug",
        cell_col="cell_line_id",
        genes_col="genes",
        expr_col="expressions",
        cap_per_pair_in_rg=None,
        max_tries_per_pair=20,
        invalid_global_gene_tids=(1, 2),
        subset_token_ids_np=None,
        old_tid_to_vocab_lut=None,
        m_tgt: int = 0,
        drop_first_gene_token: bool = True,  # ✅ new
    ):
        super().__init__()
        self.pair_to_locations = pair_to_locations
        self.pairs = list(pairs)

        self.baseline_global = np.asarray(baseline_global, dtype=np.float32)
        self.baseline_by_cellline = baseline_by_cellline or {}

        self.drug_to_target_vec_target_only = drug_to_target_vec_target_only
        self.drug2id = drug2id
        self.drug_to_smiles_np = drug_to_smiles_np

        self.n_genes_full = int(n_genes_full)
        self.steps = int(steps)
        self.max_seq_len = int(max_seq_len)
        self.batch_size = int(batch_size)

        self.pairs_per_batch = int(pairs_per_batch)
        assert self.batch_size % self.pairs_per_batch == 0
        self.cells_per_pair = self.batch_size // self.pairs_per_batch

        self.control_drug = str(control_drug)
        self.pad_id = int(pad_id)
        self.cls_id = int(cls_id)

        self.celltok_id = int(celltok_id)
        self.cell_line2id = cell_line2id or {}
        self.unk_cell_id = int(unk_cell_id)

        self.drug_col = drug_col
        self.cell_col = cell_col
        self.genes_col = genes_col
        self.expr_col = expr_col

        self.cap_per_pair_in_rg = cap_per_pair_in_rg
        self.max_tries_per_pair = int(max_tries_per_pair)
        self.seed = int(seed)

        any_vec = next(iter(self.drug_to_smiles_np.values()))
        self.smiles_dim = int(any_vec.shape[-1])

        self.invalid_global_gene_tids = np.asarray(list(set(int(x) for x in invalid_global_gene_tids)), dtype=np.int64)

        self.m_tgt = int(m_tgt)
        assert self.m_tgt > 0

        self.drop_first_gene_token = bool(drop_first_gene_token)

        # weights
        if pair_weights is None:
            self.pair_weights = None
        else:
            w = np.asarray(pair_weights, dtype=np.float64)
            assert len(w) == len(self.pairs)
            w = np.clip(w, 0.0, None)
            w = w / (w.sum() + 1e-12)
            self.pair_weights = w

        self.subset_token_ids_np = subset_token_ids_np
        self.old_tid_to_vocab_lut = old_tid_to_vocab_lut

        self._pf_cache = {}

    def _get_pf(self, file_path):
        pf = self._pf_cache.get(file_path, None)
        if pf is None:
            pf = pq.ParquetFile(file_path)
            self._pf_cache[file_path] = pf
        return pf

    def _read_row_group_df(self, file_path, rg_id, columns):
        pf = self._get_pf(file_path)
        return pf.read_row_group(rg_id, columns=columns).to_pandas()

    def _prepare_sparse_sorted(self, genes, expr):
        idx = np.asarray(genes, dtype=np.int64)
        val = np.asarray(expr, dtype=np.float32)

        # ✅ drop the first "service" element (genes[0], expr[0])
        if self.drop_first_gene_token:
            if idx.size > 0 and val.size > 0:
                L = min(idx.size, val.size)
                idx = idx[:L]
                val = val[:L]
                if L >= 1:
                    idx = idx[1:]
                    val = val[1:]

        if idx.size == 0 or val.size == 0:
            return np.asarray([], dtype=np.int64), np.asarray([], dtype=np.float32)

        L = min(idx.size, val.size)
        idx = idx[:L]
        val = val[:L]

        if self.invalid_global_gene_tids.size > 0:
            m_bad = np.isin(idx, self.invalid_global_gene_tids, assume_unique=False)
            if m_bad.any():
                keep = ~m_bad
                idx = idx[keep]
                val = val[keep]
                if idx.size == 0:
                    return idx, val

        m = (idx >= 0) & (idx < self.n_genes_full)
        idx = idx[m]
        val = val[m]
        if idx.size == 0:
            return idx, val

        order = np.argsort(idx)
        return idx[order], val[order]

    def _fill_one_row(self, row_genes, row_expr, baseline_vec, input_ids_row, values_row, attn_row):
        idx_sorted, val_sorted = self._prepare_sparse_sorted(row_genes, row_expr)
        if idx_sorted.size == 0:
            return

        delta = val_sorted - baseline_vec[idx_sorted]

        mask_sub = np.isin(idx_sorted, self.subset_token_ids_np, assume_unique=False)
        if not mask_sub.any():
            return

        idx_sorted = idx_sorted[mask_sub]
        delta = delta[mask_sub]
        if idx_sorted.size == 0:
            return

        k = min(self.max_seq_len, idx_sorted.size)
        if k <= 0:
            return

        if k == idx_sorted.size:
            top_pos = np.argsort(-np.abs(delta))
        else:
            top_pos = np.argpartition(-np.abs(delta), k - 1)[:k]
            top_pos = top_pos[np.argsort(-np.abs(delta[top_pos]))]

        sel_token_ids = idx_sorted[top_pos]
        sel_delta = delta[top_pos]

        sel_vocab_ids = self.old_tid_to_vocab_lut[sel_token_ids]
        ok = sel_vocab_ids != -1
        if not ok.any():
            return

        sel_vocab_ids = sel_vocab_ids[ok]
        sel_delta = sel_delta[ok]

        L = min(self.max_seq_len, sel_vocab_ids.size)
        if L <= 0:
            return

        input_ids_row[2:2+L] = sel_vocab_ids[:L]
        values_row[2:2+L]    = sel_delta[:L]
        attn_row[2:2+L]      = 1

    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        base_seed = self.seed if worker_info is None else (self.seed + worker_info.id)
        rng = np.random.default_rng(base_seed)

        pairs = self.pairs
        weights = self.pair_weights

        cols = [self.drug_col, self.cell_col, self.genes_col, self.expr_col]

        cnt = 0
        while True:
            if weights is None:
                chosen_idx = rng.integers(0, len(pairs), size=self.pairs_per_batch)
            else:
                chosen_idx = rng.choice(len(pairs), size=self.pairs_per_batch, replace=True, p=weights)

            chosen_pairs = [pairs[i] for i in chosen_idx]

            seq_len = 2 + self.max_seq_len
            input_ids = np.full((self.batch_size, seq_len), self.pad_id, dtype=np.int64)
            values    = np.zeros((self.batch_size, seq_len), dtype=np.float32)
            attn      = np.zeros((self.batch_size, seq_len), dtype=np.int64)

            input_ids[:, 0] = self.cls_id
            input_ids[:, 1] = self.celltok_id
            attn[:, 0:2] = 1

            y_batch = np.zeros((self.batch_size, self.m_tgt), dtype=np.float32)
            smiles_batch = np.zeros((self.batch_size, self.smiles_dim), dtype=np.float32)
            drug_id_batch = np.zeros((self.batch_size,), dtype=np.int64)
            cell_id_batch = np.zeros((self.batch_size,), dtype=np.int64)

            row_ptr = 0
            built_any = False

            for (drug_name, cell_line) in chosen_pairs:
                drug_name = str(drug_name)
                cell_line = str(cell_line)

                if drug_name == self.control_drug:
                    continue

                y_vec = self.drug_to_target_vec_target_only.get(drug_name, None)
                if y_vec is None or float(np.sum(y_vec)) <= 0.0:
                    continue

                locs = self.pair_to_locations.get((drug_name, cell_line), [])
                if not locs:
                    continue

                baseline = self.baseline_by_cellline.get(cell_line, self.baseline_global)

                sm_vec = self.drug_to_smiles_np.get(drug_name, None)
                if sm_vec is None:
                    sm_vec = np.zeros((self.smiles_dim,), dtype=np.float32)

                did = int(self.drug2id.get(drug_name, 0))
                cid = int(self.cell_line2id.get(cell_line, self.unk_cell_id))

                for _ in range(self.max_tries_per_pair):
                    fpath, rg_id = locs[rng.integers(0, len(locs))]
                    df = self._read_row_group_df(fpath, rg_id, columns=cols)

                    df = df[(df[self.drug_col].astype(str) == drug_name) &
                            (df[self.cell_col].astype(str) == cell_line)]
                    if len(df) == 0:
                        continue

                    if self.cap_per_pair_in_rg is not None and len(df) > self.cap_per_pair_in_rg:
                        df = df.sample(self.cap_per_pair_in_rg, replace=False, random_state=None)

                    replace = len(df) < self.cells_per_pair
                    df = df.sample(self.cells_per_pair, replace=replace, random_state=None)

                    for r in df.itertuples(index=False):
                        if row_ptr >= self.batch_size:
                            break

                        y_batch[row_ptr] = y_vec
                        smiles_batch[row_ptr] = sm_vec
                        drug_id_batch[row_ptr] = did
                        cell_id_batch[row_ptr] = cid

                        genes = getattr(r, self.genes_col)
                        expr  = getattr(r, self.expr_col)

                        self._fill_one_row(
                            genes, expr, baseline,
                            input_ids[row_ptr], values[row_ptr], attn[row_ptr]
                        )
                        row_ptr += 1

                    built_any = True
                    break

                if row_ptr >= self.batch_size:
                    break

            if not built_any:
                continue

            if row_ptr < self.batch_size:
                fill = self.batch_size - row_ptr
                input_ids[row_ptr:] = input_ids[:fill]
                values[row_ptr:]    = values[:fill]
                attn[row_ptr:]      = attn[:fill]
                y_batch[row_ptr:]   = y_batch[:fill]
                smiles_batch[row_ptr:] = smiles_batch[:fill]
                drug_id_batch[row_ptr:] = drug_id_batch[:fill]
                cell_id_batch[row_ptr:] = cell_id_batch[:fill]

            yield {
                "input_ids": torch.tensor(input_ids, dtype=torch.long),
                "values": torch.tensor(values, dtype=torch.float32),
                "attention_mask": torch.tensor(attn, dtype=torch.long),
                "y_targets": torch.tensor(y_batch, dtype=torch.float32),   # (B, M_TGT)
                "smiles_emb": torch.tensor(smiles_batch, dtype=torch.float32),
                "drug_id": torch.tensor(drug_id_batch, dtype=torch.long),
                "cell_id": torch.tensor(cell_id_batch, dtype=torch.long),
            }

            cnt += 1
            if cnt >= self.steps:
                return




drugs total=379, with>=1 target=264
HVG token_ids: 4000
SUBSET (HVG ∪ TARGETS) size: 4184
TARGET-ONLY size: 278
VOCAB_SIZE: 4190
drugs with>=1 target (target-only vec): 264
✅ SMILES aligned | missing=0/379
baseline_global: (62713,) baseline_by_cl: 50
train pairs: 10505
val pairs: 1168
parquet files found: 3388


Index parquet row-groups: 100%|██████████| 3388/3388 [09:23<00:00,  6.02it/s]

indexed pairs: 11673


In [3]:
# =========================================================
# 10.1) cell2id mapping + loaders
# =========================================================
cell2id_df = pd.read_csv(CELL2ID_CSV)
cell2id_df["cell_line_id"] = cell2id_df["cell_line_id"].astype(str)
cell2id_df["cell_id"] = cell2id_df["cell_id"].astype(int)

cell_line2id = dict(zip(cell2id_df["cell_line_id"], cell2id_df["cell_id"]))
NUM_CELL_LINE = int(cell2id_df["cell_id"].max()) + 1
print("loaded cell2id:", len(cell_line2id), "NUM_CELL_LINE:", NUM_CELL_LINE)

CAP_PER_PAIR_IN_RG = None

train_ds = TahoeFPParquetDatasetMultiPair(
    pair_to_locations=pair_to_locations,
    pairs=train_pairs,
    baseline_global=baseline_global,
    baseline_by_cellline=baseline_by_cl,
    drug_to_target_vec_target_only=drug_to_target_vec_tgt,
    drug2id=drug2id,
    drug_to_smiles_np=drug_to_smiles_np,
    n_genes_full=N_GENES,
    steps=STEPS_PER_EPOCH,
    max_seq_len=MAX_SEQ_LEN,
    batch_size=BATCH_SIZE,
    pairs_per_batch=PAIRS_PER_BATCH,
    control_drug=CONTROL_DRUG,
    pad_id=PAD_ID,
    cls_id=CLS_ID,
    celltok_id=CELLTOK_ID,
    cell_line2id=cell_line2id,
    unk_cell_id=0,
    pair_weights=w_train,
    seed=SEED,
    cap_per_pair_in_rg=CAP_PER_PAIR_IN_RG,
    max_tries_per_pair=20,
    subset_token_ids_np=subset_token_ids_np,
    old_tid_to_vocab_lut=old_tid_to_vocab_lut,
    m_tgt=M_TGT,
    drop_first_gene_token=DROP_FIRST_GENE_TOKEN,
)

val_ds = TahoeFPParquetDatasetMultiPair(
    pair_to_locations=pair_to_locations,
    pairs=val_pairs,
    baseline_global=baseline_global,
    baseline_by_cellline=baseline_by_cl,
    drug_to_target_vec_target_only=drug_to_target_vec_tgt,
    drug2id=drug2id,
    drug_to_smiles_np=drug_to_smiles_np,
    n_genes_full=N_GENES,
    steps=VAL_STEPS,
    max_seq_len=MAX_SEQ_LEN,
    batch_size=BATCH_SIZE,
    pairs_per_batch=PAIRS_PER_BATCH,
    control_drug=CONTROL_DRUG,
    pad_id=PAD_ID,
    cls_id=CLS_ID,
    celltok_id=CELLTOK_ID,
    cell_line2id=cell_line2id,
    unk_cell_id=0,
    pair_weights=w_val,
    seed=SEED + 123,
    cap_per_pair_in_rg=CAP_PER_PAIR_IN_RG,
    max_tries_per_pair=20,
    subset_token_ids_np=subset_token_ids_np,
    old_tid_to_vocab_lut=old_tid_to_vocab_lut,
    m_tgt=M_TGT,
    drop_first_gene_token=DROP_FIRST_GENE_TOKEN,
)

train_loader = DataLoader(
    train_ds,
    batch_size=None,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=(NUM_WORKERS > 0),
    prefetch_factor=2 if NUM_WORKERS > 0 else None,
)
val_loader = DataLoader(
    val_ds,
    batch_size=None,
    num_workers=0,
    pin_memory=True
)


# =========================================================
# 11) Model
# =========================================================
class FPEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, num_layers, pad_id,
                 max_len: int, num_cell_lines: int, cell_pos: int = 1):
        super().__init__()
        self.token_emb  = nn.Embedding(vocab_size, d_model, padding_idx=pad_id)
        self.value_proj = nn.Linear(1, d_model)
        self.pos_emb = nn.Embedding(max_len, d_model)
        self.cell_line_emb = nn.Embedding(num_cell_lines, d_model)
        self.cell_pos = int(cell_pos)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=4*d_model,
            dropout=0.1,
            batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)

    def forward(self, input_ids, values, attention_mask, cell_line_id):
        B, L = input_ids.shape
        dev = input_ids.device

        x = self.token_emb(input_ids) + self.value_proj(values.unsqueeze(-1))
        pos = torch.arange(L, device=dev).unsqueeze(0).expand(B, L)
        x = x + self.pos_emb(pos)

        if cell_line_id is not None:
            x[:, self.cell_pos, :] = x[:, self.cell_pos, :] + self.cell_line_emb(cell_line_id.to(dev)).to(x.dtype)

        key_padding_mask = (attention_mask == 0)
        h = self.encoder(x, src_key_padding_mask=key_padding_mask)
        return h[:, 0, :]


class FPModelTied(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, num_layers, pad_id, smiles_dim,
                 max_len: int, num_cell_lines: int):
        super().__init__()
        self.encoder = FPEncoder(
            vocab_size=vocab_size,
            d_model=d_model,
            n_heads=n_heads,
            num_layers=num_layers,
            pad_id=pad_id,
            max_len=max_len,
            num_cell_lines=num_cell_lines,
            cell_pos=1,
        )
        self.proj = nn.Linear(d_model, d_model)
        self.smiles_head = nn.Sequential(
            nn.Linear(d_model, 4*d_model),
            nn.BatchNorm1d(4*d_model),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(4*d_model, smiles_dim),
            nn.LayerNorm(smiles_dim),
        )

    def gene_emb_subset(self):
        return self.encoder.token_emb.weight[N_SPECIAL:, :]  # (M_SUB, d)

    def forward(self, input_ids, values, attention_mask, cell_line_id, return_smiles=False):
        h_cls = self.encoder(input_ids, values, attention_mask, cell_line_id)
        v_pred = self.proj(h_cls)
        z_pred = self.smiles_head(h_cls)
        if return_smiles:
            return v_pred, z_pred
        return v_pred


D_MODEL = 256
N_HEADS = 8
N_LAYERS = 4

model = FPModelTied(
    vocab_size=VOCAB_SIZE,
    d_model=D_MODEL,
    n_heads=N_HEADS,
    num_layers=N_LAYERS,
    pad_id=PAD_ID,
    smiles_dim=SMILES_DIM,
    max_len=(2 + MAX_SEQ_LEN),
    num_cell_lines=NUM_CELL_LINE
).to(device)


# =========================================================
# 12) Load pretrained embeddings (subset + cell)
# =========================================================
def load_pretrained_subset_into_token_emb(token_emb: nn.Embedding, npy_path: str, device):
    W = np.load(npy_path)  # (N_GENES, d)
    Wt = torch.tensor(W, dtype=torch.float32, device=device)
    d = token_emb.weight.shape[1]
    if Wt.shape[1] != d:
        raise ValueError(f"d mismatch: npy={Wt.shape[1]} vs token_emb={d}")

    loaded = 0
    with torch.no_grad():
        for sid, old_tid in enumerate(subset_token_ids):
            vid = N_SPECIAL + sid
            if 0 <= old_tid < Wt.shape[0]:
                token_emb.weight[vid].copy_(Wt[int(old_tid)])
                loaded += 1
    print(f"✅ token_emb loaded: {loaded}/{len(subset_token_ids)}")

load_pretrained_subset_into_token_emb(model.encoder.token_emb, PRETRAINED_GENE_NPY, device=device)

def load_pretrained_cell_embeddings(cell_emb_layer, cell_emb_npy, device):
    W = np.load(cell_emb_npy)
    if W.shape != tuple(cell_emb_layer.weight.shape):
        raise ValueError(f"cell emb shape mismatch: npy={W.shape} vs layer={tuple(cell_emb_layer.weight.shape)}")
    with torch.no_grad():
        cell_emb_layer.weight.copy_(torch.tensor(W, device=device, dtype=cell_emb_layer.weight.dtype))
    print(f"✅ loaded cell embeddings: {W.shape}")

load_pretrained_cell_embeddings(model.encoder.cell_line_emb, CELL_EMB_NPY, device)


# =========================================================
# 12.1) TARGET-ONLY embedding bank indices inside SUBSET
# =========================================================
target_sub_ids = [old_tid_to_subid[tid] for tid in target_token_ids]
target_sub_ids = torch.tensor(target_sub_ids, dtype=torch.long, device=device)
print("target_sub_ids:", tuple(target_sub_ids.shape))


# =========================================================
# 13) pos_weight (TARGET-ONLY)
# =========================================================
Y_list = [v for v in drug_to_target_vec_tgt.values() if float(np.sum(v)) > 0]
Y_all = np.stack(Y_list, axis=0)

pos = Y_all.sum(axis=0)
neg = Y_all.shape[0] - pos
pw = neg / (pos + 1e-6)

if POSW_MODE == "clip":
    pw = np.minimum(pw, POSW_MAX)
elif POSW_MODE == "sqrt_clip":
    pw = np.sqrt(pw)
    pw = np.minimum(pw, POSW_MAX)

pos_weight = torch.tensor(pw, dtype=torch.float32, device=device)
print("✅ pos_weight(target-only):", pos_weight.shape, "| max=", float(pos_weight.max().item()))


# =========================================================
# 14) Losses
# =========================================================
def info_nce_ranking_loss_multi_pos(
    v_pred: torch.Tensor,
    gene_emb: torch.Tensor,
    y_targets: torch.Tensor,
    num_neg: int = 256,
    num_pos: int = 8,
    tau: float = 0.1,
):
    device_ = v_pred.device
    B, _ = v_pred.shape
    losses = []

    for i in range(B):
        pos_idx = (y_targets[i] > 0.5).nonzero(as_tuple=True)[0]
        if pos_idx.numel() == 0:
            continue

        neg_idx_all = (y_targets[i] < 0.5).nonzero(as_tuple=True)[0]
        if neg_idx_all.numel() == 0:
            continue

        if num_pos is not None and num_pos > 0 and pos_idx.numel() > num_pos:
            pos_idx = pos_idx[torch.randperm(pos_idx.numel(), device=device_)[:num_pos]]

        if neg_idx_all.numel() > num_neg:
            neg_idx = neg_idx_all[torch.randperm(neg_idx_all.numel(), device=device_)[:num_neg]]
        else:
            neg_idx = neg_idx_all

        pos_emb = gene_emb[pos_idx]
        neg_emb = gene_emb[neg_idx]
        cand_emb = torch.cat([pos_emb, neg_emb], dim=0)

        v = v_pred[i].unsqueeze(0)
        scores = F.cosine_similarity(v.expand_as(cand_emb), cand_emb, dim=-1) / tau

        P = pos_emb.size(0)
        logits = scores.unsqueeze(0).repeat(P, 1)
        targets = torch.arange(P, device=device_, dtype=torch.long)
        losses.append(F.cross_entropy(logits, targets))

    if len(losses) == 0:
        return torch.tensor(0.0, device=device_)
    return torch.stack(losses).mean()

def bce_with_neg_sampling(
    pred_vec: torch.Tensor,
    y_targets: torch.Tensor,
    gene_emb: torch.Tensor,
    pos_weight_full: torch.Tensor,
    num_neg: int = 2048,
    pos_cap: int | None = None,
):
    device_ = pred_vec.device
    B, _ = pred_vec.shape
    losses = []

    for i in range(B):
        yi = y_targets[i]
        pos_idx = (yi > 0.5).nonzero(as_tuple=True)[0]
        if pos_idx.numel() == 0:
            continue

        if (pos_cap is not None) and (pos_idx.numel() > pos_cap):
            pos_idx = pos_idx[torch.randperm(pos_idx.numel(), device=device_)[:pos_cap]]

        neg_idx_all = (yi < 0.5).nonzero(as_tuple=True)[0]
        if neg_idx_all.numel() == 0:
            continue

        k = min(int(num_neg), neg_idx_all.numel())
        neg_idx = neg_idx_all[torch.randperm(neg_idx_all.numel(), device=device_)[:k]]

        idx = torch.cat([pos_idx, neg_idx], dim=0)
        logits = (pred_vec[i].unsqueeze(0) @ gene_emb[idx].T).squeeze(0)
        y_sub = yi[idx]
        pw_sub = pos_weight_full[idx]

        losses.append(F.binary_cross_entropy_with_logits(logits, y_sub, pos_weight=pw_sub, reduction="mean"))

    if len(losses) == 0:
        return torch.tensor(0.0, device=device_)
    return torch.stack(losses).mean()

def combined_target_loss_neg_sampling_tied(
    pred_vec: torch.Tensor,
    y_targets: torch.Tensor,
    gene_emb: torch.Tensor,
    pos_weight: torch.Tensor,
    lambda_cos: float = 1.0,
    lambda_bce: float = 0.1,
    lambda_rank: float = 0.5,
    bce_num_neg: int = 2048,
    bce_pos_cap: int | None = None,
    rank_num_neg: int = 256,
    rank_num_pos: int = 8,
    tau: float = 0.2,
):
    device_ = pred_vec.device

    true_vec = y_targets @ gene_emb
    num_t = y_targets.sum(dim=1, keepdim=True)
    mask = (num_t > 0).squeeze(1)

    if mask.any():
        true_vec_pos = true_vec[mask] / (num_t[mask] + 1e-6)
        pred_pos = pred_vec[mask]
        loss_cos = 1.0 - F.cosine_similarity(pred_pos, true_vec_pos, dim=-1).mean()
    else:
        loss_cos = torch.tensor(0.0, device=device_)

    loss_bce = bce_with_neg_sampling(
        pred_vec=pred_vec,
        y_targets=y_targets,
        gene_emb=gene_emb,
        pos_weight_full=pos_weight,
        num_neg=bce_num_neg,
        pos_cap=bce_pos_cap,
    )

    loss_rank = info_nce_ranking_loss_multi_pos(
        v_pred=pred_vec,
        gene_emb=gene_emb,
        y_targets=y_targets,
        num_neg=rank_num_neg,
        num_pos=rank_num_pos,
        tau=tau,
    )

    loss = lambda_cos * loss_cos + lambda_bce * loss_bce + lambda_rank * loss_rank
    return loss, loss_cos.detach(), loss_bce.detach(), loss_rank.detach()

def supervised_contrastive_loss_smiles(z_pred, z_true, drug_ids, tau: float = 0.07, remove_diagonal: bool = True):
    z_pred = F.normalize(z_pred, dim=1)
    z_true = F.normalize(z_true, dim=1)

    logits = (z_pred @ z_true.T) / tau
    labels = drug_ids.view(-1, 1)
    mask = torch.eq(labels, labels.T).float().to(z_pred.device)

    if remove_diagonal:
        mask.fill_diagonal_(0.0)

    logits = logits - logits.max(dim=1, keepdim=True).values.detach()
    log_prob = logits - torch.log(torch.exp(logits).sum(dim=1, keepdim=True) + 1e-6)

    pos_cnt = mask.sum(dim=1)
    valid = (pos_cnt > 0).float()

    mean_log_prob_pos = (mask * log_prob).sum(dim=1) / (pos_cnt + 1e-6)
    loss_per = -mean_log_prob_pos * valid
    return loss_per.sum() / (valid.sum() + 1e-6)

# ✅ NO MSE: cosine align only
def smiles_align_loss_cosine(z_pred: torch.Tensor, z_true: torch.Tensor):
    return 1.0 - F.cosine_similarity(z_pred, z_true, dim=-1).mean()

def smiles_mean_cosine(z_pred: torch.Tensor, z_true: torch.Tensor):
    return F.cosine_similarity(z_pred, z_true, dim=-1).mean()


# =========================================================
# 15) Eval (NO SMILES MSE)
# =========================================================
def compute_recall_precision_at_k(scores: torch.Tensor, y_true: torch.Tensor, k: int = 20):
    B, M = scores.shape
    kk = min(k, M)
    _, topk_idx = torch.topk(scores, k=kk, dim=1)

    recalls, precisions = [], []
    for i in range(B):
        true_labels = y_true[i]
        num_pos_ = true_labels.sum().item()
        if num_pos_ == 0:
            continue
        topk = topk_idx[i]
        num_pos_in_topk = true_labels[topk].sum().item()
        recalls.append(num_pos_in_topk / max(num_pos_, 1e-6))
        precisions.append(num_pos_in_topk / max(kk, 1))

    if len(recalls) == 0:
        return 0.0, 0.0
    return float(sum(recalls) / len(recalls)), float(sum(precisions) / len(precisions))

@torch.no_grad()
def evaluate_fp_targets_and_smiles(model, loader, device, target_sub_ids, k_list=(5,10), use_cosine=True, tau_smiles_eval=0.07):
    model.eval()

    gene_emb = model.gene_emb_subset()[target_sub_ids].to(device)  # (M_TGT, d)
    if use_cosine:
        g_norm = gene_emb / (gene_emb.norm(dim=1, keepdim=True) + 1e-8)
    else:
        g_norm = gene_emb

    recall_sums = {k: 0.0 for k in k_list}
    prec_sums   = {k: 0.0 for k in k_list}
    counts_     = {k: 0   for k in k_list}

    smiles_supcon_sum = 0.0
    smiles_cos_sum = 0.0
    n_smiles = 0

    for batch in loader:
        input_ids   = batch["input_ids"].to(device, non_blocking=True)
        values      = batch["values"].to(device, non_blocking=True)
        attn        = batch["attention_mask"].to(device, non_blocking=True)
        y_targets   = batch["y_targets"].to(device, non_blocking=True)
        smiles_true = batch["smiles_emb"].to(device, non_blocking=True)
        drug_ids    = batch["drug_id"].to(device, non_blocking=True)
        cell_id     = batch["cell_id"].to(device, non_blocking=True)

        v_pred, z_pred = model(input_ids, values, attn, cell_line_id=cell_id, return_smiles=True)

        if use_cosine:
            v_norm = v_pred / (v_pred.norm(dim=1, keepdim=True) + 1e-8)
            scores = v_norm @ g_norm.T
        else:
            scores = v_pred @ g_norm.T

        for k in k_list:
            r, p = compute_recall_precision_at_k(scores, y_targets, k=k)
            recall_sums[k] += r
            prec_sums[k]   += p
            counts_[k]     += 1

        bs = smiles_true.size(0)
        supcon = supervised_contrastive_loss_smiles(z_pred, smiles_true, drug_ids, tau=tau_smiles_eval)
        smiles_supcon_sum += supcon.item() * bs

        cos = smiles_mean_cosine(z_pred, smiles_true)
        smiles_cos_sum += cos.item() * bs

        n_smiles += bs

    out = {}
    for k in k_list:
        out[f"Recall@{k}"] = recall_sums[k] / max(counts_[k], 1)
        out[f"Precision@{k}"] = prec_sums[k] / max(counts_[k], 1)

    out["SMILES_SupCon"] = smiles_supcon_sum / max(n_smiles, 1)
    out["SMILES_COS"]    = smiles_cos_sum / max(n_smiles, 1)
    return out


# =========================================================
# 16) Train loop (NO SMILES MSE)
# =========================================================
def infinite_loader(loader):
    while True:
        for b in loader:
            yield b

USE_AMP = (device.type == "cuda")
scaler = GradScaler(enabled=USE_AMP)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=0.01)

def train_one_epoch_fixed_steps(
    model,
    train_loader,
    device,
    steps_per_epoch,
    optimizer,
    scaler,
    target_sub_ids,
    pos_weight,
    log_every=50,
    grad_clip=1.0,
):
    model.train()
    it = infinite_loader(train_loader)

    running_total = 0.0
    running_tgt   = 0.0
    running_sm    = 0.0
    running_rank_last = 0.0
    n = 0

    pbar = tqdm(range(1, steps_per_epoch + 1), desc="Train", leave=True, dynamic_ncols=True)

    for step in pbar:
        batch = next(it)

        input_ids   = batch["input_ids"].to(device, non_blocking=True)
        values      = batch["values"].to(device, non_blocking=True)
        attn        = batch["attention_mask"].to(device, non_blocking=True)
        y_targets   = batch["y_targets"].to(device, non_blocking=True)
        smiles_true = batch["smiles_emb"].to(device, non_blocking=True)
        drug_ids    = batch["drug_id"].to(device, non_blocking=True)
        cell_id     = batch["cell_id"].to(device, non_blocking=True)

        bs = input_ids.size(0)
        n += bs

        optimizer.zero_grad(set_to_none=True)

        if USE_AMP:
            with autocast(enabled=True):
                v_pred, z_pred = model(input_ids, values, attn, cell_line_id=cell_id, return_smiles=True)

                gene_emb = model.gene_emb_subset()[target_sub_ids]  # (M_TGT, d)

                loss_targets, loss_cos_t, loss_bce_t, loss_rank_t = combined_target_loss_neg_sampling_tied(
                    pred_vec=v_pred,
                    y_targets=y_targets,
                    gene_emb=gene_emb,
                    pos_weight=pos_weight,
                    lambda_cos=lambda_cos,
                    lambda_bce=lambda_bce,
                    lambda_rank=lambda_rank,
                    bce_num_neg=bce_num_neg,
                    bce_pos_cap=bce_pos_cap,
                    rank_num_neg=rank_num_neg,
                    rank_num_pos=rank_num_pos,
                    tau=0.2,
                )

                # ✅ SMILES: SupCon + cosine-align only (NO MSE)
                loss_supcon = supervised_contrastive_loss_smiles(z_pred, smiles_true, drug_ids, tau=tau_smiles)
                loss_align  = smiles_align_loss_cosine(z_pred, smiles_true)
                loss_smiles = loss_supcon + alpha_align * loss_align

                loss = loss_targets + lambda_smiles * loss_smiles

            if not torch.isfinite(loss).all():
                continue

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)

            any_grad = any(p.grad is not None for p in model.parameters())
            if not any_grad:
                scaler.update()
                continue

            if grad_clip is not None and grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

            scaler.step(optimizer)
            scaler.update()

        else:
            v_pred, z_pred = model(input_ids, values, attn, cell_line_id=cell_id, return_smiles=True)
            gene_emb = model.gene_emb_subset()[target_sub_ids]

            loss_targets, loss_cos_t, loss_bce_t, loss_rank_t = combined_target_loss_neg_sampling_tied(
                pred_vec=v_pred,
                y_targets=y_targets,
                gene_emb=gene_emb,
                pos_weight=pos_weight,
                lambda_cos=lambda_cos,
                lambda_bce=lambda_bce,
                lambda_rank=lambda_rank,
                bce_num_neg=bce_num_neg,
                bce_pos_cap=bce_pos_cap,
                rank_num_neg=rank_num_neg,
                rank_num_pos=rank_num_pos,
                tau=0.1,
            )

            loss_supcon = supervised_contrastive_loss_smiles(z_pred, smiles_true, drug_ids, tau=tau_smiles)
            loss_align  = smiles_align_loss_cosine(z_pred, smiles_true)
            loss_smiles = loss_supcon + alpha_align * loss_align
            loss = loss_targets + lambda_smiles * loss_smiles

            if not torch.isfinite(loss).all():
                continue

            loss.backward()
            if grad_clip is not None and grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()

        running_total += float(loss.item()) * bs
        running_tgt   += float(loss_targets.item()) * bs
        running_sm    += float(loss_smiles.item()) * bs
        running_rank_last = float(loss_rank_t.item())

        if step % log_every == 0:
            pos_in_batch = int((y_targets.sum(dim=1) > 0).sum().item())
            pbar.set_postfix({
                "loss": f"{running_total/max(n,1):.4f}",
                "tgt":  f"{running_tgt/max(n,1):.4f}",
                "sm":   f"{running_sm/max(n,1):.4f}",
                "rank(last)": f"{running_rank_last:.4f}",
                "pos_in_batch": f"{pos_in_batch}/{bs}",
            })

    return {
        "train_total":  running_total / max(n, 1),
        "train_target": running_tgt   / max(n, 1),
        "train_smiles": running_sm    / max(n, 1),
        "rank_last":    running_rank_last,
    }


# =========================================================
# 17) TRAIN
# =========================================================
print(">>> TRAIN START (TARGET-ONLY + DROP FIRST TOKEN + NO SMILES MSE)")

for epoch in range(1, EPOCHS + 1):
    logs = train_one_epoch_fixed_steps(
        model=model,
        train_loader=train_loader,
        device=device,
        steps_per_epoch=STEPS_PER_EPOCH,
        optimizer=optimizer,
        scaler=scaler,
        target_sub_ids=target_sub_ids,
        pos_weight=pos_weight,
        log_every=50,
    )

    print(f"\n[Epoch {epoch}/{EPOCHS}] "
          f"train_total={logs['train_total']:.4f} | "
          f"train_target={logs['train_target']:.4f} | "
          f"train_smiles={logs['train_smiles']:.4f} | "
          f"rank_last={logs['rank_last']:.4f}")

    valid_metrics = evaluate_fp_targets_and_smiles(
        model=model,
        loader=val_loader,
        device=device,
        target_sub_ids=target_sub_ids,
        k_list=(5, 10),
        use_cosine=True,
        tau_smiles_eval=tau_smiles,
    )
    print("✅ VALID metrics (NO MSE):", valid_metrics)

    if (epoch % SAVE_EVERY == 0) or (epoch == EPOCHS):
        save_checkpoint(
            save_dir=CKPT_DIR,
            model=model,
            optimizer=optimizer,
            scaler=scaler,
            epoch=epoch,
            metrics={"train": logs, "valid": valid_metrics},
            prefix="fp_target_only",
        )

print(">>> DONE")


loaded cell2id: 50 NUM_CELL_LINE: 50
✅ token_emb loaded: 4184/4184
✅ loaded cell embeddings: (50, 256)
target_sub_ids: (278,)
✅ pos_weight(target-only): torch.Size([278]) | max= 16.217267990112305


/data/aiffel/.cache/tmp/ipykernel_1229270/2984102383.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=USE_AMP)


>>> TRAIN START (TARGET-ONLY + DROP FIRST TOKEN + NO SMILES MSE)


Train:   0%|          | 0/10000 [00:00<?, ?it/s]/data/aiffel/.cache/tmp/ipykernel_1229270/2984102383.py:537: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True):
Train: 100%|██████████| 10000/10000 [1:22:56<00:00,  2.01it/s, loss=5.4889, tgt=5.2317, sm=5.1434, rank(last)=5.4443, pos_in_batch=128/128]



[Epoch 1/20] train_total=5.4889 | train_target=5.2317 | train_smiles=5.1434 | rank_last=5.4443


/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/site-packages/torch/nn/modules/transformer.py:515: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(


✅ VALID metrics (NO MSE): {'Recall@5': 0.10979057348901107, 'Precision@5': 0.030690625000000017, 'Recall@10': 0.16622548363095244, 'Precision@10': 0.022860937500000008, 'SMILES_SupCon': 4.925717622756958, 'SMILES_COS': 0.6084446209669113}


Train:   0%|          | 0/10000 [00:00<?, ?it/s]/data/aiffel/.cache/tmp/ipykernel_1229270/2984102383.py:537: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True):
Train:  23%|██▎       | 2297/10000 [17:55<1:06:12,  1.94it/s, loss=5.4352, tgt=5.1799, sm=5.1055, rank(last)=5.5646, pos_in_batch=128/128]Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 22

KeyboardInterrupt: 

In [4]:
@torch.no_grad()
def sanity_check_oracle_recall(val_loader, model, device, target_sub_ids, k_list=(5,10)):
    model.eval()
    # gene bank for targets (M_TGT, d)
    gene_emb = model.gene_emb_subset()[target_sub_ids].to(device)
    g_norm = gene_emb / (gene_emb.norm(dim=1, keepdim=True) + 1e-8)

    recall_sums = {k: 0.0 for k in k_list}
    prec_sums   = {k: 0.0 for k in k_list}
    n_batches = 0

    for batch in val_loader:
        y = batch["y_targets"].to(device)  # (B, M_TGT)
        if y.ndim != 2 or y.size(1) != g_norm.size(0):
            print("❌ SHAPE MISMATCH:", y.shape, "vs gene_bank:", g_norm.shape)
            return None

        # oracle vector: mean of true target embeddings
        true_vec = y @ g_norm                      # (B, d)
        denom = y.sum(dim=1, keepdim=True).clamp(min=1.0)
        v_oracle = true_vec / denom                # (B, d)
        v_norm = v_oracle / (v_oracle.norm(dim=1, keepdim=True) + 1e-8)

        scores = v_norm @ g_norm.T                 # (B, M_TGT)

        for k in k_list:
            r, p = compute_recall_precision_at_k(scores, y, k=k)
            recall_sums[k] += r
            prec_sums[k]   += p

        n_batches += 1
        if n_batches >= 20:  # достаточно 20 батчей
            break

    out = {f"Oracle_Recall@{k}": recall_sums[k]/max(n_batches,1) for k in k_list}
    out.update({f"Oracle_Precision@{k}": prec_sums[k]/max(n_batches,1) for k in k_list})
    print("✅ ORACLE sanity:", out)
    return out

# запусти
sanity_check_oracle_recall(val_loader, model, device, target_sub_ids, k_list=(5,10))


✅ ORACLE sanity: {'Oracle_Recall@5': 0.9872271825396826, 'Oracle_Recall@10': 0.9991666666666668, 'Oracle_Precision@5': 0.36312500000000003, 'Oracle_Precision@10': 0.1912500000000001}


{'Oracle_Recall@5': 0.9872271825396826,
 'Oracle_Recall@10': 0.9991666666666668,
 'Oracle_Precision@5': 0.36312500000000003,
 'Oracle_Precision@10': 0.1912500000000001}

In [5]:
# =========================================================
# FP (TARGET-ONLY) TRAINING SCRIPT
# - ✅ TARGET-ONLY (M_TGT=~280)
# - ✅ DROP FIRST "service" element in genes/expr
# - ✅ NO SMILES MSE (SupCon + cosine-align only)
# - ✅ FIX: BCE now uses COSINE logits (same geometry as eval/rank)
# - ✅ Recommended: lower BCE weight, higher Rank weight
# =========================================================

import os, glob, ast, random
from collections import defaultdict

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler

from tqdm import tqdm

import pyarrow.parquet as pq

import scanpy as sc
from scipy import sparse

from sklearn.model_selection import train_test_split


# =========================================================
# 0) PATHS / HYPERPARAMS
# =========================================================
PARQUET_DIR    = "/data/aiffel/data/Tahoe-100M/data"
GENE_META_PATH = "/data/aiffel/data/Tahoe-100M/metadata/gene_metadata.parquet"
DRUG_META_PATH = "/data/aiffel/data/Tahoe-100M/metadata/drug_metadata.parquet"
COUNTS_CSV     = "/data/aiffel/babayakga/making_data/aiffel/babayakga/making_data/tahoe_counts_per_drug_cell_line.csv"
DMSO_PATH      = "/data/aiffel/babayakga/outputs/dmso.h5ad"

SMILES_EMB_PATH       = "/data/aiffel/babayakga/smiles_emb/drug_smiles_emb_all1.pt"
PRETRAINED_GENE_NPY   = "/data/aiffel/babayakga/pretraining/checkpoints_with_cell/gene_embeddings.npy"
CELL2ID_CSV           = "/data/aiffel/babayakga/pretraining/checkpoints_with_cell/cell2id.csv"
CELL_EMB_NPY          = "/data/aiffel/babayakga/pretraining/checkpoints_with_cell/cell_embeddings.npy"

CONTROL_DRUG = "DMSO_TF"
SEED = 42

MAX_SEQ_LEN = 256
BATCH_SIZE  = 128
STEPS_PER_EPOCH = 10000
VAL_STEPS       = 500

# ✅ weights (recommendation to make Recall move)
lambda_cos    = 1.0
lambda_bce    = 0.05   # ⬇
lambda_rank   = 1.0    # ⬆

lambda_smiles = 0.05
alpha_align   = 0.5
tau_smiles    = 0.07

# ✅ ranking strength
rank_num_neg = 1024
rank_num_pos = 4
tau_rank     = 0.07

# ✅ BCE sampling
bce_num_neg  = 2048
bce_pos_cap  = None
tau_bce      = 0.10    # cosine logits temperature

EPOCHS = 20
LR = 1e-5

# ✅ pos_weight: temporarily off (often helps Recall start moving)
POSW_MODE = "none"     # ["none", "clip", "sqrt_clip"]
POSW_MAX  = 10.0       # if you later use clip/sqrt_clip

NUM_WORKERS = 4
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

HVG_K = 4000
PAIRS_PER_BATCH = 16

# ✅ drop first "service" element from genes/expressions
DROP_FIRST_GENE_TOKEN = True

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ✅ TF32 safe speed-up on Ampere+
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass


# =========================================================
# (extra) checkpoint save
# =========================================================
CKPT_DIR = "/data/aiffel/babayakga/checkpoints/fp_target_only_cosbce_rank1_drop1"
SAVE_EVERY = 2
os.makedirs(CKPT_DIR, exist_ok=True)

def save_checkpoint(save_dir, model, optimizer, scaler, epoch, metrics=None, prefix="fp"):
    ckpt = {
        "epoch": int(epoch),
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scaler_state_dict": scaler.state_dict() if scaler is not None else None,
        "metrics": metrics if metrics is not None else {},
    }
    path = os.path.join(save_dir, f"{prefix}_epoch{epoch:03d}.pt")
    torch.save(ckpt, path)
    print(f"💾 checkpoint saved: {path}")


# =========================================================
# 1) gene_metadata load
# =========================================================
gene_md = pd.read_parquet(GENE_META_PATH).copy()
gene_md["gene_symbol"] = gene_md["gene_symbol"].astype(str)
gene_md["ensembl_id"]  = gene_md["ensembl_id"].astype(str)
gene_md["token_id"]    = gene_md["token_id"].astype(int)
gene_md = gene_md.sort_values("token_id").reset_index(drop=True)

N_GENES = int(gene_md["token_id"].max()) + 1
symbol_to_ensg_lower = dict(zip(gene_md["gene_symbol"].str.lower(), gene_md["ensembl_id"]))
ensg_to_token_id = dict(zip(gene_md["ensembl_id"].values, gene_md["token_id"].values))


# =========================================================
# 2) drug_metadata -> targets
# =========================================================
def parse_targets(x):
    if x is None:
        return []
    if isinstance(x, float) and np.isnan(x):
        return []
    if isinstance(x, (list, tuple)):
        return [str(t).strip() for t in x if str(t).strip()]
    if isinstance(x, str):
        s = x.strip()
        if (s.startswith("[") and s.endswith("]")) or (s.startswith("(") and s.endswith(")")):
            try:
                out = ast.literal_eval(s)
                if isinstance(out, (list, tuple)):
                    return [str(t).strip() for t in out if str(t).strip()]
            except Exception:
                pass
        for sep in [";", ","]:
            if sep in s:
                return [t.strip() for t in s.split(sep) if t.strip()]
        return [s]
    return [str(x).strip()]

drug_meta_df = pd.read_parquet(DRUG_META_PATH).copy()
drug_meta_df["drug"] = drug_meta_df["drug"].astype(str)

drug_to_target_tokenids = {}
all_target_tokenids = set()

for _, row in drug_meta_df.iterrows():
    drug = str(row["drug"])
    targets = parse_targets(row.get("targets", None))

    tids = []
    for t in targets:
        t = str(t).strip()
        if not t:
            continue
        if t.startswith("ENSG"):
            ensg = t
        else:
            ensg = symbol_to_ensg_lower.get(t.lower(), None)
        if ensg is None:
            continue
        tid = ensg_to_token_id.get(ensg, None)
        if tid is None:
            continue
        tid = int(tid)
        if 0 <= tid < N_GENES:
            tids.append(tid)

    tids = sorted(set(tids))
    drug_to_target_tokenids[drug] = tids
    all_target_tokenids.update(tids)

drug_has_targets = {d: (len(tids) > 0) for d, tids in drug_to_target_tokenids.items()}
print(f"drugs total={len(drug_to_target_tokenids)}, with>=1 target={sum(drug_has_targets.values())}")


# =========================================================
# 3) HVG from DMSO
# =========================================================
def compute_hvg_token_ids_from_dmso(dmso_h5ad_path: str, control_drug: str, HVG_K: int, ensg_to_token_id: dict):
    ad = sc.read_h5ad(dmso_h5ad_path)
    obs = ad.obs
    m = (obs["drug"].astype(str).values == control_drug)
    idx = np.where(m)[0]
    if idx.size == 0:
        raise ValueError(f"DMSO adata 안에 control drug({control_drug})가 없습니다.")

    X = ad.X.tocsr() if sparse.issparse(ad.X) else sparse.csr_matrix(ad.X)
    Xc = X[idx]

    mean = np.asarray(Xc.mean(axis=0)).ravel()
    mean2 = np.asarray(Xc.multiply(Xc).mean(axis=0)).ravel()
    var = (mean2 - mean**2).astype(np.float32)

    ensgs = ad.var_names.astype(str).tolist()

    token_ids = []
    vars_ = []
    for j, ensg in enumerate(ensgs):
        tid = ensg_to_token_id.get(ensg, None)
        if tid is None:
            continue
        token_ids.append(int(tid))
        vars_.append(float(var[j]))

    token_ids = np.asarray(token_ids, dtype=np.int64)
    vars_ = np.asarray(vars_, dtype=np.float32)

    if token_ids.size == 0:
        raise ValueError("DMSO var_names와 gene_metadata의 ENSG mapping이 거의 안 맞습니다.")

    k = min(int(HVG_K), token_ids.size)
    top = np.argpartition(-vars_, k-1)[:k]
    return set(token_ids[top].tolist())

hvg_token_ids = compute_hvg_token_ids_from_dmso(
    dmso_h5ad_path=DMSO_PATH,
    control_drug=CONTROL_DRUG,
    HVG_K=HVG_K,
    ensg_to_token_id=ensg_to_token_id,
)
print("HVG token_ids:", len(hvg_token_ids))

# INPUT subset = HVG ∪ TARGETS
subset_token_ids = sorted(set(hvg_token_ids) | set(all_target_tokenids))
M_SUB = len(subset_token_ids)
print("SUBSET (HVG ∪ TARGETS) size:", M_SUB)

# OUTPUT target-only = TARGETS only
target_token_ids = sorted(set(all_target_tokenids))
M_TGT = len(target_token_ids)
print("TARGET-ONLY size:", M_TGT)

old_tid_to_subid = {tid: i for i, tid in enumerate(subset_token_ids)}
old_tid_to_tgtid = {tid: i for i, tid in enumerate(target_token_ids)}


# =========================================================
# 4) subset vocab + LUT (fast mapping)
# =========================================================
SPECIAL_TOKENS = ["[PAD]", "[CLS]", "[DRUG]", "[TARGET]", "[CELL]", "[MASK]"]
local_token_to_id = {tok: i for i, tok in enumerate(SPECIAL_TOKENS)}
N_SPECIAL = len(SPECIAL_TOKENS)

VOCAB_SIZE = N_SPECIAL + M_SUB
PAD_ID = local_token_to_id["[PAD]"]
CLS_ID = local_token_to_id["[CLS]"]
CELLTOK_ID = local_token_to_id["[CELL]"]

print("VOCAB_SIZE:", VOCAB_SIZE)

old_tid_to_vocab_lut = np.full((N_GENES,), -1, dtype=np.int64)
for sid, old_tid in enumerate(subset_token_ids):
    if 0 <= old_tid < N_GENES:
        old_tid_to_vocab_lut[old_tid] = N_SPECIAL + sid

subset_token_ids_np = np.asarray(subset_token_ids, dtype=np.int64)


# =========================================================
# 5) y_targets (drug -> TARGET-ONLY multi-hot)
# =========================================================
drug_to_target_vec_tgt = {}
for d, tids in drug_to_target_tokenids.items():
    vec = np.zeros(M_TGT, dtype=np.float32)
    for tid in tids:
        j = old_tid_to_tgtid.get(int(tid), None)
        if j is not None:
            vec[j] = 1.0
    drug_to_target_vec_tgt[d] = vec

n_nonzero = sum(float(v.sum()) > 0 for v in drug_to_target_vec_tgt.values())
print("drugs with>=1 target (target-only vec):", n_nonzero)


# =========================================================
# 6) SMILES embeddings
# =========================================================
obj = torch.load(SMILES_EMB_PATH, map_location="cpu")
assert isinstance(obj, dict) and "drug" in obj and "emb" in obj

drug_list_saved = [str(d) for d in obj["drug"]]
emb_matrix = obj["emb"].to(dtype=torch.float32).cpu().numpy()
SMILES_DIM = int(emb_matrix.shape[1])

drug_to_smiles_np_raw = {d: emb_matrix[i].astype(np.float32, copy=False) for i, d in enumerate(drug_list_saved)}

drug_names = drug_meta_df["drug"].astype(str).tolist()
drug_to_smiles_np = {}
missing = 0
for d in drug_names:
    v = drug_to_smiles_np_raw.get(d, None)
    if v is None:
        drug_to_smiles_np[d] = np.zeros((SMILES_DIM,), dtype=np.float32)
        missing += 1
    else:
        drug_to_smiles_np[d] = v
print(f"✅ SMILES aligned | missing={missing}/{len(drug_names)}")

drug2id = {d: i for i, d in enumerate(sorted(set(drug_names)))}


# =========================================================
# 7) DMSO baselines (gene-space)
# =========================================================
def build_dmso_baselines_gene_space(dmso_h5ad_path: str, control_drug: str, N_GENES: int, ensg_to_token_id: dict):
    adata = sc.read_h5ad(dmso_h5ad_path)
    obs = adata.obs
    X = adata.X.tocsr() if sparse.issparse(adata.X) else sparse.csr_matrix(adata.X)

    m = (obs["drug"].astype(str).values == control_drug)
    idx = np.where(m)[0]
    if idx.size == 0:
        raise ValueError(f"DMSO adata 안에 control drug({control_drug})가 없습니다.")

    ensgs = adata.var_names.astype(str).tolist()

    token_ids, cols = [], []
    for j, ensg in enumerate(ensgs):
        tid = ensg_to_token_id.get(ensg, None)
        if tid is None:
            continue
        token_ids.append(int(tid))
        cols.append(j)

    token_ids = np.asarray(token_ids, dtype=np.int64)
    cols = np.asarray(cols, dtype=np.int64)

    Xc = X[idx][:, cols]
    mean_global_sub = np.asarray(Xc.mean(axis=0)).ravel().astype(np.float32)

    baseline_global = np.zeros(N_GENES, dtype=np.float32)
    baseline_global[token_ids] = mean_global_sub

    baseline_by_cl = {}
    cl_values = obs["cell_line_id"].astype(str).values
    for cl in np.unique(cl_values):
        cl_idx = np.where(m & (cl_values == cl))[0]
        if cl_idx.size == 0:
            continue
        Xcl = X[cl_idx][:, cols]
        mean_cl_sub = np.asarray(Xcl.mean(axis=0)).ravel().astype(np.float32)
        v = np.zeros(N_GENES, dtype=np.float32)
        v[token_ids] = mean_cl_sub
        baseline_by_cl[str(cl)] = v

    return baseline_global, baseline_by_cl

baseline_global, baseline_by_cl = build_dmso_baselines_gene_space(
    dmso_h5ad_path=DMSO_PATH,
    control_drug=CONTROL_DRUG,
    N_GENES=N_GENES,
    ensg_to_token_id=ensg_to_token_id,
)
print("baseline_global:", baseline_global.shape, "baseline_by_cl:", len(baseline_by_cl))


# =========================================================
# 8) split (drug, cell_line) pairs + weights
# =========================================================
DRUG_COL, CELL_COL, N_COL = "drug", "cell_line_id", "n_cells"
MIN_TRAIN = 1000
TEST_SIZE = 0.1

counts = pd.read_csv(COUNTS_CSV)
counts[DRUG_COL] = counts[DRUG_COL].astype(str)
counts[CELL_COL] = counts[CELL_COL].astype(str)

train_pool = counts[counts[N_COL] >= MIN_TRAIN].copy()
pairs_df = train_pool[[DRUG_COL, CELL_COL]].drop_duplicates()

pairs_df = pairs_df[pairs_df[DRUG_COL] != CONTROL_DRUG]
pairs_df = pairs_df[pairs_df[DRUG_COL].map(lambda d: drug_has_targets.get(str(d), False))]

train_df, val_df = train_test_split(
    pairs_df,
    test_size=TEST_SIZE,
    random_state=SEED,
    stratify=pairs_df[DRUG_COL],
)

train_pairs = list(zip(train_df[DRUG_COL], train_df[CELL_COL]))
val_pairs   = list(zip(val_df[DRUG_COL],   val_df[CELL_COL]))

print("train pairs:", len(train_pairs))
print("val pairs:", len(val_pairs))

def make_pair_weights_from_counts(counts_df, pairs, drug_col="drug", cell_col="cell_line_id", n_col="n_cells",
                                  mode="inv_sqrt", eps=1.0):
    tmp = counts_df[[drug_col, cell_col, n_col]].copy()
    tmp[drug_col] = tmp[drug_col].astype(str)
    tmp[cell_col] = tmp[cell_col].astype(str)

    pair2n = {(d, c): int(n) for d, c, n in tmp.values}

    w = []
    for p in pairs:
        n = pair2n.get(p, 0)
        if mode == "inv":
            ww = 1.0 / (n + eps)
        elif mode == "inv_log":
            ww = 1.0 / np.log1p(n + eps)
        else:
            ww = 1.0 / np.sqrt(n + eps)
        w.append(float(ww))

    w = np.asarray(w, dtype=np.float64)
    w = np.clip(w, 0.0, None)
    w = w / (w.sum() + 1e-12)
    return w

w_train = make_pair_weights_from_counts(counts, train_pairs, mode="inv_sqrt")
w_val   = make_pair_weights_from_counts(counts, val_pairs,   mode="inv_sqrt")


# =========================================================
# 9) parquet row-group indexing
# =========================================================
PARQUET_FILES = sorted(glob.glob(os.path.join(PARQUET_DIR, "**", "*.parquet"), recursive=True))
print("parquet files found:", len(PARQUET_FILES))

def build_pair_to_locations(parquet_files, valid_pairs_set, drug_col="drug", cell_col="cell_line_id"):
    out = defaultdict(list)
    for f in tqdm(parquet_files, desc="Index parquet row-groups", dynamic_ncols=True):
        pf = pq.ParquetFile(f)
        for rg in range(pf.num_row_groups):
            tbl = pf.read_row_group(rg, columns=[drug_col, cell_col])
            df = tbl.to_pandas()
            pairs_here = set(zip(df[drug_col].astype(str), df[cell_col].astype(str)))
            inter = pairs_here.intersection(valid_pairs_set)
            for p in inter:
                out[p].append((f, rg))
    return out

valid_pairs_set = set(train_pairs) | set(val_pairs)
pair_to_locations = build_pair_to_locations(PARQUET_FILES, valid_pairs_set, drug_col=DRUG_COL, cell_col=CELL_COL)
print("indexed pairs:", len(pair_to_locations))


# =========================================================
# 10) Dataset (drop first genes/expr token + target-only labels)
# =========================================================
class TahoeFPParquetDatasetMultiPair(torch.utils.data.IterableDataset):
    def __init__(
        self,
        pair_to_locations,
        pairs,
        baseline_global,
        baseline_by_cellline,
        drug_to_target_vec_target_only,   # (M_TGT,)
        drug2id,
        drug_to_smiles_np,
        n_genes_full,
        steps,
        max_seq_len=256,
        batch_size=128,
        pairs_per_batch=16,
        control_drug="DMSO_TF",
        pad_id=0,
        cls_id=1,
        celltok_id=4,
        cell_line2id=None,
        unk_cell_id=0,
        pair_weights=None,
        seed=42,
        drug_col="drug",
        cell_col="cell_line_id",
        genes_col="genes",
        expr_col="expressions",
        cap_per_pair_in_rg=None,
        max_tries_per_pair=20,
        invalid_global_gene_tids=(1, 2),
        subset_token_ids_np=None,
        old_tid_to_vocab_lut=None,
        m_tgt: int = 0,
        drop_first_gene_token: bool = True,
    ):
        super().__init__()
        self.pair_to_locations = pair_to_locations
        self.pairs = list(pairs)

        self.baseline_global = np.asarray(baseline_global, dtype=np.float32)
        self.baseline_by_cellline = baseline_by_cellline or {}

        self.drug_to_target_vec_target_only = drug_to_target_vec_target_only
        self.drug2id = drug2id
        self.drug_to_smiles_np = drug_to_smiles_np

        self.n_genes_full = int(n_genes_full)
        self.steps = int(steps)
        self.max_seq_len = int(max_seq_len)
        self.batch_size = int(batch_size)

        self.pairs_per_batch = int(pairs_per_batch)
        assert self.batch_size % self.pairs_per_batch == 0
        self.cells_per_pair = self.batch_size // self.pairs_per_batch

        self.control_drug = str(control_drug)
        self.pad_id = int(pad_id)
        self.cls_id = int(cls_id)

        self.celltok_id = int(celltok_id)
        self.cell_line2id = cell_line2id or {}
        self.unk_cell_id = int(unk_cell_id)

        self.drug_col = drug_col
        self.cell_col = cell_col
        self.genes_col = genes_col
        self.expr_col = expr_col

        self.cap_per_pair_in_rg = cap_per_pair_in_rg
        self.max_tries_per_pair = int(max_tries_per_pair)
        self.seed = int(seed)

        any_vec = next(iter(self.drug_to_smiles_np.values()))
        self.smiles_dim = int(any_vec.shape[-1])

        self.invalid_global_gene_tids = np.asarray(list(set(int(x) for x in invalid_global_gene_tids)), dtype=np.int64)

        self.m_tgt = int(m_tgt)
        assert self.m_tgt > 0

        self.drop_first_gene_token = bool(drop_first_gene_token)

        # weights
        if pair_weights is None:
            self.pair_weights = None
        else:
            w = np.asarray(pair_weights, dtype=np.float64)
            assert len(w) == len(self.pairs)
            w = np.clip(w, 0.0, None)
            w = w / (w.sum() + 1e-12)
            self.pair_weights = w

        self.subset_token_ids_np = subset_token_ids_np
        self.old_tid_to_vocab_lut = old_tid_to_vocab_lut

        self._pf_cache = {}

    def _get_pf(self, file_path):
        pf = self._pf_cache.get(file_path, None)
        if pf is None:
            pf = pq.ParquetFile(file_path)
            self._pf_cache[file_path] = pf
        return pf

    def _read_row_group_df(self, file_path, rg_id, columns):
        pf = self._get_pf(file_path)
        return pf.read_row_group(rg_id, columns=columns).to_pandas()

    def _prepare_sparse_sorted(self, genes, expr):
        idx = np.asarray(genes, dtype=np.int64)
        val = np.asarray(expr, dtype=np.float32)

        # ✅ drop the first "service" element (genes[0], expr[0])
        if self.drop_first_gene_token:
            if idx.size > 0 and val.size > 0:
                L = min(idx.size, val.size)
                idx = idx[:L]
                val = val[:L]
                if L >= 1:
                    idx = idx[1:]
                    val = val[1:]

        if idx.size == 0 or val.size == 0:
            return np.asarray([], dtype=np.int64), np.asarray([], dtype=np.float32)

        L = min(idx.size, val.size)
        idx = idx[:L]
        val = val[:L]

        if self.invalid_global_gene_tids.size > 0:
            m_bad = np.isin(idx, self.invalid_global_gene_tids, assume_unique=False)
            if m_bad.any():
                keep = ~m_bad
                idx = idx[keep]
                val = val[keep]
                if idx.size == 0:
                    return idx, val

        m = (idx >= 0) & (idx < self.n_genes_full)
        idx = idx[m]
        val = val[m]
        if idx.size == 0:
            return idx, val

        order = np.argsort(idx)
        return idx[order], val[order]

    def _fill_one_row(self, row_genes, row_expr, baseline_vec, input_ids_row, values_row, attn_row):
        idx_sorted, val_sorted = self._prepare_sparse_sorted(row_genes, row_expr)
        if idx_sorted.size == 0:
            return

        delta = val_sorted - baseline_vec[idx_sorted]

        mask_sub = np.isin(idx_sorted, self.subset_token_ids_np, assume_unique=False)
        if not mask_sub.any():
            return

        idx_sorted = idx_sorted[mask_sub]
        delta = delta[mask_sub]
        if idx_sorted.size == 0:
            return

        k = min(self.max_seq_len, idx_sorted.size)
        if k <= 0:
            return

        if k == idx_sorted.size:
            top_pos = np.argsort(-np.abs(delta))
        else:
            top_pos = np.argpartition(-np.abs(delta), k - 1)[:k]
            top_pos = top_pos[np.argsort(-np.abs(delta[top_pos]))]

        sel_token_ids = idx_sorted[top_pos]
        sel_delta = delta[top_pos]

        sel_vocab_ids = self.old_tid_to_vocab_lut[sel_token_ids]
        ok = sel_vocab_ids != -1
        if not ok.any():
            return

        sel_vocab_ids = sel_vocab_ids[ok]
        sel_delta = sel_delta[ok]

        L = min(self.max_seq_len, sel_vocab_ids.size)
        if L <= 0:
            return

        input_ids_row[2:2+L] = sel_vocab_ids[:L]
        values_row[2:2+L]    = sel_delta[:L]
        attn_row[2:2+L]      = 1

    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        base_seed = self.seed if worker_info is None else (self.seed + worker_info.id)
        rng = np.random.default_rng(base_seed)

        pairs = self.pairs
        weights = self.pair_weights

        cols = [self.drug_col, self.cell_col, self.genes_col, self.expr_col]

        cnt = 0
        while True:
            if weights is None:
                chosen_idx = rng.integers(0, len(pairs), size=self.pairs_per_batch)
            else:
                chosen_idx = rng.choice(len(pairs), size=self.pairs_per_batch, replace=True, p=weights)

            chosen_pairs = [pairs[i] for i in chosen_idx]

            seq_len = 2 + self.max_seq_len
            input_ids = np.full((self.batch_size, seq_len), self.pad_id, dtype=np.int64)
            values    = np.zeros((self.batch_size, seq_len), dtype=np.float32)
            attn      = np.zeros((self.batch_size, seq_len), dtype=np.int64)

            input_ids[:, 0] = self.cls_id
            input_ids[:, 1] = self.celltok_id
            attn[:, 0:2] = 1

            y_batch = np.zeros((self.batch_size, self.m_tgt), dtype=np.float32)
            smiles_batch = np.zeros((self.batch_size, self.smiles_dim), dtype=np.float32)
            drug_id_batch = np.zeros((self.batch_size,), dtype=np.int64)
            cell_id_batch = np.zeros((self.batch_size,), dtype=np.int64)

            row_ptr = 0
            built_any = False

            for (drug_name, cell_line) in chosen_pairs:
                drug_name = str(drug_name)
                cell_line = str(cell_line)

                if drug_name == self.control_drug:
                    continue

                y_vec = self.drug_to_target_vec_target_only.get(drug_name, None)
                if y_vec is None or float(np.sum(y_vec)) <= 0.0:
                    continue

                locs = self.pair_to_locations.get((drug_name, cell_line), [])
                if not locs:
                    continue

                baseline = self.baseline_by_cellline.get(cell_line, self.baseline_global)

                sm_vec = self.drug_to_smiles_np.get(drug_name, None)
                if sm_vec is None:
                    sm_vec = np.zeros((self.smiles_dim,), dtype=np.float32)

                did = int(self.drug2id.get(drug_name, 0))
                cid = int(self.cell_line2id.get(cell_line, self.unk_cell_id))

                for _ in range(self.max_tries_per_pair):
                    fpath, rg_id = locs[rng.integers(0, len(locs))]
                    df = self._read_row_group_df(fpath, rg_id, columns=cols)

                    df = df[(df[self.drug_col].astype(str) == drug_name) &
                            (df[self.cell_col].astype(str) == cell_line)]
                    if len(df) == 0:
                        continue

                    if self.cap_per_pair_in_rg is not None and len(df) > self.cap_per_pair_in_rg:
                        df = df.sample(self.cap_per_pair_in_rg, replace=False, random_state=None)

                    replace = len(df) < self.cells_per_pair
                    df = df.sample(self.cells_per_pair, replace=replace, random_state=None)

                    for r in df.itertuples(index=False):
                        if row_ptr >= self.batch_size:
                            break

                        y_batch[row_ptr] = y_vec
                        smiles_batch[row_ptr] = sm_vec
                        drug_id_batch[row_ptr] = did
                        cell_id_batch[row_ptr] = cid

                        genes = getattr(r, self.genes_col)
                        expr  = getattr(r, self.expr_col)

                        self._fill_one_row(
                            genes, expr, baseline,
                            input_ids[row_ptr], values[row_ptr], attn[row_ptr]
                        )
                        row_ptr += 1

                    built_any = True
                    break

                if row_ptr >= self.batch_size:
                    break

            if not built_any:
                continue

            if row_ptr < self.batch_size:
                fill = self.batch_size - row_ptr
                input_ids[row_ptr:] = input_ids[:fill]
                values[row_ptr:]    = values[:fill]
                attn[row_ptr:]      = attn[:fill]
                y_batch[row_ptr:]   = y_batch[:fill]
                smiles_batch[row_ptr:] = smiles_batch[:fill]
                drug_id_batch[row_ptr:] = drug_id_batch[:fill]
                cell_id_batch[row_ptr:] = cell_id_batch[:fill]

            yield {
                "input_ids": torch.tensor(input_ids, dtype=torch.long),
                "values": torch.tensor(values, dtype=torch.float32),
                "attention_mask": torch.tensor(attn, dtype=torch.long),
                "y_targets": torch.tensor(y_batch, dtype=torch.float32),   # (B, M_TGT)
                "smiles_emb": torch.tensor(smiles_batch, dtype=torch.float32),
                "drug_id": torch.tensor(drug_id_batch, dtype=torch.long),
                "cell_id": torch.tensor(cell_id_batch, dtype=torch.long),
            }

            cnt += 1
            if cnt >= self.steps:
                return


# =========================================================
# 10.1) cell2id mapping + loaders
# =========================================================
cell2id_df = pd.read_csv(CELL2ID_CSV)
cell2id_df["cell_line_id"] = cell2id_df["cell_line_id"].astype(str)
cell2id_df["cell_id"] = cell2id_df["cell_id"].astype(int)

cell_line2id = dict(zip(cell2id_df["cell_line_id"], cell2id_df["cell_id"]))
NUM_CELL_LINE = int(cell2id_df["cell_id"].max()) + 1
print("loaded cell2id:", len(cell_line2id), "NUM_CELL_LINE:", NUM_CELL_LINE)

CAP_PER_PAIR_IN_RG = None

train_ds = TahoeFPParquetDatasetMultiPair(
    pair_to_locations=pair_to_locations,
    pairs=train_pairs,
    baseline_global=baseline_global,
    baseline_by_cellline=baseline_by_cl,
    drug_to_target_vec_target_only=drug_to_target_vec_tgt,
    drug2id=drug2id,
    drug_to_smiles_np=drug_to_smiles_np,
    n_genes_full=N_GENES,
    steps=STEPS_PER_EPOCH,
    max_seq_len=MAX_SEQ_LEN,
    batch_size=BATCH_SIZE,
    pairs_per_batch=PAIRS_PER_BATCH,
    control_drug=CONTROL_DRUG,
    pad_id=PAD_ID,
    cls_id=CLS_ID,
    celltok_id=CELLTOK_ID,
    cell_line2id=cell_line2id,
    unk_cell_id=0,
    pair_weights=w_train,
    seed=SEED,
    cap_per_pair_in_rg=CAP_PER_PAIR_IN_RG,
    max_tries_per_pair=20,
    subset_token_ids_np=subset_token_ids_np,
    old_tid_to_vocab_lut=old_tid_to_vocab_lut,
    m_tgt=M_TGT,
    drop_first_gene_token=DROP_FIRST_GENE_TOKEN,
)

val_ds = TahoeFPParquetDatasetMultiPair(
    pair_to_locations=pair_to_locations,
    pairs=val_pairs,
    baseline_global=baseline_global,
    baseline_by_cellline=baseline_by_cl,
    drug_to_target_vec_target_only=drug_to_target_vec_tgt,
    drug2id=drug2id,
    drug_to_smiles_np=drug_to_smiles_np,
    n_genes_full=N_GENES,
    steps=VAL_STEPS,
    max_seq_len=MAX_SEQ_LEN,
    batch_size=BATCH_SIZE,
    pairs_per_batch=PAIRS_PER_BATCH,
    control_drug=CONTROL_DRUG,
    pad_id=PAD_ID,
    cls_id=CLS_ID,
    celltok_id=CELLTOK_ID,
    cell_line2id=cell_line2id,
    unk_cell_id=0,
    pair_weights=w_val,
    seed=SEED + 123,
    cap_per_pair_in_rg=CAP_PER_PAIR_IN_RG,
    max_tries_per_pair=20,
    subset_token_ids_np=subset_token_ids_np,
    old_tid_to_vocab_lut=old_tid_to_vocab_lut,
    m_tgt=M_TGT,
    drop_first_gene_token=DROP_FIRST_GENE_TOKEN,
)

train_loader = DataLoader(
    train_ds,
    batch_size=None,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=(NUM_WORKERS > 0),
    prefetch_factor=2 if NUM_WORKERS > 0 else None,
)
val_loader = DataLoader(
    val_ds,
    batch_size=None,
    num_workers=0,
    pin_memory=True
)


# =========================================================
# 11) Model
# =========================================================
class FPEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, num_layers, pad_id,
                 max_len: int, num_cell_lines: int, cell_pos: int = 1):
        super().__init__()
        self.token_emb  = nn.Embedding(vocab_size, d_model, padding_idx=pad_id)
        self.value_proj = nn.Linear(1, d_model)
        self.pos_emb = nn.Embedding(max_len, d_model)
        self.cell_line_emb = nn.Embedding(num_cell_lines, d_model)
        self.cell_pos = int(cell_pos)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=4*d_model,
            dropout=0.1,
            batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)

    def forward(self, input_ids, values, attention_mask, cell_line_id):
        B, L = input_ids.shape
        dev = input_ids.device

        x = self.token_emb(input_ids) + self.value_proj(values.unsqueeze(-1))
        pos = torch.arange(L, device=dev).unsqueeze(0).expand(B, L)
        x = x + self.pos_emb(pos)

        if cell_line_id is not None:
            x[:, self.cell_pos, :] = x[:, self.cell_pos, :] + self.cell_line_emb(cell_line_id.to(dev)).to(x.dtype)

        key_padding_mask = (attention_mask == 0)
        h = self.encoder(x, src_key_padding_mask=key_padding_mask)
        return h[:, 0, :]


class FPModelTied(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, num_layers, pad_id, smiles_dim,
                 max_len: int, num_cell_lines: int):
        super().__init__()
        self.encoder = FPEncoder(
            vocab_size=vocab_size,
            d_model=d_model,
            n_heads=n_heads,
            num_layers=num_layers,
            pad_id=pad_id,
            max_len=max_len,
            num_cell_lines=num_cell_lines,
            cell_pos=1,
        )
        self.proj = nn.Linear(d_model, d_model)
        self.smiles_head = nn.Sequential(
            nn.Linear(d_model, 4*d_model),
            nn.BatchNorm1d(4*d_model),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(4*d_model, smiles_dim),
            nn.LayerNorm(smiles_dim),
        )

    def gene_emb_subset(self):
        return self.encoder.token_emb.weight[N_SPECIAL:, :]  # (M_SUB, d)

    def forward(self, input_ids, values, attention_mask, cell_line_id, return_smiles=False):
        h_cls = self.encoder(input_ids, values, attention_mask, cell_line_id)
        v_pred = self.proj(h_cls)
        z_pred = self.smiles_head(h_cls)
        if return_smiles:
            return v_pred, z_pred
        return v_pred


D_MODEL = 256
N_HEADS = 8
N_LAYERS = 4

model = FPModelTied(
    vocab_size=VOCAB_SIZE,
    d_model=D_MODEL,
    n_heads=N_HEADS,
    num_layers=N_LAYERS,
    pad_id=PAD_ID,
    smiles_dim=SMILES_DIM,
    max_len=(2 + MAX_SEQ_LEN),
    num_cell_lines=NUM_CELL_LINE
).to(device)


# =========================================================
# 12) Load pretrained embeddings (subset + cell)
# =========================================================
def load_pretrained_subset_into_token_emb(token_emb: nn.Embedding, npy_path: str, device):
    W = np.load(npy_path)  # (N_GENES, d)
    Wt = torch.tensor(W, dtype=torch.float32, device=device)
    d = token_emb.weight.shape[1]
    if Wt.shape[1] != d:
        raise ValueError(f"d mismatch: npy={Wt.shape[1]} vs token_emb={d}")

    loaded = 0
    with torch.no_grad():
        for sid, old_tid in enumerate(subset_token_ids):
            vid = N_SPECIAL + sid
            if 0 <= old_tid < Wt.shape[0]:
                token_emb.weight[vid].copy_(Wt[int(old_tid)])
                loaded += 1
    print(f"✅ token_emb loaded: {loaded}/{len(subset_token_ids)}")

load_pretrained_subset_into_token_emb(model.encoder.token_emb, PRETRAINED_GENE_NPY, device=device)

def load_pretrained_cell_embeddings(cell_emb_layer, cell_emb_npy, device):
    W = np.load(cell_emb_npy)
    if W.shape != tuple(cell_emb_layer.weight.shape):
        raise ValueError(f"cell emb shape mismatch: npy={W.shape} vs layer={tuple(cell_emb_layer.weight.shape)}")
    with torch.no_grad():
        cell_emb_layer.weight.copy_(torch.tensor(W, device=device, dtype=cell_emb_layer.weight.dtype))
    print(f"✅ loaded cell embeddings: {W.shape}")

load_pretrained_cell_embeddings(model.encoder.cell_line_emb, CELL_EMB_NPY, device)


# =========================================================
# 12.1) TARGET-ONLY embedding bank indices inside SUBSET
# =========================================================
target_sub_ids = [old_tid_to_subid[tid] for tid in target_token_ids]
target_sub_ids = torch.tensor(target_sub_ids, dtype=torch.long, device=device)
print("target_sub_ids:", tuple(target_sub_ids.shape))


# =========================================================
# 13) pos_weight (TARGET-ONLY)
# =========================================================
Y_list = [v for v in drug_to_target_vec_tgt.values() if float(np.sum(v)) > 0]
Y_all = np.stack(Y_list, axis=0)

pos = Y_all.sum(axis=0)
neg = Y_all.shape[0] - pos
pw = neg / (pos + 1e-6)

if POSW_MODE == "clip":
    pw = np.minimum(pw, POSW_MAX)
elif POSW_MODE == "sqrt_clip":
    pw = np.sqrt(pw)
    pw = np.minimum(pw, POSW_MAX)
elif POSW_MODE == "none":
    pw = np.ones_like(pw, dtype=np.float32)

pos_weight = torch.tensor(pw, dtype=torch.float32, device=device)
print("✅ pos_weight(target-only):", pos_weight.shape, "| max=", float(pos_weight.max().item()))


# =========================================================
# 14) Losses
# =========================================================
def info_nce_ranking_loss_multi_pos(
    v_pred: torch.Tensor,
    gene_emb: torch.Tensor,
    y_targets: torch.Tensor,
    num_neg: int = 256,
    num_pos: int = 8,
    tau: float = 0.1,
):
    device_ = v_pred.device
    B, _ = v_pred.shape
    losses = []

    # cosine geometry
    v_pred = F.normalize(v_pred, dim=1)
    gene_emb = F.normalize(gene_emb, dim=1)

    for i in range(B):
        pos_idx = (y_targets[i] > 0.5).nonzero(as_tuple=True)[0]
        if pos_idx.numel() == 0:
            continue

        neg_idx_all = (y_targets[i] < 0.5).nonzero(as_tuple=True)[0]
        if neg_idx_all.numel() == 0:
            continue

        if num_pos is not None and num_pos > 0 and pos_idx.numel() > num_pos:
            pos_idx = pos_idx[torch.randperm(pos_idx.numel(), device=device_)[:num_pos]]

        if neg_idx_all.numel() > num_neg:
            neg_idx = neg_idx_all[torch.randperm(neg_idx_all.numel(), device=device_)[:num_neg]]
        else:
            neg_idx = neg_idx_all

        pos_emb = gene_emb[pos_idx]
        neg_emb = gene_emb[neg_idx]
        cand_emb = torch.cat([pos_emb, neg_emb], dim=0)

        v = v_pred[i].unsqueeze(0)
        scores = (v @ cand_emb.T).squeeze(0) / tau  # cosine logits

        P = pos_emb.size(0)
        logits = scores.unsqueeze(0).repeat(P, 1)
        targets = torch.arange(P, device=device_, dtype=torch.long)
        losses.append(F.cross_entropy(logits, targets))

    if len(losses) == 0:
        return torch.tensor(0.0, device=device_)
    return torch.stack(losses).mean()


# ✅ FIXED: BCE now uses cosine logits (same geometry as eval/rank)
def bce_with_neg_sampling_cosine(
    pred_vec: torch.Tensor,
    y_targets: torch.Tensor,
    gene_emb: torch.Tensor,
    pos_weight_full: torch.Tensor,
    num_neg: int = 2048,
    pos_cap: int | None = None,
    tau_bce: float = 0.10,
):
    device_ = pred_vec.device
    B, _ = pred_vec.shape
    losses = []

    gene_emb = F.normalize(gene_emb, dim=1)

    for i in range(B):
        yi = y_targets[i]
        pos_idx = (yi > 0.5).nonzero(as_tuple=True)[0]
        if pos_idx.numel() == 0:
            continue

        if (pos_cap is not None) and (pos_idx.numel() > pos_cap):
            pos_idx = pos_idx[torch.randperm(pos_idx.numel(), device=device_)[:pos_cap]]

        neg_idx_all = (yi < 0.5).nonzero(as_tuple=True)[0]
        if neg_idx_all.numel() == 0:
            continue

        k = min(int(num_neg), neg_idx_all.numel())
        neg_idx = neg_idx_all[torch.randperm(neg_idx_all.numel(), device=device_)[:k]]

        idx = torch.cat([pos_idx, neg_idx], dim=0)

        v = F.normalize(pred_vec[i], dim=0)  # (d,)
        logits = (v @ gene_emb[idx].T) / tau_bce  # cosine logits

        y_sub = yi[idx]
        pw_sub = pos_weight_full[idx]

        losses.append(F.binary_cross_entropy_with_logits(logits, y_sub, pos_weight=pw_sub, reduction="mean"))

    if len(losses) == 0:
        return torch.tensor(0.0, device=device_)
    return torch.stack(losses).mean()


def combined_target_loss_neg_sampling_tied(
    pred_vec: torch.Tensor,
    y_targets: torch.Tensor,
    gene_emb: torch.Tensor,
    pos_weight: torch.Tensor,
    lambda_cos: float = 1.0,
    lambda_bce: float = 0.1,
    lambda_rank: float = 0.5,
    bce_num_neg: int = 2048,
    bce_pos_cap: int | None = None,
    rank_num_neg: int = 256,
    rank_num_pos: int = 8,
    tau_rank: float = 0.1,
    tau_bce: float = 0.10,
):
    device_ = pred_vec.device

    # cosine (direction) loss
    gene_emb_norm = F.normalize(gene_emb, dim=1)
    pred_norm = F.normalize(pred_vec, dim=1)

    true_vec = y_targets @ gene_emb_norm
    num_t = y_targets.sum(dim=1, keepdim=True)
    mask = (num_t > 0).squeeze(1)

    if mask.any():
        true_vec_pos = true_vec[mask] / (num_t[mask] + 1e-6)
        true_vec_pos = F.normalize(true_vec_pos, dim=1)
        pred_pos = pred_norm[mask]
        loss_cos = 1.0 - (pred_pos * true_vec_pos).sum(dim=1).mean()
    else:
        loss_cos = torch.tensor(0.0, device=device_)

    # BCE with cosine logits
    loss_bce = bce_with_neg_sampling_cosine(
        pred_vec=pred_vec,
        y_targets=y_targets,
        gene_emb=gene_emb,            # internally normalized
        pos_weight_full=pos_weight,
        num_neg=bce_num_neg,
        pos_cap=bce_pos_cap,
        tau_bce=tau_bce,
    )

    # rank (InfoNCE) with cosine logits
    loss_rank = info_nce_ranking_loss_multi_pos(
        v_pred=pred_vec,
        gene_emb=gene_emb,
        y_targets=y_targets,
        num_neg=rank_num_neg,
        num_pos=rank_num_pos,
        tau=tau_rank,
    )

    loss = lambda_cos * loss_cos + lambda_bce * loss_bce + lambda_rank * loss_rank
    return loss, loss_cos.detach(), loss_bce.detach(), loss_rank.detach()


def supervised_contrastive_loss_smiles(z_pred, z_true, drug_ids, tau: float = 0.07, remove_diagonal: bool = True):
    z_pred = F.normalize(z_pred, dim=1)
    z_true = F.normalize(z_true, dim=1)

    logits = (z_pred @ z_true.T) / tau
    labels = drug_ids.view(-1, 1)
    mask = torch.eq(labels, labels.T).float().to(z_pred.device)

    if remove_diagonal:
        mask.fill_diagonal_(0.0)

    logits = logits - logits.max(dim=1, keepdim=True).values.detach()
    log_prob = logits - torch.log(torch.exp(logits).sum(dim=1, keepdim=True) + 1e-6)

    pos_cnt = mask.sum(dim=1)
    valid = (pos_cnt > 0).float()

    mean_log_prob_pos = (mask * log_prob).sum(dim=1) / (pos_cnt + 1e-6)
    loss_per = -mean_log_prob_pos * valid
    return loss_per.sum() / (valid.sum() + 1e-6)

# ✅ NO MSE: cosine align only
def smiles_align_loss_cosine(z_pred: torch.Tensor, z_true: torch.Tensor):
    return 1.0 - F.cosine_similarity(z_pred, z_true, dim=-1).mean()

def smiles_mean_cosine(z_pred: torch.Tensor, z_true: torch.Tensor):
    return F.cosine_similarity(z_pred, z_true, dim=-1).mean()


# =========================================================
# 15) Eval (NO SMILES MSE)
# =========================================================
def compute_recall_precision_at_k(scores: torch.Tensor, y_true: torch.Tensor, k: int = 20):
    B, M = scores.shape
    kk = min(k, M)
    _, topk_idx = torch.topk(scores, k=kk, dim=1)

    recalls, precisions = [], []
    for i in range(B):
        true_labels = y_true[i]
        num_pos_ = true_labels.sum().item()
        if num_pos_ == 0:
            continue
        topk = topk_idx[i]
        num_pos_in_topk = true_labels[topk].sum().item()
        recalls.append(num_pos_in_topk / max(num_pos_, 1e-6))
        precisions.append(num_pos_in_topk / max(kk, 1))

    if len(recalls) == 0:
        return 0.0, 0.0
    return float(sum(recalls) / len(recalls)), float(sum(precisions) / len(precisions))

@torch.no_grad()
def evaluate_fp_targets_and_smiles(model, loader, device, target_sub_ids, k_list=(5,10), tau_smiles_eval=0.07):
    model.eval()

    gene_emb = model.gene_emb_subset()[target_sub_ids].to(device)  # (M_TGT, d)
    g_norm = gene_emb / (gene_emb.norm(dim=1, keepdim=True) + 1e-8)

    recall_sums = {k: 0.0 for k in k_list}
    prec_sums   = {k: 0.0 for k in k_list}
    counts_     = {k: 0   for k in k_list}

    smiles_supcon_sum = 0.0
    smiles_cos_sum = 0.0
    n_smiles = 0

    for batch in loader:
        input_ids   = batch["input_ids"].to(device, non_blocking=True)
        values      = batch["values"].to(device, non_blocking=True)
        attn        = batch["attention_mask"].to(device, non_blocking=True)
        y_targets   = batch["y_targets"].to(device, non_blocking=True)
        smiles_true = batch["smiles_emb"].to(device, non_blocking=True)
        drug_ids    = batch["drug_id"].to(device, non_blocking=True)
        cell_id     = batch["cell_id"].to(device, non_blocking=True)

        v_pred, z_pred = model(input_ids, values, attn, cell_line_id=cell_id, return_smiles=True)

        v_norm = v_pred / (v_pred.norm(dim=1, keepdim=True) + 1e-8)
        scores = v_norm @ g_norm.T

        for k in k_list:
            r, p = compute_recall_precision_at_k(scores, y_targets, k=k)
            recall_sums[k] += r
            prec_sums[k]   += p
            counts_[k]     += 1

        bs = smiles_true.size(0)
        supcon = supervised_contrastive_loss_smiles(z_pred, smiles_true, drug_ids, tau=tau_smiles_eval)
        smiles_supcon_sum += supcon.item() * bs

        cos = smiles_mean_cosine(z_pred, smiles_true)
        smiles_cos_sum += cos.item() * bs

        n_smiles += bs

    out = {}
    for k in k_list:
        out[f"Recall@{k}"] = recall_sums[k] / max(counts_[k], 1)
        out[f"Precision@{k}"] = prec_sums[k] / max(counts_[k], 1)

    out["SMILES_SupCon"] = smiles_supcon_sum / max(n_smiles, 1)
    out["SMILES_COS"]    = smiles_cos_sum / max(n_smiles, 1)
    return out


# =========================================================
# 16) Train loop (NO SMILES MSE)
# =========================================================
def infinite_loader(loader):
    while True:
        for b in loader:
            yield b

USE_AMP = (device.type == "cuda")
scaler = GradScaler(enabled=USE_AMP)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=0.01)

def train_one_epoch_fixed_steps(
    model,
    train_loader,
    device,
    steps_per_epoch,
    optimizer,
    scaler,
    target_sub_ids,
    pos_weight,
    log_every=50,
    grad_clip=1.0,
):
    model.train()
    it = infinite_loader(train_loader)

    running_total = 0.0
    running_tgt   = 0.0
    running_sm    = 0.0
    running_rank_last = 0.0
    n = 0

    pbar = tqdm(range(1, steps_per_epoch + 1), desc="Train", leave=True, dynamic_ncols=True)

    for step in pbar:
        batch = next(it)

        input_ids   = batch["input_ids"].to(device, non_blocking=True)
        values      = batch["values"].to(device, non_blocking=True)
        attn        = batch["attention_mask"].to(device, non_blocking=True)
        y_targets   = batch["y_targets"].to(device, non_blocking=True)
        smiles_true = batch["smiles_emb"].to(device, non_blocking=True)
        drug_ids    = batch["drug_id"].to(device, non_blocking=True)
        cell_id     = batch["cell_id"].to(device, non_blocking=True)

        bs = input_ids.size(0)
        n += bs

        optimizer.zero_grad(set_to_none=True)

        if USE_AMP:
            with autocast(enabled=True):
                v_pred, z_pred = model(input_ids, values, attn, cell_line_id=cell_id, return_smiles=True)

                gene_emb = model.gene_emb_subset()[target_sub_ids]  # (M_TGT, d)

                loss_targets, loss_cos_t, loss_bce_t, loss_rank_t = combined_target_loss_neg_sampling_tied(
                    pred_vec=v_pred,
                    y_targets=y_targets,
                    gene_emb=gene_emb,
                    pos_weight=pos_weight,
                    lambda_cos=lambda_cos,
                    lambda_bce=lambda_bce,
                    lambda_rank=lambda_rank,
                    bce_num_neg=bce_num_neg,
                    bce_pos_cap=bce_pos_cap,
                    rank_num_neg=rank_num_neg,
                    rank_num_pos=rank_num_pos,
                    tau_rank=tau_rank,
                    tau_bce=tau_bce,
                )

                # ✅ SMILES: SupCon + cosine-align only (NO MSE)
                loss_supcon = supervised_contrastive_loss_smiles(z_pred, smiles_true, drug_ids, tau=tau_smiles)
                loss_align  = smiles_align_loss_cosine(z_pred, smiles_true)
                loss_smiles = loss_supcon + alpha_align * loss_align

                loss = loss_targets + lambda_smiles * loss_smiles

            if not torch.isfinite(loss).all():
                continue

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)

            any_grad = any(p.grad is not None for p in model.parameters())
            if not any_grad:
                scaler.update()
                continue

            if grad_clip is not None and grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

            scaler.step(optimizer)
            scaler.update()

        else:
            v_pred, z_pred = model(input_ids, values, attn, cell_line_id=cell_id, return_smiles=True)
            gene_emb = model.gene_emb_subset()[target_sub_ids]

            loss_targets, loss_cos_t, loss_bce_t, loss_rank_t = combined_target_loss_neg_sampling_tied(
                pred_vec=v_pred,
                y_targets=y_targets,
                gene_emb=gene_emb,
                pos_weight=pos_weight,
                lambda_cos=lambda_cos,
                lambda_bce=lambda_bce,
                lambda_rank=lambda_rank,
                bce_num_neg=bce_num_neg,
                bce_pos_cap=bce_pos_cap,
                rank_num_neg=rank_num_neg,
                rank_num_pos=rank_num_pos,
                tau_rank=tau_rank,
                tau_bce=tau_bce,
            )

            loss_supcon = supervised_contrastive_loss_smiles(z_pred, smiles_true, drug_ids, tau=tau_smiles)
            loss_align  = smiles_align_loss_cosine(z_pred, smiles_true)
            loss_smiles = loss_supcon + alpha_align * loss_align
            loss = loss_targets + lambda_smiles * loss_smiles

            if not torch.isfinite(loss).all():
                continue

            loss.backward()
            if grad_clip is not None and grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()

        running_total += float(loss.item()) * bs
        running_tgt   += float(loss_targets.item()) * bs
        running_sm    += float(loss_smiles.item()) * bs
        running_rank_last = float(loss_rank_t.item())

        if step % log_every == 0:
            pos_in_batch = int((y_targets.sum(dim=1) > 0).sum().item())
            pbar.set_postfix({
                "loss": f"{running_total/max(n,1):.4f}",
                "tgt":  f"{running_tgt/max(n,1):.4f}",
                "sm":   f"{running_sm/max(n,1):.4f}",
                "rank(last)": f"{running_rank_last:.4f}",
                "pos_in_batch": f"{pos_in_batch}/{bs}",
            })

    return {
        "train_total":  running_total / max(n, 1),
        "train_target": running_tgt   / max(n, 1),
        "train_smiles": running_sm    / max(n, 1),
        "rank_last":    running_rank_last,
    }


# =========================================================
# 17) TRAIN
# =========================================================
print(">>> TRAIN START (TARGET-ONLY + DROP FIRST TOKEN + NO SMILES MSE + COSINE-BCE + STRONG RANK)")

for epoch in range(1, EPOCHS + 1):
    logs = train_one_epoch_fixed_steps(
        model=model,
        train_loader=train_loader,
        device=device,
        steps_per_epoch=STEPS_PER_EPOCH,
        optimizer=optimizer,
        scaler=scaler,
        target_sub_ids=target_sub_ids,
        pos_weight=pos_weight,
        log_every=50,
    )

    print(f"\n[Epoch {epoch}/{EPOCHS}] "
          f"train_total={logs['train_total']:.4f} | "
          f"train_target={logs['train_target']:.4f} | "
          f"train_smiles={logs['train_smiles']:.4f} | "
          f"rank_last={logs['rank_last']:.4f}")

    valid_metrics = evaluate_fp_targets_and_smiles(
        model=model,
        loader=val_loader,
        device=device,
        target_sub_ids=target_sub_ids,
        k_list=(5, 10),
        tau_smiles_eval=tau_smiles,
    )
    print("✅ VALID metrics:", valid_metrics)

    if (epoch % SAVE_EVERY == 0) or (epoch == EPOCHS):
        save_checkpoint(
            save_dir=CKPT_DIR,
            model=model,
            optimizer=optimizer,
            scaler=scaler,
            epoch=epoch,
            metrics={"train": logs, "valid": valid_metrics},
            prefix="fp_target_only",
        )

print(">>> DONE")


drugs total=379, with>=1 target=264
HVG token_ids: 4000
SUBSET (HVG ∪ TARGETS) size: 4184
TARGET-ONLY size: 278
VOCAB_SIZE: 4190
drugs with>=1 target (target-only vec): 264
✅ SMILES aligned | missing=0/379
baseline_global: (62713,) baseline_by_cl: 50
train pairs: 10505
val pairs: 1168
parquet files found: 3388


Index parquet row-groups: 100%|██████████| 3388/3388 [11:53<00:00,  4.75it/s]


indexed pairs: 11673
loaded cell2id: 50 NUM_CELL_LINE: 50


/data/aiffel/.cache/tmp/ipykernel_1229270/2194420474.py:1295: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=USE_AMP)


✅ token_emb loaded: 4184/4184
✅ loaded cell embeddings: (50, 256)
target_sub_ids: (278,)
✅ pos_weight(target-only): torch.Size([278]) | max= 1.0
>>> TRAIN START (TARGET-ONLY + DROP FIRST TOKEN + NO SMILES MSE + COSINE-BCE + STRONG RANK)


...


[Epoch 1/20] train_total=6.4862 | train_target=6.2293 | train_smiles=5.1380 | rank_last=5.4741
✅ VALID metrics: {'Recall@5': 0.10973453525641032, 'Precision@5': 0.029868750000000003, 'Recall@10': 0.16933756772741157, 'Precision@10': 0.0232609375, 'SMILES_SupCon': 4.930289658546448, 'SMILES_COS': 0.6480854507684708}


Train:   0%|          | 0/10000 [00:00<?, ?it/s]/data/aiffel/.cache/tmp/ipykernel_1229270/2194420474.py:1338: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True):
Train: 100%|██████████| 10000/10000 [1:21:03<00:00,  2.06it/s, loss=6.3893, tgt=6.1348, sm=5.0901, rank(last)=5.2580, pos_in_batch=128/128]



[Epoch 2/20] train_total=6.3893 | train_target=6.1348 | train_smiles=5.0901 | rank_last=5.2580
✅ VALID metrics: {'Recall@5': 0.11743688663766795, 'Precision@5': 0.031562499999999986, 'Recall@10': 0.18473509758470694, 'Precision@10': 0.024776562499999977, 'SMILES_SupCon': 4.9025642166137695, 'SMILES_COS': 0.6413964116573334}
💾 checkpoint saved: /data/aiffel/babayakga/checkpoints/fp_target_only_cosbce_rank1_drop1/fp_target_only_epoch002.pt


Train:   0%|          | 0/10000 [00:00<?, ?it/s]/data/aiffel/.cache/tmp/ipykernel_1229270/2194420474.py:1338: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True):
Train:   1%|▏         | 149/10000 [01:08<1:15:03,  2.19it/s, loss=6.3315, tgt=6.0784, sm=5.0627, rank(last)=5.2791, pos_in_batch=128/128]Traceback (most recent call last):
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/shutil.py", line 725, in rmtree
    _rmtree_safe_fd(fd, path, onerr

KeyboardInterrupt: 

## 2

In [1]:
# =========================================================
# FP (TARGET-ONLY) + ORGAN TOKEN + SMILES CLIP  (READY SCRIPT)
#
# ✅ Variant A scaling: raw counts -> log1p -> delta -> clip (+ optional asinh)
# ✅ Drop first "service" element from genes/expr
# ✅ Stable gene order after selection: sort by gene_id (deterministic)
# ✅ ORGAN token injected at fixed position [CLS][ORGAN] (NO cell-line embedding)
# ✅ Positional embeddings REMOVED (we agreed to remove pos emb)
# ✅ Targets: cosine vector loss + cosine-BCE (neg sampling) + InfoNCE rank
# ✅ SMILES: CLIP loss (+ optional cosine align), NO SupCon, NO MSE
# ✅ Batch is CLIP-safe: unique drug in batch (1 cell per drug)
# ✅ Skip missing/zero SMILES vectors
# ✅ OneCycleLR fixed for grad accumulation (total_steps = real updates)
# ✅ AMP + grad accumulation
#
# NOTE:
# - This script trains BOTH target head and SMILES CLIP jointly.
# - If you want SMILES-only warmup, set lambda_targets=0 for first N epochs.
# =========================================================

import os, glob, ast, random
from collections import defaultdict

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from tqdm import tqdm
import pyarrow.parquet as pq

import scanpy as sc
from scipy import sparse
from sklearn.model_selection import train_test_split


# =========================================================
# 0) PATHS / HYPERPARAMS
# =========================================================
PARQUET_DIR    = "/data/aiffel/data/Tahoe-100M/data"
GENE_META_PATH = "/data/aiffel/data/Tahoe-100M/metadata/gene_metadata.parquet"
DRUG_META_PATH = "/data/aiffel/data/Tahoe-100M/metadata/drug_metadata.parquet"
COUNTS_CSV     = "/data/aiffel/babayakga/making_data/aiffel/babayakga/making_data/tahoe_counts_per_drug_cell_line.csv"
DMSO_PATH      = "/data/aiffel/babayakga/outputs/dmso.h5ad"
CELL_LINE_META_PATH = "/data/aiffel/data/Tahoe-100M/metadata/cell_line_metadata.parquet"

SMILES_EMB_PATH       = "/data/aiffel/babayakga/smiles_emb/drug_smiles_emb_all1.pt"
PRETRAINED_GENE_NPY   = "/data/aiffel/babayakga/pretraining/checkpoints_with_cell/gene_embeddings.npy"  # optional

CONTROL_DRUG = "DMSO_TF"
SEED = 42

# sequence
MAX_SEQ_LEN = 256
HVG_K = 4000

# training
BATCH_SIZE  = 128
ACCUM_STEPS = 4
STEPS_PER_EPOCH = 7000
VAL_STEPS       = 300
EPOCHS          = 20

LR           = 1e-4
WEIGHT_DECAY = 0.01
MAX_GRAD_NORM = 1.0

# targets loss weights (from our conclusions)
lambda_cos  = 1.0
lambda_bce  = 0.05
lambda_rank = 1.0

# bce/rank knobs
bce_num_neg  = 2048
bce_pos_cap  = None
tau_bce      = 0.15

rank_num_neg = 1024
rank_num_pos = 4
tau_rank     = 0.15

# SMILES (CLIP)
lambda_smiles = 0.05
alpha_align   = 0.5
TAU_INIT      = 0.10  # initial tau (learnable temperature)

# overall mixing
lambda_targets = 1.0   # set 0.0 for SMILES-only warmup if you want

# data sampling
NUM_WORKERS = 4
MIN_TRAIN_CELLS_PER_PAIR = 1000
TEST_SIZE = 0.1

# Variant A scaling
USE_LOG1P_EXPR   = True
USE_ASINH_DELTA  = False
DELTA_CLIP_ABS   = 5.0

# misc
DROP_FIRST_GENE_TOKEN = True

# device
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass


# =========================================================
# 1) gene_metadata
# =========================================================
gene_md = pd.read_parquet(GENE_META_PATH).copy()
gene_md["gene_symbol"] = gene_md["gene_symbol"].astype(str)
gene_md["ensembl_id"]  = gene_md["ensembl_id"].astype(str)
gene_md["token_id"]    = gene_md["token_id"].astype(int)
gene_md = gene_md.sort_values("token_id").reset_index(drop=True)

N_GENES = int(gene_md["token_id"].max()) + 1
symbol_to_ensg_lower = dict(zip(gene_md["gene_symbol"].str.lower(), gene_md["ensembl_id"]))
ensg_to_token_id = dict(zip(gene_md["ensembl_id"].values, gene_md["token_id"].values))


# =========================================================
# 2) drug_metadata -> targets
# =========================================================
def parse_targets(x):
    if x is None:
        return []
    if isinstance(x, float) and np.isnan(x):
        return []
    if isinstance(x, (list, tuple)):
        return [str(t).strip() for t in x if str(t).strip()]
    if isinstance(x, str):
        s = x.strip()
        if (s.startswith("[") and s.endswith("]")) or (s.startswith("(") and s.endswith(")")):
            try:
                out = ast.literal_eval(s)
                if isinstance(out, (list, tuple)):
                    return [str(t).strip() for t in out if str(t).strip()]
            except Exception:
                pass
        for sep in [";", ","]:
            if sep in s:
                return [t.strip() for t in s.split(sep) if t.strip()]
        return [s] if s else []
    return [str(x).strip()]

drug_meta_df = pd.read_parquet(DRUG_META_PATH).copy()
drug_meta_df["drug"] = drug_meta_df["drug"].astype(str)

drug_to_target_tokenids = {}
all_target_tokenids = set()

for _, row in drug_meta_df.iterrows():
    drug = str(row["drug"])
    targets = parse_targets(row.get("targets", None))

    tids = []
    for t in targets:
        t = str(t).strip()
        if not t:
            continue
        if t.startswith("ENSG"):
            ensg = t
        else:
            ensg = symbol_to_ensg_lower.get(t.lower(), None)
        if ensg is None:
            continue
        tid = ensg_to_token_id.get(ensg, None)
        if tid is None:
            continue
        tid = int(tid)
        if 0 <= tid < N_GENES:
            tids.append(tid)

    tids = sorted(set(tids))
    drug_to_target_tokenids[drug] = tids
    all_target_tokenids.update(tids)

drug_has_targets = {d: (len(tids) > 0) for d, tids in drug_to_target_tokenids.items()}
print(f"[targets] drugs total={len(drug_to_target_tokenids)}, with>=1 target={sum(drug_has_targets.values())}")


# =========================================================
# 3) HVG from DMSO
# =========================================================
def compute_hvg_token_ids_from_dmso(dmso_h5ad_path: str, control_drug: str, HVG_K: int, ensg_to_token_id: dict):
    ad = sc.read_h5ad(dmso_h5ad_path)
    obs = ad.obs
    m = (obs["drug"].astype(str).values == str(control_drug))
    idx = np.where(m)[0]
    if idx.size == 0:
        raise ValueError(f"No DMSO cells found: control_drug={control_drug}")

    X = ad.X.tocsr() if sparse.issparse(ad.X) else sparse.csr_matrix(ad.X)
    Xc = X[idx]

    mean = np.asarray(Xc.mean(axis=0)).ravel()
    mean2 = np.asarray(Xc.multiply(Xc).mean(axis=0)).ravel()
    var = (mean2 - mean**2).astype(np.float32)

    ensgs = ad.var_names.astype(str).tolist()
    token_ids, vars_ = [], []
    for j, ensg in enumerate(ensgs):
        tid = ensg_to_token_id.get(ensg, None)
        if tid is None:
            continue
        token_ids.append(int(tid))
        vars_.append(float(var[j]))

    token_ids = np.asarray(token_ids, dtype=np.int64)
    vars_ = np.asarray(vars_, dtype=np.float32)
    if token_ids.size == 0:
        raise ValueError("ENSG mapping failed.")

    k = min(int(HVG_K), token_ids.size)
    top = np.argpartition(-vars_, k-1)[:k]
    return set(token_ids[top].tolist())

hvg_token_ids = compute_hvg_token_ids_from_dmso(DMSO_PATH, CONTROL_DRUG, HVG_K, ensg_to_token_id)
print("[HVG] token_ids:", len(hvg_token_ids))

# INPUT subset = HVG ∪ TARGETS
subset_token_ids = sorted(set(hvg_token_ids) | set(all_target_tokenids))
M_SUB = len(subset_token_ids)
print("[subset] HVG ∪ TARGETS size:", M_SUB)

# OUTPUT target-only = TARGETS only
target_token_ids = sorted(set(all_target_tokenids))
M_TGT = len(target_token_ids)
print("[target-only] size:", M_TGT)

old_tid_to_subid = {tid: i for i, tid in enumerate(subset_token_ids)}
old_tid_to_tgtid = {tid: i for i, tid in enumerate(target_token_ids)}


# =========================================================
# 4) vocab + LUT (include ORGAN token)
# =========================================================
SPECIAL_TOKENS = ["[PAD]", "[CLS]", "[ORGAN]", "[MASK]"]
local_token_to_id = {tok: i for i, tok in enumerate(SPECIAL_TOKENS)}
N_SPECIAL = len(SPECIAL_TOKENS)

VOCAB_SIZE = N_SPECIAL + M_SUB
PAD_ID   = local_token_to_id["[PAD]"]
CLS_ID   = local_token_to_id["[CLS]"]
ORGAN_TOK_ID = local_token_to_id["[ORGAN]"]

old_tid_to_vocab_lut = np.full((N_GENES,), -1, dtype=np.int64)
for sid, old_tid in enumerate(subset_token_ids):
    if 0 <= old_tid < N_GENES:
        old_tid_to_vocab_lut[old_tid] = N_SPECIAL + sid

subset_token_ids_np = np.asarray(subset_token_ids, dtype=np.int64)
print("[vocab] VOCAB_SIZE:", VOCAB_SIZE, "| N_SPECIAL:", N_SPECIAL)


# =========================================================
# 5) y_targets (drug -> TARGET-ONLY multi-hot)
# =========================================================
drug_to_target_vec_tgt = {}
for d, tids in drug_to_target_tokenids.items():
    vec = np.zeros(M_TGT, dtype=np.float32)
    for tid in tids:
        j = old_tid_to_tgtid.get(int(tid), None)
        if j is not None:
            vec[j] = 1.0
    drug_to_target_vec_tgt[d] = vec

print("[targets] drugs with>=1 target vec:", sum(float(v.sum()) > 0 for v in drug_to_target_vec_tgt.values()))


# =========================================================
# 6) organ mapping: cell_line_id -> organ_id   (FIX: UNK=0, organs start at 1)
# =========================================================
cl_meta = pd.read_parquet(CELL_LINE_META_PATH).copy()
cl_meta["Cell_ID_Cellosaur"] = cl_meta["Cell_ID_Cellosaur"].astype(str)
cl_meta["Organ"] = cl_meta["Organ"].astype(str)

cl_meta_small = cl_meta[["Cell_ID_Cellosaur", "Organ"]].dropna().drop_duplicates()
organs = sorted(cl_meta_small["Organ"].unique().tolist())

UNK_ORGAN_ID = 0
organ2id = {o: i+1 for i, o in enumerate(organs)}  # shift by 1
NUM_ORGANS = len(organs) + 1

cellline2organid = {
    str(cvcl): int(organ2id.get(str(org), UNK_ORGAN_ID))
    for cvcl, org in cl_meta_small.values
}
print("[organ] NUM_ORGANS:", NUM_ORGANS, "| mapped cell_lines:", len(cellline2organid), "| UNK_ORGAN_ID:", UNK_ORGAN_ID)


# =========================================================
# 7) SMILES embeddings + bank (for retrieval)
# =========================================================
obj = torch.load(SMILES_EMB_PATH, map_location="cpu")
assert isinstance(obj, dict) and "drug" in obj and "emb" in obj

drug_list_saved = [str(d) for d in obj["drug"]]
emb_matrix = obj["emb"].to(dtype=torch.float32).cpu().numpy()
SMILES_DIM = int(emb_matrix.shape[1])
drug_to_smiles_np_raw = {d: emb_matrix[i].astype(np.float32, copy=False) for i, d in enumerate(drug_list_saved)}

drug_names_all = sorted(set(drug_meta_df["drug"].astype(str).tolist()))
drug2id = {d: i for i, d in enumerate(drug_names_all)}

drug_to_smiles_np = {}
missing = 0
zeroed  = 0
for d in drug_names_all:
    v = drug_to_smiles_np_raw.get(d, None)
    if v is None:
        drug_to_smiles_np[d] = np.zeros((SMILES_DIM,), dtype=np.float32)
        missing += 1
    else:
        vv = v.astype(np.float32, copy=False)
        if np.abs(vv).sum() == 0.0:
            zeroed += 1
        drug_to_smiles_np[d] = vv
print(f"[SMILES] missing={missing}/{len(drug_names_all)} | zero_vec={zeroed}")

smiles_bank_np = np.stack([drug_to_smiles_np[d] for d in drug_names_all], axis=0).astype(np.float32)
print("[SMILES] bank:", smiles_bank_np.shape)


# =========================================================
# 8) DMSO baselines (Variant A: log1p BEFORE mean)
# =========================================================
def build_dmso_baselines_gene_space(dmso_h5ad_path: str, control_drug: str, N_GENES: int, ensg_to_token_id: dict, use_log1p: bool):
    adata = sc.read_h5ad(dmso_h5ad_path)
    obs = adata.obs
    X = adata.X.tocsr() if sparse.issparse(adata.X) else sparse.csr_matrix(adata.X)

    m = (obs["drug"].astype(str).values == str(control_drug))
    idx = np.where(m)[0]
    if idx.size == 0:
        raise ValueError("No DMSO cells.")

    ensgs = adata.var_names.astype(str).tolist()
    token_ids, cols = [], []
    for j, ensg in enumerate(ensgs):
        tid = ensg_to_token_id.get(ensg, None)
        if tid is None:
            continue
        token_ids.append(int(tid)); cols.append(j)

    token_ids = np.asarray(token_ids, dtype=np.int64)
    cols = np.asarray(cols, dtype=np.int64)

    Xc = X[idx][:, cols]
    if use_log1p:
        Xc = Xc.copy()
        Xc.data = np.log1p(np.clip(Xc.data, a_min=0.0, a_max=None))

    mean_global_sub = np.asarray(Xc.mean(axis=0)).ravel().astype(np.float32)
    baseline_global = np.zeros(N_GENES, dtype=np.float32)
    baseline_global[token_ids] = mean_global_sub

    baseline_by_cl = {}
    cl_values = obs["cell_line_id"].astype(str).values
    for cl in np.unique(cl_values):
        cl_idx = np.where(m & (cl_values == cl))[0]
        if cl_idx.size == 0:
            continue
        Xcl = X[cl_idx][:, cols]
        if use_log1p:
            Xcl = Xcl.copy()
            Xcl.data = np.log1p(np.clip(Xcl.data, a_min=0.0, a_max=None))
        mean_cl_sub = np.asarray(Xcl.mean(axis=0)).ravel().astype(np.float32)

        v = np.zeros(N_GENES, dtype=np.float32)
        v[token_ids] = mean_cl_sub
        baseline_by_cl[str(cl)] = v

    return baseline_global, baseline_by_cl

baseline_global, baseline_by_cl = build_dmso_baselines_gene_space(
    DMSO_PATH, CONTROL_DRUG, N_GENES, ensg_to_token_id, use_log1p=USE_LOG1P_EXPR
)
print("[baseline] global:", baseline_global.shape, "| by_cl:", len(baseline_by_cl))


# =========================================================
# 9) split (drug, cell_line) pairs + weights (filter: has targets)
# =========================================================
DRUG_COL, CELL_COL, N_COL = "drug", "cell_line_id", "n_cells"

counts = pd.read_csv(COUNTS_CSV)
counts[DRUG_COL] = counts[DRUG_COL].astype(str)
counts[CELL_COL] = counts[CELL_COL].astype(str)
counts[N_COL]    = counts[N_COL].astype(int)

pairs_df = counts[counts[N_COL] >= MIN_TRAIN_CELLS_PER_PAIR][[DRUG_COL, CELL_COL]].drop_duplicates().copy()
pairs_df = pairs_df[pairs_df[DRUG_COL] != str(CONTROL_DRUG)].copy()
pairs_df = pairs_df[pairs_df[DRUG_COL].map(lambda d: drug_has_targets.get(str(d), False))].copy()
pairs_df = pairs_df[pairs_df[DRUG_COL].isin(set(drug2id.keys()))].copy()

train_df, val_df = train_test_split(
    pairs_df,
    test_size=TEST_SIZE,
    random_state=SEED,
    stratify=pairs_df[DRUG_COL],
)

train_pairs = list(zip(train_df[DRUG_COL], train_df[CELL_COL]))
val_pairs   = list(zip(val_df[DRUG_COL],   val_df[CELL_COL]))
print("[split] train pairs:", len(train_pairs), "| val pairs:", len(val_pairs))

def make_pair_weights_from_counts(counts_df, pairs, mode="inv_sqrt", eps=1.0):
    pair2n = {(str(d), str(c)): int(n) for d, c, n in counts_df[[DRUG_COL, CELL_COL, N_COL]].values}
    w = []
    for p in pairs:
        n = pair2n.get((str(p[0]), str(p[1])), 0)
        if mode == "inv":
            ww = 1.0 / (n + eps)
        elif mode == "inv_log":
            ww = 1.0 / np.log1p(n + eps)
        else:
            ww = 1.0 / np.sqrt(n + eps)
        w.append(float(ww))
    w = np.asarray(w, dtype=np.float64)
    w = np.clip(w, 0.0, None)
    w = w / (w.sum() + 1e-12)
    return w

w_train = make_pair_weights_from_counts(counts, train_pairs, mode="inv_sqrt")
w_val   = make_pair_weights_from_counts(counts, val_pairs,   mode="inv_sqrt")


# =========================================================
# 10) parquet row-group indexing
# =========================================================
PARQUET_FILES = sorted(glob.glob(os.path.join(PARQUET_DIR, "**", "*.parquet"), recursive=True))
print("[parquet] files:", len(PARQUET_FILES))

def build_pair_to_locations(parquet_files, valid_pairs_set, drug_col="drug", cell_col="cell_line_id"):
    out = defaultdict(list)
    for f in tqdm(parquet_files, desc="Index parquet row-groups", dynamic_ncols=True):
        pf = pq.ParquetFile(f)
        for rg in range(pf.num_row_groups):
            tbl = pf.read_row_group(rg, columns=[drug_col, cell_col])
            df = tbl.to_pandas()
            pairs_here = set(zip(df[drug_col].astype(str), df[cell_col].astype(str)))
            inter = pairs_here.intersection(valid_pairs_set)
            for p in inter:
                out[p].append((f, rg))
    return out

valid_pairs_set = set(train_pairs) | set(val_pairs)
pair_to_locations = build_pair_to_locations(PARQUET_FILES, valid_pairs_set, drug_col=DRUG_COL, cell_col=CELL_COL)
print("[parquet] indexed pairs:", len(pair_to_locations))


# =========================================================
# 11) Dataset (unique drug batch + Variant A + stable ordering + organ_id)
# =========================================================
class TahoeFPParquetDataset_UniqueDrug(torch.utils.data.IterableDataset):
    def __init__(
        self,
        pair_to_locations,
        pairs,
        baseline_global,
        baseline_by_cellline,
        drug_to_target_vec_target_only,   # (M_TGT,)
        drug2id,
        drug_to_smiles_np,
        cellline2organid,
        unk_organ_id,
        n_genes_full,
        steps,
        max_seq_len=256,
        batch_size=128,
        control_drug="DMSO_TF",
        pad_id=0,
        cls_id=1,
        organtok_id=2,
        pair_weights=None,
        seed=42,
        drug_col="drug",
        cell_col="cell_line_id",
        genes_col="genes",
        expr_col="expressions",
        cap_per_pair_in_rg=None,
        max_tries_per_pair=20,
        invalid_global_gene_tids=(1, 2),
        subset_token_ids_np=None,
        old_tid_to_vocab_lut=None,
        m_tgt: int = 0,
        drop_first_gene_token: bool = True,
        use_log1p_expr: bool = True,
        use_asinh_delta: bool = False,
        delta_clip_abs: float = 5.0,
        stable_sort_selected_by_gene_id: bool = True,
    ):
        super().__init__()
        self.pair_to_locations = pair_to_locations
        self.pairs = list(pairs)
        self.baseline_global = np.asarray(baseline_global, dtype=np.float32)
        self.baseline_by_cellline = baseline_by_cellline or {}
        self.drug_to_target_vec_target_only = drug_to_target_vec_target_only
        self.drug2id = drug2id
        self.drug_to_smiles_np = drug_to_smiles_np
        self.cellline2organid = cellline2organid or {}
        self.unk_organ_id = int(unk_organ_id)

        self.n_genes_full = int(n_genes_full)
        self.steps = int(steps)
        self.max_seq_len = int(max_seq_len)
        self.batch_size = int(batch_size)

        self.control_drug = str(control_drug)
        self.pad_id = int(pad_id)
        self.cls_id = int(cls_id)
        self.organtok_id = int(organtok_id)

        self.drug_col = drug_col
        self.cell_col = cell_col
        self.genes_col = genes_col
        self.expr_col = expr_col

        self.cap_per_pair_in_rg = cap_per_pair_in_rg
        self.max_tries_per_pair = int(max_tries_per_pair)
        self.seed = int(seed)

        any_vec = next(iter(self.drug_to_smiles_np.values()))
        self.smiles_dim = int(any_vec.shape[-1])

        self.invalid_global_gene_tids = np.asarray(list(set(int(x) for x in invalid_global_gene_tids)), dtype=np.int64)

        self.m_tgt = int(m_tgt); assert self.m_tgt > 0
        self.drop_first_gene_token = bool(drop_first_gene_token)

        self.subset_token_ids_np = subset_token_ids_np
        self.old_tid_to_vocab_lut = old_tid_to_vocab_lut

        self.use_log1p_expr = bool(use_log1p_expr)
        self.use_asinh_delta = bool(use_asinh_delta)
        self.delta_clip_abs = float(delta_clip_abs)
        self.stable_sort_selected_by_gene_id = bool(stable_sort_selected_by_gene_id)

        if pair_weights is None:
            self.pair_weights = None
        else:
            w = np.asarray(pair_weights, dtype=np.float64)
            assert len(w) == len(self.pairs)
            w = np.clip(w, 0.0, None)
            w = w / (w.sum() + 1e-12)
            self.pair_weights = w

        self._pf_cache = {}

    def _get_pf(self, file_path):
        pf = self._pf_cache.get(file_path, None)
        if pf is None:
            pf = pq.ParquetFile(file_path)
            self._pf_cache[file_path] = pf
        return pf

    def _read_row_group_df(self, file_path, rg_id, columns):
        pf = self._get_pf(file_path)
        return pf.read_row_group(rg_id, columns=columns).to_pandas()

    def _scale_delta(self, delta: np.ndarray) -> np.ndarray:
        if self.delta_clip_abs and self.delta_clip_abs > 0:
            delta = np.clip(delta, -self.delta_clip_abs, self.delta_clip_abs)
        if self.use_asinh_delta:
            delta = np.arcsinh(delta)
        return delta.astype(np.float32, copy=False)

    def _prepare_sparse_sorted_drop0(self, genes, expr):
        if genes is None or expr is None:
            return np.asarray([], dtype=np.int64), np.asarray([], dtype=np.float32)

        idx = np.asarray(genes, dtype=np.int64)
        val = np.asarray(expr, dtype=np.float32)
        L = min(idx.size, val.size)
        idx = idx[:L]; val = val[:L]

        if self.drop_first_gene_token and L >= 1:
            idx = idx[1:]
            val = val[1:]

        if idx.size == 0:
            return idx, val

        # Variant A: log1p before delta
        if self.use_log1p_expr:
            val = np.log1p(np.clip(val, a_min=0.0, a_max=None))

        if self.invalid_global_gene_tids.size > 0:
            m_bad = np.isin(idx, self.invalid_global_gene_tids, assume_unique=False)
            if m_bad.any():
                keep = ~m_bad
                idx = idx[keep]; val = val[keep]
                if idx.size == 0:
                    return idx, val

        m = (idx >= 0) & (idx < self.n_genes_full)
        idx = idx[m]; val = val[m]
        if idx.size == 0:
            return idx, val

        order = np.argsort(idx)
        return idx[order], val[order]

    def _fill_one_row(self, row_genes, row_expr, baseline_vec, input_ids_row, values_row, attn_row):
        idx_sorted, val_sorted = self._prepare_sparse_sorted_drop0(row_genes, row_expr)
        if idx_sorted.size == 0:
            return False

        delta = val_sorted - baseline_vec[idx_sorted]
        delta = self._scale_delta(delta)

        mask_sub = np.isin(idx_sorted, self.subset_token_ids_np, assume_unique=False)
        if not mask_sub.any():
            return False

        idx_sub = idx_sorted[mask_sub]
        del_sub = delta[mask_sub]
        if idx_sub.size == 0:
            return False

        k = min(self.max_seq_len, idx_sub.size)
        top = np.argpartition(-np.abs(del_sub), k - 1)[:k]
        sel_tid = idx_sub[top]
        sel_del = del_sub[top]

        # stable order: sort by gene_id (deterministic)
        if self.stable_sort_selected_by_gene_id:
            o2 = np.argsort(sel_tid)
            sel_tid = sel_tid[o2]
            sel_del = sel_del[o2]

        sel_vid = self.old_tid_to_vocab_lut[sel_tid]
        ok = sel_vid != -1
        if not ok.any():
            return False

        sel_vid = sel_vid[ok]
        sel_del = sel_del[ok]

        L = min(self.max_seq_len, sel_vid.size)
        if L <= 0:
            return False

        # layout: [CLS][ORGAN] + genes...
        input_ids_row[2:2+L] = sel_vid[:L]
        values_row[2:2+L]    = sel_del[:L]
        attn_row[2:2+L]      = 1
        return True

    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        base_seed = self.seed if worker_info is None else (self.seed + worker_info.id)
        rng = np.random.default_rng(base_seed)

        pairs = self.pairs
        weights = self.pair_weights
        n_pairs = len(pairs)

        cols = [self.drug_col, self.cell_col, self.genes_col, self.expr_col]
        seq_len = 2 + self.max_seq_len

        cnt = 0
        while True:
            chosen = []
            seen_drugs = set()

            tries = 0
            while len(chosen) < self.batch_size and tries < 80:
                tries += 1
                draw = min(max(self.batch_size * 8, 512), max(n_pairs, 1))
                if weights is None:
                    cand_idx = rng.integers(0, n_pairs, size=draw)
                else:
                    cand_idx = rng.choice(n_pairs, size=draw, replace=True, p=weights)

                for ii in cand_idx:
                    drug_name, cell_line = pairs[int(ii)]
                    drug_name = str(drug_name); cell_line = str(cell_line)

                    if drug_name == self.control_drug:
                        continue
                    if drug_name in seen_drugs:
                        continue

                    y_vec = self.drug_to_target_vec_target_only.get(drug_name, None)
                    if y_vec is None or float(y_vec.sum()) <= 0.0:
                        continue

                    sm = self.drug_to_smiles_np.get(drug_name, None)
                    if sm is None or (not np.isfinite(sm).all()) or (np.abs(sm).sum() == 0.0):
                        continue

                    if not self.pair_to_locations.get((drug_name, cell_line), []):
                        continue

                    chosen.append((drug_name, cell_line))
                    seen_drugs.add(drug_name)
                    if len(chosen) >= self.batch_size:
                        break

            if len(chosen) < self.batch_size:
                continue

            input_ids = np.full((self.batch_size, seq_len), self.pad_id, dtype=np.int64)
            values    = np.zeros((self.batch_size, seq_len), dtype=np.float32)
            attn      = np.zeros((self.batch_size, seq_len), dtype=np.int64)

            input_ids[:, 0] = self.cls_id
            input_ids[:, 1] = self.organtok_id
            attn[:, 0:2] = 1

            y_batch       = np.zeros((self.batch_size, self.m_tgt), dtype=np.float32)
            smiles_batch  = np.zeros((self.batch_size, self.smiles_dim), dtype=np.float32)
            drug_id_batch = np.zeros((self.batch_size,), dtype=np.int64)
            organ_id_batch = np.zeros((self.batch_size,), dtype=np.int64)

            row_ptr = 0
            built_any = False

            for (drug_name, cell_line) in chosen:
                locs = self.pair_to_locations.get((drug_name, cell_line), [])
                if not locs:
                    continue

                baseline = self.baseline_by_cellline.get(cell_line, self.baseline_global)
                did = int(self.drug2id.get(drug_name, 0))
                oid = int(self.cellline2organid.get(cell_line, self.unk_organ_id))

                y_vec = self.drug_to_target_vec_target_only[drug_name]
                sm_vec = self.drug_to_smiles_np[drug_name]

                ok_row = False
                for _ in range(self.max_tries_per_pair):
                    fpath, rg_id = locs[rng.integers(0, len(locs))]
                    df = self._read_row_group_df(fpath, rg_id, columns=cols)

                    df = df[(df[self.drug_col].astype(str) == drug_name) &
                            (df[self.cell_col].astype(str) == cell_line)]
                    if len(df) == 0:
                        continue

                    r = df.sample(1, random_state=None).itertuples(index=False).__next__()

                    ok_row = self._fill_one_row(
                        getattr(r, self.genes_col),
                        getattr(r, self.expr_col),
                        baseline,
                        input_ids[row_ptr], values[row_ptr], attn[row_ptr]
                    )
                    if ok_row:
                        y_batch[row_ptr] = y_vec
                        smiles_batch[row_ptr] = sm_vec
                        drug_id_batch[row_ptr] = did
                        organ_id_batch[row_ptr] = oid
                        row_ptr += 1
                        built_any = True
                        break  # ✅ break only if ok_row

                if row_ptr >= self.batch_size:
                    break

            if not built_any:
                continue

            if row_ptr < self.batch_size:
                fill = self.batch_size - row_ptr
                input_ids[row_ptr:]      = input_ids[:fill]
                values[row_ptr:]         = values[:fill]
                attn[row_ptr:]           = attn[:fill]
                y_batch[row_ptr:]        = y_batch[:fill]
                smiles_batch[row_ptr:]   = smiles_batch[:fill]
                drug_id_batch[row_ptr:]  = drug_id_batch[:fill]
                organ_id_batch[row_ptr:] = organ_id_batch[:fill]

            yield {
                "input_ids": torch.tensor(input_ids, dtype=torch.long),
                "values": torch.tensor(values, dtype=torch.float32),
                "attention_mask": torch.tensor(attn, dtype=torch.long),
                "y_targets": torch.tensor(y_batch, dtype=torch.float32),
                "smiles_emb": torch.tensor(smiles_batch, dtype=torch.float32),
                "drug_id": torch.tensor(drug_id_batch, dtype=torch.long),
                "organ_id": torch.tensor(organ_id_batch, dtype=torch.long),
            }

            cnt += 1
            if cnt >= self.steps:
                return


train_ds = TahoeFPParquetDataset_UniqueDrug(
    pair_to_locations=pair_to_locations,
    pairs=train_pairs,
    baseline_global=baseline_global,
    baseline_by_cellline=baseline_by_cl,
    drug_to_target_vec_target_only=drug_to_target_vec_tgt,
    drug2id=drug2id,
    drug_to_smiles_np=drug_to_smiles_np,
    cellline2organid=cellline2organid,
    unk_organ_id=UNK_ORGAN_ID,
    n_genes_full=N_GENES,
    steps=STEPS_PER_EPOCH,
    max_seq_len=MAX_SEQ_LEN,
    batch_size=BATCH_SIZE,
    control_drug=CONTROL_DRUG,
    pad_id=PAD_ID,
    cls_id=CLS_ID,
    organtok_id=ORGAN_TOK_ID,
    pair_weights=w_train,
    seed=SEED,
    subset_token_ids_np=subset_token_ids_np,
    old_tid_to_vocab_lut=old_tid_to_vocab_lut,
    m_tgt=M_TGT,
    drop_first_gene_token=DROP_FIRST_GENE_TOKEN,
    use_log1p_expr=USE_LOG1P_EXPR,
    use_asinh_delta=USE_ASINH_DELTA,
    delta_clip_abs=DELTA_CLIP_ABS,
    stable_sort_selected_by_gene_id=True,
)

val_ds = TahoeFPParquetDataset_UniqueDrug(
    pair_to_locations=pair_to_locations,
    pairs=val_pairs,
    baseline_global=baseline_global,
    baseline_by_cellline=baseline_by_cl,
    drug_to_target_vec_target_only=drug_to_target_vec_tgt,
    drug2id=drug2id,
    drug_to_smiles_np=drug_to_smiles_np,
    cellline2organid=cellline2organid,
    unk_organ_id=UNK_ORGAN_ID,
    n_genes_full=N_GENES,
    steps=VAL_STEPS,
    max_seq_len=MAX_SEQ_LEN,
    batch_size=BATCH_SIZE,
    control_drug=CONTROL_DRUG,
    pad_id=PAD_ID,
    cls_id=CLS_ID,
    organtok_id=ORGAN_TOK_ID,
    pair_weights=w_val,
    seed=SEED + 123,
    subset_token_ids_np=subset_token_ids_np,
    old_tid_to_vocab_lut=old_tid_to_vocab_lut,
    m_tgt=M_TGT,
    drop_first_gene_token=DROP_FIRST_GENE_TOKEN,
    use_log1p_expr=USE_LOG1P_EXPR,
    use_asinh_delta=USE_ASINH_DELTA,
    delta_clip_abs=DELTA_CLIP_ABS,
    stable_sort_selected_by_gene_id=True,
)

train_loader = DataLoader(
    train_ds,
    batch_size=None,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=(NUM_WORKERS > 0),
    prefetch_factor=2 if NUM_WORKERS > 0 else None,
)
val_loader = DataLoader(
    val_ds,
    batch_size=None,
    num_workers=0,
    pin_memory=True,
)


# =========================================================
# 12) Model: ORGAN embedding + NO positional emb + learnable CLIP temperature
# =========================================================
class FPEncoderWithOrgan(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, num_layers, pad_id,
                 max_len: int, num_organs: int, organ_pos: int = 1, use_pos_emb: bool = False):
        super().__init__()
        self.token_emb  = nn.Embedding(vocab_size, d_model, padding_idx=pad_id)
        self.value_proj = nn.Linear(1, d_model)

        # ✅ we agreed to remove positional embeddings
        self.use_pos_emb = bool(use_pos_emb)
        if self.use_pos_emb:
            self.pos_emb = nn.Embedding(max_len, d_model)

        self.organ_emb = nn.Embedding(num_organs, d_model)
        self.organ_pos = int(organ_pos)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_heads, dim_feedforward=4*d_model,
            dropout=0.1, batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)

    def forward(self, input_ids, values, attention_mask, organ_id):
        B, L = input_ids.shape
        dev = input_ids.device

        x = self.token_emb(input_ids) + self.value_proj(values.unsqueeze(-1))

        if self.use_pos_emb:
            pos = torch.arange(L, device=dev).unsqueeze(0).expand(B, L)
            x = x + self.pos_emb(pos)

        if organ_id is not None:
            x[:, self.organ_pos, :] = x[:, self.organ_pos, :] + self.organ_emb(organ_id.to(dev)).to(x.dtype)

        key_padding_mask = (attention_mask == 0)
        h = self.encoder(x, src_key_padding_mask=key_padding_mask)
        return h[:, 0, :]


class FPModelTied_OrganCLIP(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, num_layers, pad_id, smiles_dim,
                 max_len: int, num_organs: int, n_special: int, tau_init: float = 0.10):
        super().__init__()
        self.n_special = int(n_special)

        self.encoder = FPEncoderWithOrgan(
            vocab_size=vocab_size,
            d_model=d_model,
            n_heads=n_heads,
            num_layers=num_layers,
            pad_id=pad_id,
            max_len=max_len,
            num_organs=num_organs,
            organ_pos=1,          # [CLS][ORGAN]
            use_pos_emb=False,    # ✅ removed
        )
        self.proj = nn.Linear(d_model, d_model)

        self.smiles_head = nn.Sequential(
            nn.Linear(d_model, 4*d_model),
            nn.BatchNorm1d(4*d_model),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(4*d_model, smiles_dim),
        )

        # CLIP logit scale: logits = (z1 @ z2.T) * exp(logit_scale)
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1.0 / float(tau_init)))

    def gene_emb_subset(self):
        return self.encoder.token_emb.weight[self.n_special:, :]  # (M_SUB, d)

    def get_tau(self):
        # tau = 1/exp(scale)
        return (1.0 / self.logit_scale.exp()).clamp(0.01, 0.5)

    def forward(self, input_ids, values, attention_mask, organ_id, return_smiles=False):
        h_cls = self.encoder(input_ids, values, attention_mask, organ_id=organ_id)
        v_pred = self.proj(h_cls)
        z_pred = self.smiles_head(h_cls)
        if return_smiles:
            return v_pred, z_pred
        return v_pred


D_MODEL = 256
N_HEADS = 8
N_LAYERS = 4

model = FPModelTied_OrganCLIP(
    vocab_size=VOCAB_SIZE,
    d_model=D_MODEL,
    n_heads=N_HEADS,
    num_layers=N_LAYERS,
    pad_id=PAD_ID,
    smiles_dim=SMILES_DIM,
    max_len=(2 + MAX_SEQ_LEN),
    num_organs=NUM_ORGANS,
    n_special=N_SPECIAL,
    tau_init=TAU_INIT,
).to(device)


# =========================================================
# 13) (optional) Load pretrained gene embeddings into subset token emb
# =========================================================
def load_pretrained_subset_into_token_emb(token_emb: nn.Embedding, npy_path: str, device):
    if (npy_path is None) or (not os.path.exists(npy_path)):
        print("⚠️ PRETRAINED_GENE_NPY not found. Skip loading.")
        return
    W = np.load(npy_path)  # (N_GENES, d)
    Wt = torch.tensor(W, dtype=torch.float32, device=device)
    d = token_emb.weight.shape[1]
    if Wt.shape[1] != d:
        raise ValueError(f"d mismatch: npy={Wt.shape[1]} vs token_emb={d}")

    loaded = 0
    with torch.no_grad():
        for sid, old_tid in enumerate(subset_token_ids):
            vid = N_SPECIAL + sid
            if 0 <= old_tid < Wt.shape[0]:
                token_emb.weight[vid].copy_(Wt[int(old_tid)])
                loaded += 1
    print(f"✅ token_emb loaded: {loaded}/{len(subset_token_ids)}")

load_pretrained_subset_into_token_emb(model.encoder.token_emb, PRETRAINED_GENE_NPY, device=device)


# =========================================================
# 14) target_sub_ids (TARGET-ONLY indices inside SUBSET)
# =========================================================
target_sub_ids = torch.tensor([old_tid_to_subid[tid] for tid in target_token_ids],
                              dtype=torch.long, device=device)
print("[target_sub_ids]:", tuple(target_sub_ids.shape))


# =========================================================
# 15) pos_weight (TARGET-ONLY)  (simple: ones)
# =========================================================
pos_weight = torch.ones((M_TGT,), dtype=torch.float32, device=device)


# =========================================================
# 16) Losses (Targets + SMILES CLIP)
# =========================================================
def info_nce_ranking_loss_multi_pos(
    v_pred: torch.Tensor,
    gene_emb: torch.Tensor,
    y_targets: torch.Tensor,
    num_neg: int = 256,
    num_pos: int = 8,
    tau: float = 0.1,
):
    device_ = v_pred.device
    B, _ = v_pred.shape
    losses = []

    v_pred = F.normalize(v_pred, dim=1)
    gene_emb = F.normalize(gene_emb, dim=1)

    for i in range(B):
        pos_idx = (y_targets[i] > 0.5).nonzero(as_tuple=True)[0]
        if pos_idx.numel() == 0:
            continue

        neg_idx_all = (y_targets[i] < 0.5).nonzero(as_tuple=True)[0]
        if neg_idx_all.numel() == 0:
            continue

        if num_pos and pos_idx.numel() > num_pos:
            pos_idx = pos_idx[torch.randperm(pos_idx.numel(), device=device_)[:num_pos]]

        if neg_idx_all.numel() > num_neg:
            neg_idx = neg_idx_all[torch.randperm(neg_idx_all.numel(), device=device_)[:num_neg]]
        else:
            neg_idx = neg_idx_all

        pos_emb = gene_emb[pos_idx]
        neg_emb = gene_emb[neg_idx]
        cand_emb = torch.cat([pos_emb, neg_emb], dim=0)

        v = v_pred[i].unsqueeze(0)
        scores = (v @ cand_emb.T).squeeze(0) / tau

        P = pos_emb.size(0)
        logits = scores.unsqueeze(0).repeat(P, 1)
        targets = torch.arange(P, device=device_, dtype=torch.long)
        losses.append(F.cross_entropy(logits, targets))

    if len(losses) == 0:
        return torch.tensor(0.0, device=device_)
    return torch.stack(losses).mean()


def bce_with_neg_sampling_cosine(
    pred_vec: torch.Tensor,
    y_targets: torch.Tensor,
    gene_emb: torch.Tensor,
    pos_weight_full: torch.Tensor,
    num_neg: int = 2048,
    pos_cap: int | None = None,
    tau_bce: float = 0.10,
):
    device_ = pred_vec.device
    B, _ = pred_vec.shape
    losses = []

    gene_emb = F.normalize(gene_emb, dim=1)

    for i in range(B):
        yi = y_targets[i]
        pos_idx = (yi > 0.5).nonzero(as_tuple=True)[0]
        if pos_idx.numel() == 0:
            continue

        if (pos_cap is not None) and (pos_idx.numel() > pos_cap):
            pos_idx = pos_idx[torch.randperm(pos_idx.numel(), device=device_)[:pos_cap]]

        neg_idx_all = (yi < 0.5).nonzero(as_tuple=True)[0]
        if neg_idx_all.numel() == 0:
            continue

        k = min(int(num_neg), neg_idx_all.numel())
        neg_idx = neg_idx_all[torch.randperm(neg_idx_all.numel(), device=device_)[:k]]

        idx = torch.cat([pos_idx, neg_idx], dim=0)

        v = F.normalize(pred_vec[i], dim=0)
        logits = (v @ gene_emb[idx].T) / tau_bce

        y_sub = yi[idx]
        pw_sub = pos_weight_full[idx]

        losses.append(F.binary_cross_entropy_with_logits(logits, y_sub, pos_weight=pw_sub, reduction="mean"))

    if len(losses) == 0:
        return torch.tensor(0.0, device=device_)
    return torch.stack(losses).mean()


def combined_target_loss_neg_sampling_tied(
    pred_vec: torch.Tensor,
    y_targets: torch.Tensor,
    gene_emb: torch.Tensor,
    pos_weight: torch.Tensor,
    lambda_cos: float = 1.0,
    lambda_bce: float = 0.1,
    lambda_rank: float = 0.5,
    bce_num_neg: int = 2048,
    bce_pos_cap: int | None = None,
    rank_num_neg: int = 256,
    rank_num_pos: int = 8,
    tau_rank: float = 0.1,
    tau_bce: float = 0.10,
):
    device_ = pred_vec.device

    gene_emb_norm = F.normalize(gene_emb, dim=1)
    pred_norm = F.normalize(pred_vec, dim=1)

    true_vec = y_targets @ gene_emb_norm
    num_t = y_targets.sum(dim=1, keepdim=True)
    mask = (num_t > 0).squeeze(1)

    if mask.any():
        true_vec_pos = true_vec[mask] / (num_t[mask] + 1e-6)
        true_vec_pos = F.normalize(true_vec_pos, dim=1)
        pred_pos = pred_norm[mask]
        loss_cos = 1.0 - (pred_pos * true_vec_pos).sum(dim=1).mean()
    else:
        loss_cos = torch.tensor(0.0, device=device_)

    loss_bce = bce_with_neg_sampling_cosine(
        pred_vec=pred_vec,
        y_targets=y_targets,
        gene_emb=gene_emb,
        pos_weight_full=pos_weight,
        num_neg=bce_num_neg,
        pos_cap=bce_pos_cap,
        tau_bce=tau_bce,
    )

    loss_rank = info_nce_ranking_loss_multi_pos(
        v_pred=pred_vec,
        gene_emb=gene_emb,
        y_targets=y_targets,
        num_neg=rank_num_neg,
        num_pos=rank_num_pos,
        tau=tau_rank,
    )

    loss = lambda_cos * loss_cos + lambda_bce * loss_bce + lambda_rank * loss_rank
    return loss, loss_cos.detach(), loss_bce.detach(), loss_rank.detach()


# --- SMILES CLIP loss (+ optional cosine align) ---
def clip_loss(z_pred: torch.Tensor, z_true: torch.Tensor, tau: torch.Tensor):
    z1 = F.normalize(z_pred, dim=1)
    z2 = F.normalize(z_true, dim=1)
    logits = (z1 @ z2.T) / tau
    labels = torch.arange(z_pred.size(0), device=z_pred.device, dtype=torch.long)
    return 0.5 * (F.cross_entropy(logits, labels) + F.cross_entropy(logits.T, labels))

def smiles_align_loss_cosine(z_pred: torch.Tensor, z_true: torch.Tensor):
    z1 = F.normalize(z_pred, dim=1)
    z2 = F.normalize(z_true, dim=1)
    return 1.0 - (z1 * z2).sum(dim=1).mean()


# =========================================================
# 17) Eval
# =========================================================
def compute_recall_precision_at_k(scores: torch.Tensor, y_true: torch.Tensor, k: int = 20):
    B, M = scores.shape
    kk = min(k, M)
    _, topk_idx = torch.topk(scores, k=kk, dim=1)

    recalls, precisions = [], []
    for i in range(B):
        true_labels = y_true[i]
        num_pos_ = true_labels.sum().item()
        if num_pos_ == 0:
            continue
        topk = topk_idx[i]
        num_pos_in_topk = true_labels[topk].sum().item()
        recalls.append(num_pos_in_topk / max(num_pos_, 1e-6))
        precisions.append(num_pos_in_topk / max(kk, 1))

    if len(recalls) == 0:
        return 0.0, 0.0
    return float(sum(recalls) / len(recalls)), float(sum(precisions) / len(precisions))

@torch.no_grad()
def smiles_retrieval_hitk(z_pred: torch.Tensor, drug_id: torch.Tensor, smiles_bank_t: torch.Tensor, k_list=(1,5,10)):
    z = F.normalize(z_pred.float(), dim=1)
    b = F.normalize(smiles_bank_t.float(), dim=1)
    logits = z @ b.T
    out = {}
    for k in k_list:
        topk = torch.topk(logits, k=min(k, logits.size(1)), dim=1).indices
        hit = (topk == drug_id.view(-1,1)).any(dim=1).float().mean().item()
        out[f"Hit@{k}"] = float(hit)
    true_vec = b[drug_id]
    out["TrueCos"] = float((z * true_vec).sum(dim=1).mean().item())
    return out

@torch.no_grad()
def evaluate_fp(model, loader, device, target_sub_ids, smiles_bank_t, k_list=(5,10), hitk=(1,5,10)):
    model.eval()

    gene_emb = model.gene_emb_subset()[target_sub_ids].to(device)
    g_norm = F.normalize(gene_emb, dim=1)

    recall_sums = {k: 0.0 for k in k_list}
    prec_sums   = {k: 0.0 for k in k_list}
    counts_     = {k: 0   for k in k_list}

    hit_sums = {f"Hit@{k}": 0.0 for k in hitk}
    hit_sums["TrueCos"] = 0.0
    clip_sum = 0.0
    tau_sum  = 0.0
    n = 0

    for batch in loader:
        input_ids = batch["input_ids"].to(device, non_blocking=True)
        values    = batch["values"].to(device, non_blocking=True)
        attn      = batch["attention_mask"].to(device, non_blocking=True)
        y_targets = batch["y_targets"].to(device, non_blocking=True)
        z_true    = batch["smiles_emb"].to(device, non_blocking=True)
        drug_id   = batch["drug_id"].to(device, non_blocking=True)
        organ_id  = batch["organ_id"].to(device, non_blocking=True)

        v_pred, z_pred = model(input_ids, values, attn, organ_id=organ_id, return_smiles=True)
        v_norm = F.normalize(v_pred, dim=1)
        scores = v_norm @ g_norm.T

        for k in k_list:
            r, p = compute_recall_precision_at_k(scores, y_targets, k=k)
            recall_sums[k] += r
            prec_sums[k]   += p
            counts_[k]     += 1

        bs = input_ids.size(0)
        m = smiles_retrieval_hitk(z_pred, drug_id, smiles_bank_t, k_list=hitk)
        for k in hitk:
            hit_sums[f"Hit@{k}"] += m[f"Hit@{k}"] * bs
        hit_sums["TrueCos"] += m["TrueCos"] * bs

        tau = model.get_tau()
        clip_sum += float(clip_loss(z_pred, z_true, tau=tau).item()) * bs
        tau_sum  += float(tau.item()) * bs

        n += bs

    out = {}
    for k in k_list:
        out[f"Recall@{k}"] = recall_sums[k] / max(counts_[k], 1)
        out[f"Precision@{k}"] = prec_sums[k] / max(counts_[k], 1)

    for k in hitk:
        out[f"SMILES_Hit@{k}"] = hit_sums[f"Hit@{k}"] / max(n, 1)
    out["SMILES_TrueCos"] = hit_sums["TrueCos"] / max(n, 1)
    out["SMILES_CLIP"] = clip_sum / max(n, 1)
    out["tau"] = tau_sum / max(n, 1)
    return out


# =========================================================
# 18) Train loop (OneCycleLR fixed + grad accumulation + AMP)
# =========================================================
def infinite_loader(loader):
    while True:
        for b in loader:
            yield b

USE_AMP = (device.type == "cuda")
scaler = torch.amp.GradScaler("cuda", enabled=USE_AMP)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

updates_per_epoch = max(1, STEPS_PER_EPOCH // max(1, ACCUM_STEPS))
total_updates = EPOCHS * updates_per_epoch

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=LR,
    total_steps=total_updates,
    pct_start=0.05,
    anneal_strategy="cos",
    div_factor=10.0,
    final_div_factor=100.0,
)

smiles_bank_t = torch.tensor(smiles_bank_np, dtype=torch.float32, device=device)

def train_one_epoch_fixed_steps(
    model,
    train_loader,
    device,
    steps_per_epoch,
    optimizer,
    scheduler,
    scaler,
    target_sub_ids,
    pos_weight,
    smiles_bank_t,
    log_every=50,
    grad_clip=1.0,
    accum_steps=1,
):
    model.train()
    it = infinite_loader(train_loader)

    run_total = 0.0
    run_tgt   = 0.0
    run_clip  = 0.0
    run_align = 0.0
    run_rank_last = 0.0

    run_hit5 = 0.0
    run_truecos = 0.0
    n = 0

    optimizer.zero_grad(set_to_none=True)
    update_count = 0

    pbar = tqdm(range(1, steps_per_epoch + 1), desc="Train", leave=True, dynamic_ncols=True)

    for step in pbar:
        batch = next(it)

        input_ids = batch["input_ids"].to(device, non_blocking=True)
        values    = batch["values"].to(device, non_blocking=True)
        attn      = batch["attention_mask"].to(device, non_blocking=True)
        y_targets = batch["y_targets"].to(device, non_blocking=True)
        z_true    = batch["smiles_emb"].to(device, non_blocking=True)
        drug_id   = batch["drug_id"].to(device, non_blocking=True)
        organ_id  = batch["organ_id"].to(device, non_blocking=True)

        bs = input_ids.size(0)
        n += bs

        if USE_AMP:
            with torch.amp.autocast("cuda", enabled=True):
                v_pred, z_pred = model(input_ids, values, attn, organ_id=organ_id, return_smiles=True)

                # --- Targets ---
                gene_emb = model.gene_emb_subset()[target_sub_ids]  # (M_TGT, d)
                loss_targets, loss_cos_t, loss_bce_t, loss_rank_t = combined_target_loss_neg_sampling_tied(
                    pred_vec=v_pred,
                    y_targets=y_targets,
                    gene_emb=gene_emb,
                    pos_weight=pos_weight,
                    lambda_cos=lambda_cos,
                    lambda_bce=lambda_bce,
                    lambda_rank=lambda_rank,
                    bce_num_neg=bce_num_neg,
                    bce_pos_cap=bce_pos_cap,
                    rank_num_neg=rank_num_neg,
                    rank_num_pos=rank_num_pos,
                    tau_rank=tau_rank,
                    tau_bce=tau_bce,
                )

                # --- SMILES CLIP ---
                tau = model.get_tau()
                loss_c = clip_loss(z_pred, z_true, tau=tau)
                loss_a = smiles_align_loss_cosine(z_pred, z_true)
                loss_smiles = loss_c + alpha_align * loss_a

                loss = (lambda_targets * loss_targets + lambda_smiles * loss_smiles) / float(accum_steps)

            if not torch.isfinite(loss).all():
                optimizer.zero_grad(set_to_none=True)
                continue

            scaler.scale(loss).backward()

            do_update = (step % accum_steps) == 0
            if do_update:
                scaler.unscale_(optimizer)
                if grad_clip is not None and grad_clip > 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)

                scheduler.step()
                update_count += 1

        else:
            v_pred, z_pred = model(input_ids, values, attn, organ_id=organ_id, return_smiles=True)

            gene_emb = model.gene_emb_subset()[target_sub_ids]
            loss_targets, loss_cos_t, loss_bce_t, loss_rank_t = combined_target_loss_neg_sampling_tied(
                pred_vec=v_pred,
                y_targets=y_targets,
                gene_emb=gene_emb,
                pos_weight=pos_weight,
                lambda_cos=lambda_cos,
                lambda_bce=lambda_bce,
                lambda_rank=lambda_rank,
                bce_num_neg=bce_num_neg,
                bce_pos_cap=bce_pos_cap,
                rank_num_neg=rank_num_neg,
                rank_num_pos=rank_num_pos,
                tau_rank=tau_rank,
                tau_bce=tau_bce,
            )

            tau = model.get_tau()
            loss_c = clip_loss(z_pred, z_true, tau=tau)
            loss_a = smiles_align_loss_cosine(z_pred, z_true)
            loss_smiles = loss_c + alpha_align * loss_a

            loss = (lambda_targets * loss_targets + lambda_smiles * loss_smiles) / float(accum_steps)

            if not torch.isfinite(loss).all():
                optimizer.zero_grad(set_to_none=True)
                continue

            loss.backward()

            do_update = (step % accum_steps) == 0
            if do_update:
                if grad_clip is not None and grad_clip > 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)

                scheduler.step()
                update_count += 1

        # logging metrics (cheap)
        with torch.no_grad():
            m = smiles_retrieval_hitk(z_pred, drug_id, smiles_bank_t, k_list=(5,))
            run_hit5 += float(m["Hit@5"]) * bs
            run_truecos += float(m["TrueCos"]) * bs

        run_total += float((lambda_targets * loss_targets + lambda_smiles * loss_smiles).item()) * bs
        run_tgt   += float(loss_targets.item()) * bs
        run_clip  += float(loss_c.item()) * bs
        run_align += float(loss_a.item()) * bs
        run_rank_last = float(loss_rank_t.item())

        if step % log_every == 0:
            lr_now = optimizer.param_groups[0]["lr"]
            pbar.set_postfix({
                "lr": f"{lr_now:.2e}",
                "tau": f"{float(model.get_tau().item()):.3f}",
                "tot": f"{run_total/max(n,1):.4f}",
                "tgt": f"{run_tgt/max(n,1):.4f}",
                "clip": f"{run_clip/max(n,1):.4f}",
                "align": f"{run_align/max(n,1):.4f}",
                "rank(last)": f"{run_rank_last:.4f}",
                "Hit@5": f"{run_hit5/max(n,1):.3f}",
                "TrueCos": f"{run_truecos/max(n,1):.3f}",
                "upd": f"{update_count}/{updates_per_epoch}",
            })

    return {
        "train_total": run_total / max(n,1),
        "train_tgt": run_tgt / max(n,1),
        "train_clip": run_clip / max(n,1),
        "train_align": run_align / max(n,1),
        "train_hit5": run_hit5 / max(n,1),
        "train_truecos": run_truecos / max(n,1),
        "rank_last": run_rank_last,
        "tau": float(model.get_tau().item()),
        "lr": optimizer.param_groups[0]["lr"],
        "updates": update_count,
    }


# =========================================================
# 19) TRAIN
# =========================================================
print(">>> TRAIN START: FP(TARGET-ONLY) + ORGAN + SMILES CLIP | Variant A(log1p->delta->clip) | NO pos_emb")

for epoch in range(1, EPOCHS + 1):
    logs = train_one_epoch_fixed_steps(
        model=model,
        train_loader=train_loader,
        device=device,
        steps_per_epoch=STEPS_PER_EPOCH,
        optimizer=optimizer,
        scheduler=scheduler,
        scaler=scaler,
        target_sub_ids=target_sub_ids,
        pos_weight=pos_weight,
        smiles_bank_t=smiles_bank_t,
        log_every=50,
        grad_clip=MAX_GRAD_NORM,
        accum_steps=max(1, int(ACCUM_STEPS)),
    )

    print(
        f"\n[Epoch {epoch}/{EPOCHS}] "
        f"lr={logs['lr']:.2e} | tau={logs['tau']:.3f} | "
        f"train_total={logs['train_total']:.4f} | "
        f"train_tgt={logs['train_tgt']:.4f} | "
        f"train_clip={logs['train_clip']:.4f} | "
        f"train_align={logs['train_align']:.4f} | "
        f"rank_last={logs['rank_last']:.4f} | "
        f"Hit@5={logs['train_hit5']:.3f} | TrueCos={logs['train_truecos']:.3f}"
    )

    valid = evaluate_fp(
        model=model,
        loader=val_loader,
        device=device,
        target_sub_ids=target_sub_ids,
        smiles_bank_t=smiles_bank_t,
        k_list=(5,10),
        hitk=(1,5,10),
    )
    print("✅ VALID:", valid)

print(">>> DONE")


[targets] drugs total=379, with>=1 target=264
[HVG] token_ids: 4000
[subset] HVG ∪ TARGETS size: 4184
[target-only] size: 278
[vocab] VOCAB_SIZE: 4188 | N_SPECIAL: 4
[targets] drugs with>=1 target vec: 264
[organ] NUM_ORGANS: 16 | mapped cell_lines: 102 | UNK_ORGAN_ID: 0
[SMILES] missing=0/379 | zero_vec=2
[SMILES] bank: (379, 768)
[baseline] global: (62713,) | by_cl: 50
[split] train pairs: 10505 | val pairs: 1168
[parquet] files: 3388


Index parquet row-groups: 100%|██████████| 3388/3388 [13:31<00:00,  4.18it/s]


[parquet] indexed pairs: 11673
✅ token_emb loaded: 4184/4184
[target_sub_ids]: (278,)
>>> TRAIN START: FP(TARGET-ONLY) + ORGAN + SMILES CLIP | Variant A(log1p->delta->clip) | NO pos_emb


Train: 100%|██████████| 7000/7000 [2:47:20<00:00,  1.43s/it, lr=1.00e-04, tau=0.106, tot=6.4425, tgt=6.1915, clip=4.8591, align=0.3241, rank(last)=5.1235, Hit@5=0.018, TrueCos=0.676, upd=1750/1750]  



[Epoch 1/20] lr=1.00e-04 | tau=0.106 | train_total=6.4425 | train_tgt=6.1915 | train_clip=4.8591 | train_align=0.3241 | rank_last=5.1235 | Hit@5=0.018 | TrueCos=0.676


...


[Epoch 15/20] lr=1.62e-05 | tau=0.025 | train_total=5.0146 | train_tgt=4.8151 | train_clip=3.6979 | train_align=0.5870 | rank_last=4.3114 | Hit@5=0.291 | TrueCos=0.413
✅ VALID: {'Recall@5': 0.2933345646685491, 'Precision@5': 0.08872916666666665, 'Recall@10': 0.40821422371031785, 'Precision@10': 0.06409375000000002, 'SMILES_Hit@1': 0.127265625, 'SMILES_Hit@5': 0.267734375, 'SMILES_Hit@10': 0.340703125, 'SMILES_TrueCos': 0.4091666708389918, 'SMILES_CLIP': 3.799344880580902, 'tau': 0.024924729019403458}


Train: 100%|██████████| 7000/7000 [2:27:37<00:00,  1.27s/it, lr=1.06e-05, tau=0.024, tot=5.0031, tgt=4.8044, clip=3.6800, align=0.5901, rank(last)=4.2728, Hit@5=0.295, TrueCos=0.410, upd=1750/1750]  



[Epoch 16/20] lr=1.06e-05 | tau=0.024 | train_total=5.0031 | train_tgt=4.8044 | train_clip=3.6800 | train_align=0.5901 | rank_last=4.2728 | Hit@5=0.295 | TrueCos=0.410
✅ VALID: {'Recall@5': 0.29402793040293046, 'Precision@5': 0.0888958333333333, 'Recall@10': 0.41049326382529516, 'Precision@10': 0.06436718749999999, 'SMILES_Hit@1': 0.12734375, 'SMILES_Hit@5': 0.2702864583333333, 'SMILES_Hit@10': 0.34197916666666667, 'SMILES_TrueCos': 0.406905122200648, 'SMILES_CLIP': 3.781051870981852, 'tau': 0.02439727634191513}


Train:  29%|██▉       | 2032/7000 [1:01:00<2:29:09,  1.80s/it, lr=9.22e-06, tau=0.024, tot=4.9950, tgt=4.7968, clip=3.6670, align=0.5911, rank(last)=4.3602, Hit@5=0.297, TrueCos=0.409, upd=500/1750]
Traceback (most recent call last):
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
Traceback (most recent call last):
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
Traceback (most recent call last):
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/data/aiffel/miniconda3/envs/babayakga/lib/python3.10/multiprocessing/util.py", line 300, in _run_finalizers
    finaliz

KeyboardInterrupt: 

In [2]:
import os, time
import random
import numpy as np
import torch

# =========================
# CONFIG: куда сохранять
# =========================
CKPT_DIR  = "/data/aiffel/babayakga/checkpoints/f_p_final"          # поменяй при желании
CKPT_NAME = "fp_smalltargets.pt"  # имя файла
ckpt_path = os.path.join(CKPT_DIR, CKPT_NAME)
os.makedirs(CKPT_DIR, exist_ok=True)

# =========================
# 1) RNG states (детерминизм)
# =========================
def get_rng_state_bundle():
    out = {}
    # python random
    try:
        out["python_random_state"] = random.getstate()
    except Exception as e:
        out["python_random_state"] = None
        out["python_random_state_err"] = repr(e)

    # numpy
    try:
        out["numpy_random_state"] = np.random.get_state()
    except Exception as e:
        out["numpy_random_state"] = None
        out["numpy_random_state_err"] = repr(e)

    # torch cpu
    try:
        out["torch_rng_state"] = torch.get_rng_state()
    except Exception as e:
        out["torch_rng_state"] = None
        out["torch_rng_state_err"] = repr(e)

    # torch cuda (все девайсы)
    try:
        if torch.cuda.is_available():
            out["torch_cuda_rng_state_all"] = torch.cuda.get_rng_state_all()
        else:
            out["torch_cuda_rng_state_all"] = None
    except Exception as e:
        out["torch_cuda_rng_state_all"] = None
        out["torch_cuda_rng_state_all_err"] = repr(e)

    return out

rng_bundle = get_rng_state_bundle()

# =========================
# 2) EXTRA (вшиваем LUT прямо в .pt)
# =========================
EXTRA = {
    "SPECIAL_TOKENS": SPECIAL_TOKENS,
    "N_SPECIAL": int(N_SPECIAL),
    "VOCAB_SIZE": int(VOCAB_SIZE),
    "PAD_ID": int(PAD_ID),
    "CLS_ID": int(CLS_ID),
    "ORGAN_TOK_ID": int(ORGAN_TOK_ID),

    "subset_token_ids": list(map(int, subset_token_ids)),
    "target_token_ids": list(map(int, target_token_ids)),

    "UNK_ORGAN_ID": int(UNK_ORGAN_ID),
    "organ2id": organ2id,
    "NUM_ORGANS": int(NUM_ORGANS),

    "D_MODEL": int(D_MODEL),
    "N_HEADS": int(N_HEADS),
    "N_LAYERS": int(N_LAYERS),
    "MAX_SEQ_LEN": int(MAX_SEQ_LEN),
    "SMILES_DIM": int(SMILES_DIM),

    "SEED": int(SEED),
    "CONTROL_DRUG": str(CONTROL_DRUG),
    "HVG_K": int(HVG_K),

    "USE_LOG1P_EXPR": bool(USE_LOG1P_EXPR),
    "USE_ASINH_DELTA": bool(USE_ASINH_DELTA),
    "DELTA_CLIP_ABS": float(DELTA_CLIP_ABS),
    "DROP_FIRST_GENE_TOKEN": bool(DROP_FIRST_GENE_TOKEN),
}

# LUT обычно небольшой — вшиваем внутрь .pt
# (делаем CPU tensor int64 для надёжности)
if "old_tid_to_vocab_lut" in globals() and isinstance(old_tid_to_vocab_lut, np.ndarray):
    lut_tensor = torch.from_numpy(old_tid_to_vocab_lut.astype(np.int64, copy=False)).cpu()
elif "old_tid_to_vocab_lut" in globals() and torch.is_tensor(old_tid_to_vocab_lut):
    lut_tensor = old_tid_to_vocab_lut.detach().to(dtype=torch.int64, device="cpu")
else:
    lut_tensor = None

# =========================
# 3) PAYLOAD
# =========================
payload = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "model_class": model.__class__.__name__,
    "model_state": model.state_dict(),

    "optimizer_state": optimizer.state_dict() if "optimizer" in globals() and optimizer is not None else None,
    "scheduler_state": scheduler.state_dict() if "scheduler" in globals() and scheduler is not None else None,
    "scaler_state": scaler.state_dict() if "scaler" in globals() and scaler is not None else None,

    # если есть последние метрики в памяти
    "metrics": {"valid": valid} if "valid" in globals() else {},

    "extra": EXTRA,

    # ✅ LUT inside checkpoint
    "old_tid_to_vocab_lut": lut_tensor,

    # ✅ RNG states
    "rng_state": rng_bundle,
}

# =========================
# 4) ATOMIC SAVE
# =========================
tmp_path = ckpt_path + ".tmp"
torch.save(payload, tmp_path)
os.replace(tmp_path, ckpt_path)

print(f"✅ Saved FULL checkpoint (model+opt+sched+scaler+RNG+LUT): {ckpt_path}")
print(f"   - LUT saved: {lut_tensor is not None}")
print(f"   - optimizer saved: {payload['optimizer_state'] is not None}")
print(f"   - scheduler saved: {payload['scheduler_state'] is not None}")
print(f"   - scaler saved: {payload['scaler_state'] is not None}")
print(f"   - cuda rng saved: {payload['rng_state'].get('torch_cuda_rng_state_all') is not None}")


✅ Saved FULL checkpoint (model+opt+sched+scaler+RNG+LUT): /data/aiffel/babayakga/checkpoints/f_p_final/fp_smalltargets.pt
   - LUT saved: True
   - optimizer saved: True
   - scheduler saved: True
   - scaler saved: True
   - cuda rng saved: True


In [10]:
# ============================================================
# LOAD + VERIFY checkpoint (PyTorch 2.6+ compatible)
# - fixes UnpicklingError by using weights_only=False (trusted ckpt)
# - prints EXTRA + LUT sanity
# - optionally restores RNG
# - rebuilds model skeleton + loads weights
# - optionally restores optimizer/scaler
# - runs 1 dummy forward pass


CKPT_PATH = "/data/aiffel/babayakga/checkpoints/f_p_final/fp_smalltargets.pt"  # <- your path
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")


# -------------------------
# 0) helper: pretty print
# -------------------------
def _pp(d, keys):
    for k in keys:
        print(f"{k}: {d.get(k, None)}")


# -------------------------
# 1) load checkpoint
#    ✅ PyTorch 2.6 changed default: weights_only=True
#    Your ckpt contains numpy RNG state -> needs trusted load.
# -------------------------
ckpt = torch.load(CKPT_PATH, map_location="cpu", weights_only=False)
print("✅ loaded:", CKPT_PATH)
print("top-level keys:", sorted(list(ckpt.keys())))

required = ["model_state", "extra", "rng_state"]
for k in required:
    assert k in ckpt, f"Missing key in checkpoint: {k}"

extra = ckpt["extra"]

print("\n--- EXTRA (summary) ---")
_pp(extra, [
    "VOCAB_SIZE", "N_SPECIAL", "PAD_ID", "CLS_ID",
    "ORGAN_TOK_ID", "NUM_ORGANS", "UNK_ORGAN_ID",
    "D_MODEL", "N_HEADS", "N_LAYERS", "MAX_SEQ_LEN",
    "SMILES_DIM", "SEED", "CONTROL_DRUG", "HVG_K",
    "USE_LOG1P_EXPR", "USE_ASINH_DELTA", "DELTA_CLIP_ABS",
    "DROP_FIRST_GENE_TOKEN",
])
print("len(subset_token_ids) =", len(extra.get("subset_token_ids", [])))
print("len(target_token_ids) =", len(extra.get("target_token_ids", [])))


# -------------------------
# LUT sanity
# -------------------------
lut = ckpt.get("old_tid_to_vocab_lut", None)
print("\n--- LUT ---")
if lut is None:
    print("❌ LUT is None (not saved)")
else:
    print("✅ LUT:", type(lut), "shape=", tuple(lut.shape), "dtype=", lut.dtype, "device=", lut.device)
    lut_np = lut.cpu().numpy()
    n_special = int(extra["N_SPECIAL"])
    bad = np.logical_and(lut_np != -1, lut_np < n_special).sum()
    print("bad LUT entries (!= -1 but < N_SPECIAL):", int(bad))


# -------------------------
# 2) restore RNG (optional, for exact resume)
# -------------------------
def restore_rng(rng_state: dict):
    # python
    if rng_state.get("python_random_state", None) is not None:
        random.setstate(rng_state["python_random_state"])
    # numpy
    if rng_state.get("numpy_random_state", None) is not None:
        np.random.set_state(rng_state["numpy_random_state"])
    # torch cpu
    if rng_state.get("torch_rng_state", None) is not None:
        torch.set_rng_state(rng_state["torch_rng_state"])
    # torch cuda
    if torch.cuda.is_available() and rng_state.get("torch_cuda_rng_state_all", None) is not None:
        torch.cuda.set_rng_state_all(rng_state["torch_cuda_rng_state_all"])

restore_rng(ckpt["rng_state"])
print("\n✅ RNG restored from checkpoint")


# -------------------------
# 3) rebuild model from EXTRA and load weights
# IMPORTANT:
#   You must have your model class definition available in runtime.
#   Example: FPModelTied_OrganCLIP must be defined ABOVE this cell/script.
# -------------------------
model_class_name = ckpt.get("model_class", "UNKNOWN")
print("\nmodel_class saved in ckpt:", model_class_name)

# === CHANGE THIS if your saved class differs ===
ModelClass = FPModelTied_OrganCLIP  # <-- must exist in your runtime

VOCAB_SIZE  = int(extra["VOCAB_SIZE"])
PAD_ID      = int(extra["PAD_ID"])
N_SPECIAL   = int(extra["N_SPECIAL"])
NUM_ORGANS  = int(extra["NUM_ORGANS"])
D_MODEL     = int(extra["D_MODEL"])
N_HEADS     = int(extra["N_HEADS"])
N_LAYERS    = int(extra["N_LAYERS"])
MAX_SEQ_LEN = int(extra["MAX_SEQ_LEN"])
SMILES_DIM  = int(extra["SMILES_DIM"])

model1 = ModelClass(
    vocab_size=VOCAB_SIZE,
    d_model=D_MODEL,
    n_heads=N_HEADS,
    num_layers=N_LAYERS,
    pad_id=PAD_ID,
    smiles_dim=SMILES_DIM,
    max_len=(2 + MAX_SEQ_LEN),
    num_organs=NUM_ORGANS,
    n_special=N_SPECIAL,
 
).to(device)

missing, unexpected = model1.load_state_dict(ckpt["model_state"], strict=False)
print("\n--- load_state_dict ---")
print("missing keys:", missing)
print("unexpected keys:", unexpected)
assert len(unexpected) == 0, "Unexpected keys => model definition mismatch"
print("✅ model weights loaded")

model1.eval()


# -------------------------
# 4) restore optimizer/scheduler/scaler (optional)
# -------------------------
optimizer = None
scheduler = None
scaler = None

if ckpt.get("optimizer_state", None) is not None:
    optimizer = torch.optim.AdamW(model1.parameters(), lr=1e-4)  # lr can be dummy
    optimizer.load_state_dict(ckpt["optimizer_state"])
    print("✅ optimizer restored")

if ckpt.get("scheduler_state", None) is not None and optimizer is not None:
    print("⚠️ scheduler_state exists, but scheduler recreation is user-specific. Skipping by default.")

if ckpt.get("scaler_state", None) is not None and device.type == "cuda":
    scaler = torch.amp.GradScaler("cuda", enabled=True)
    scaler.load_state_dict(ckpt["scaler_state"])
    print("✅ scaler restored")


# -------------------------
# 5) quick forward sanity test (no dataloader needed)
# -------------------------
B = 2
L = 2 + MAX_SEQ_LEN

dummy_input_ids = torch.full((B, L), int(extra["PAD_ID"]), dtype=torch.long, device=device)
dummy_values    = torch.zeros((B, L), dtype=torch.float32, device=device)
dummy_attn      = torch.zeros((B, L), dtype=torch.long, device=device)

dummy_input_ids[:, 0] = int(extra["CLS_ID"])
dummy_input_ids[:, 1] = int(extra["ORGAN_TOK_ID"])
dummy_attn[:, :2] = 1

dummy_organ = torch.zeros((B,), dtype=torch.long, device=device)  # UNK organ id usually 0

with torch.no_grad():
    v_pred, z_pred = model1(dummy_input_ids, dummy_values, dummy_attn, organ_id=dummy_organ, return_smiles=True)

print("\n--- forward OK ---")
print("v_pred:", tuple(v_pred.shape), v_pred.dtype)
print("z_pred:", tuple(z_pred.shape), z_pred.dtype)
print("tau:", float(model1.get_tau().item()))


✅ loaded: /data/aiffel/babayakga/checkpoints/f_p_final/fp_smalltargets.pt
top-level keys: ['extra', 'metrics', 'model_class', 'model_state', 'old_tid_to_vocab_lut', 'optimizer_state', 'rng_state', 'scaler_state', 'scheduler_state', 'timestamp']

--- EXTRA (summary) ---
VOCAB_SIZE: 4188
N_SPECIAL: 4
PAD_ID: 0
CLS_ID: 1
ORGAN_TOK_ID: 2
NUM_ORGANS: 16
UNK_ORGAN_ID: 0
D_MODEL: 256
N_HEADS: 8
N_LAYERS: 4
MAX_SEQ_LEN: 256
SMILES_DIM: 768
SEED: 42
CONTROL_DRUG: DMSO_TF
HVG_K: 4000
USE_LOG1P_EXPR: True
USE_ASINH_DELTA: False
DELTA_CLIP_ABS: 5.0
DROP_FIRST_GENE_TOKEN: True
len(subset_token_ids) = 4184
len(target_token_ids) = 278

--- LUT ---
✅ LUT: <class 'torch.Tensor'> shape= (62713,) dtype= torch.int64 device= cpu
bad LUT entries (!= -1 but < N_SPECIAL): 0

✅ RNG restored from checkpoint

model_class saved in ckpt: FPModelTied_OrganCLIP

--- load_state_dict ---
missing keys: []
unexpected keys: []
✅ model weights loaded
✅ optimizer restored
⚠️ scheduler_state exists, but scheduler recreation

In [6]:
from typing import Dict, Tuple, Iterable, List
# =========================================================
# Ranking metrics for targets: mAP@K, NDCG@K, Coverage ceiling
# =========================================================

@torch.no_grad()
def average_precision_at_k(scores_1d: torch.Tensor, y_true_1d: torch.Tensor, k: int) -> float:
    """
    AP@K for ONE sample (binary relevance).
    scores_1d: (M,)
    y_true_1d: (M,) in {0,1}
    """
    M = scores_1d.numel()
    kk = min(int(k), int(M))
    if kk <= 0:
        return 0.0

    pos_total = float(y_true_1d.sum().item())
    if pos_total <= 0:
        return 0.0

    topk = torch.topk(scores_1d, k=kk, dim=0).indices
    rel = y_true_1d[topk].float()  # (kk,)

    # precision@i only when rel[i]=1
    cumsum_rel = torch.cumsum(rel, dim=0)
    ranks = torch.arange(1, kk + 1, device=scores_1d.device, dtype=torch.float32)
    precision_i = cumsum_rel / ranks

    ap = (precision_i * rel).sum() / max(1.0, min(pos_total, float(kk)))
    return float(ap.item())


@torch.no_grad()
def mean_average_precision_at_k(scores: torch.Tensor, y_true: torch.Tensor, k: int) -> float:
    """
    mAP@K over batch (binary relevance).
    scores: (B, M)
    y_true: (B, M) in {0,1}
    """
    B = scores.size(0)
    aps = []
    for i in range(B):
        aps.append(average_precision_at_k(scores[i], y_true[i], k))
    return float(sum(aps) / max(1, len(aps)))


@torch.no_grad()
def ndcg_at_k(scores_1d: torch.Tensor, y_true_1d: torch.Tensor, k: int) -> float:
    """
    NDCG@K for ONE sample (binary relevance).
    DCG = sum_{i=1..K} rel_i / log2(i+1)
    IDCG computed from sorted relevances (all ones first).
    """
    M = scores_1d.numel()
    kk = min(int(k), int(M))
    if kk <= 0:
        return 0.0

    pos_total = int(y_true_1d.sum().item())
    if pos_total <= 0:
        return 0.0

    topk = torch.topk(scores_1d, k=kk, dim=0).indices
    rel = y_true_1d[topk].float()  # (kk,)

    denom = torch.log2(torch.arange(2, kk + 2, device=scores_1d.device, dtype=torch.float32))
    dcg = (rel / denom).sum()

    ideal_k = min(pos_total, kk)
    ideal_rel = torch.ones((ideal_k,), device=scores_1d.device, dtype=torch.float32)
    idcg = (ideal_rel / denom[:ideal_k]).sum()

    return float((dcg / (idcg + 1e-12)).item())


@torch.no_grad()
def mean_ndcg_at_k(scores: torch.Tensor, y_true: torch.Tensor, k: int) -> float:
    B = scores.size(0)
    vals = []
    for i in range(B):
        vals.append(ndcg_at_k(scores[i], y_true[i], k))
    return float(sum(vals) / max(1, len(vals)))


@torch.no_grad()
def recall_precision_at_k(scores: torch.Tensor, y_true: torch.Tensor, k: int) -> Tuple[float, float]:
    """
    Same spirit as your compute_recall_precision_at_k, but vectorized-ish and robust.
    """
    B, M = scores.shape
    kk = min(int(k), int(M))
    if kk <= 0:
        return 0.0, 0.0

    topk = torch.topk(scores, k=kk, dim=1).indices  # (B, kk)
    rel = torch.gather(y_true, 1, topk).float()     # (B, kk)
    pos_total = y_true.sum(dim=1).float()           # (B,)

    mask = pos_total > 0
    if not mask.any():
        return 0.0, 0.0

    num_pos_in_topk = rel.sum(dim=1)  # (B,)
    recall = (num_pos_in_topk[mask] / (pos_total[mask] + 1e-12)).mean()
    precision = (num_pos_in_topk[mask] / float(kk)).mean()
    return float(recall.item()), float(precision.item())


@torch.no_grad()
def coverage_ceiling_recall_at_k(y_true: torch.Tensor, k: int) -> Dict[str, float]:
    """
    Coverage vs #targets:
    - avg #targets
    - fraction of samples with <=k targets
    - avg ceiling recall@k = min(k, #targets) / #targets
    - median #targets
    """
    k = int(k)
    num_t = y_true.sum(dim=1).float()  # (B,)
    mask = num_t > 0
    if not mask.any():
        return {
            "avg_targets": 0.0,
            "median_targets": 0.0,
            "frac_targets_le_k": 0.0,
            "avg_recall_ceiling": 0.0,
        }

    nt = num_t[mask]
    ceiling = torch.minimum(nt, torch.tensor(float(k), device=y_true.device)) / (nt + 1e-12)
    frac_le = (nt <= float(k)).float().mean()

    # median (torch median)
    median = nt.median()

    return {
        "avg_targets": float(nt.mean().item()),
        "median_targets": float(median.item()),
        "frac_targets_le_k": float(frac_le.item()),
        "avg_recall_ceiling": float(ceiling.mean().item()),
    }


# =========================================================
# Retrieval metrics for SMILES: MRR, median rank, plus Hit@K, TrueCos
# =========================================================

@torch.no_grad()
def smiles_retrieval_metrics(
    z_pred: torch.Tensor,
    drug_id: torch.Tensor,
    smiles_bank_t: torch.Tensor,
    k_list: Iterable[int] = (1, 5, 10),
) -> Dict[str, float]:
    """
    z_pred: (B, D) predicted SMILES vector
    drug_id: (B,) true drug index in bank
    smiles_bank_t: (N, D)
    Returns: Hit@K, TrueCos, MRR, median_rank, mean_rank
    """
    z = F.normalize(z_pred.float(), dim=1)
    b = F.normalize(smiles_bank_t.float(), dim=1)

    logits = z @ b.T  # (B, N)
    B, N = logits.shape

    # ranks: higher logits => better
    # rank = 1 + number of items with score > true_score (ties -> worst-ish; acceptable)
    true_scores = logits.gather(1, drug_id.view(-1, 1))  # (B,1)
    better = (logits > true_scores).sum(dim=1)           # (B,)
    rank = better + 1                                   # (B,) 1..N

    out = {}

    # Hit@K
    for k in k_list:
        k = min(int(k), N)
        topk = torch.topk(logits, k=k, dim=1).indices
        hit = (topk == drug_id.view(-1, 1)).any(dim=1).float().mean()
        out[f"Hit@{k}"] = float(hit.item())

    # TrueCos
    true_vec = b[drug_id]
    out["TrueCos"] = float((z * true_vec).sum(dim=1).mean().item())

    # MRR / ranks
    out["MRR"] = float((1.0 / rank.float()).mean().item())
    out["median_rank"] = float(rank.float().median().item())
    out["mean_rank"] = float(rank.float().mean().item())

    return out


# =========================================================
# Drop-in replacement: evaluate_fp + new metrics
# =========================================================

@torch.no_grad()
def evaluate_fp_with_ranking_and_retrieval(
    model,
    loader,
    device,
    target_sub_ids,
    smiles_bank_t,
    k_list_targets: Iterable[int] = (5, 10, 20),
    k_list_smiles: Iterable[int] = (1, 5, 10),
) -> Dict[str, float]:
    """
    Adds:
      - Targets: mAP@K, NDCG@K, Recall@K, Precision@K + Coverage ceiling stats
      - SMILES: Hit@K, TrueCos, MRR, median rank, mean rank, CLIP loss, tau
    """
    model.eval()

    gene_emb = model.gene_emb_subset()[target_sub_ids].to(device)  # (M_TGT, d)
    g_norm = F.normalize(gene_emb, dim=1)

    # accumulators
    out_sum = defaultdict(float)
    n_batches = 0
    n_samples = 0

    clip_sum = 0.0
    tau_sum = 0.0

    # coverage stats accum (per K)
    cov_sums = {k: defaultdict(float) for k in k_list_targets}
    cov_counts = {k: 0 for k in k_list_targets}

    for batch in loader:
        input_ids = batch["input_ids"].to(device, non_blocking=True)
        values    = batch["values"].to(device, non_blocking=True)
        attn      = batch["attention_mask"].to(device, non_blocking=True)
        y_targets = batch["y_targets"].to(device, non_blocking=True)  # (B, M_TGT)
        z_true    = batch["smiles_emb"].to(device, non_blocking=True)
        drug_id   = batch["drug_id"].to(device, non_blocking=True)
        organ_id  = batch["organ_id"].to(device, non_blocking=True)

        v_pred, z_pred = model(input_ids, values, attn, organ_id=organ_id, return_smiles=True)
        v_norm = F.normalize(v_pred, dim=1)
        scores = v_norm @ g_norm.T  # (B, M_TGT)

        B = scores.size(0)
        n_batches += 1
        n_samples += B

        # ---- Targets metrics ----
        for k in k_list_targets:
            r, p = recall_precision_at_k(scores, y_targets, k=k)
            ap = mean_average_precision_at_k(scores, y_targets, k=k)
            nd = mean_ndcg_at_k(scores, y_targets, k=k)

            out_sum[f"Recall@{k}"] += r
            out_sum[f"Precision@{k}"] += p
            out_sum[f"mAP@{k}"] += ap
            out_sum[f"NDCG@{k}"] += nd

            cov = coverage_ceiling_recall_at_k(y_targets, k=k)
            for kk, vv in cov.items():
                cov_sums[k][kk] += float(vv)
            cov_counts[k] += 1

        # ---- SMILES retrieval metrics ----
        m = smiles_retrieval_metrics(z_pred, drug_id, smiles_bank_t, k_list=k_list_smiles)
        for key, val in m.items():
            out_sum[f"SMILES_{key}"] += float(val) * B  # weight by batch size

        # ---- CLIP loss / tau ----
        tau = model.get_tau()
        clip_sum += float(clip_loss(z_pred, z_true, tau=tau).item()) * B
        tau_sum  += float(tau.item()) * B

    out = {}

    # Average batch-averaged target metrics
    for k in k_list_targets:
        out[f"Recall@{k}"] = out_sum[f"Recall@{k}"] / max(1, n_batches)
        out[f"Precision@{k}"] = out_sum[f"Precision@{k}"] / max(1, n_batches)
        out[f"mAP@{k}"] = out_sum[f"mAP@{k}"] / max(1, n_batches)
        out[f"NDCG@{k}"] = out_sum[f"NDCG@{k}"] / max(1, n_batches)

        # Coverage ceiling stats (averaged over batches)
        cc = cov_counts[k]
        if cc > 0:
            out[f"Coverage@{k}_avg_targets"] = cov_sums[k]["avg_targets"] / cc
            out[f"Coverage@{k}_median_targets"] = cov_sums[k]["median_targets"] / cc
            out[f"Coverage@{k}_frac_targets_le_k"] = cov_sums[k]["frac_targets_le_k"] / cc
            out[f"Coverage@{k}_avg_recall_ceiling"] = cov_sums[k]["avg_recall_ceiling"] / cc
            # optional: "normalized recall" = Recall@K / ceiling (if you want)
            ceil = out[f"Coverage@{k}_avg_recall_ceiling"]
            out[f"Recall@{k}_over_ceiling"] = out[f"Recall@{k}"] / max(ceil, 1e-9)

    # SMILES metrics averaged over samples (we weighted by B already)
    for key in ["Hit@1", "Hit@5", "Hit@10", "TrueCos", "MRR", "median_rank", "mean_rank"]:
        sk = f"SMILES_{key}"
        if sk in out_sum:
            out[sk] = out_sum[sk] / max(1, n_samples)

    out["SMILES_CLIP"] = clip_sum / max(1, n_samples)
    out["tau"] = tau_sum / max(1, n_samples)
    out["n_samples"] = float(n_samples)

    return out


valid = evaluate_fp_with_ranking_and_retrieval(
    model=model,
    loader=val_loader,
    device=device,
    target_sub_ids=target_sub_ids,
    smiles_bank_t=smiles_bank_t,
    k_list_targets=(5, 10, 20),
    k_list_smiles=(1, 5, 10),
)
print("✅ VALID:", valid)


✅ VALID: {'Recall@5': 0.2924789224068324, 'Precision@5': 0.08897396000723044, 'mAP@5': 0.19173023119471813, 'NDCG@5': 0.22381376816503082, 'Coverage@5_avg_targets': 2.095833333333333, 'Coverage@5_median_targets': 1.14, 'Coverage@5_frac_targets_le_k': 0.9517708333333333, 'Coverage@5_avg_recall_ceiling': 0.9826236150662104, 'Recall@5_over_ceiling': 0.2976510211258507, 'Recall@10': 0.4075360565384229, 'Precision@10': 0.06435156346609196, 'mAP@10': 0.21013024507517306, 'NDCG@10': 0.2646626206972481, 'Coverage@10_avg_targets': 2.095833333333333, 'Coverage@10_median_targets': 1.14, 'Coverage@10_frac_targets_le_k': 0.9886458333333333, 'Coverage@10_avg_recall_ceiling': 0.9976251810789108, 'Recall@10_over_ceiling': 0.4085061847553619, 'Recall@20': 0.5443653134504954, 'Precision@20': 0.0442695319528381, 'mAP@20': 0.2221575258799324, 'NDCG@20': 0.30403822676472675, 'Coverage@20_avg_targets': 2.095833333333333, 'Coverage@20_median_targets': 1.14, 'Coverage@20_frac_targets_le_k': 1.0, 'Coverage@20_